# Chapter 7 Appendix


## 7.1 Web Scraping Script for U.S. States and Washington, D.C.

In [ ]:


# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
#import PyPDF2
#import glob

driver_path = 'C:/Users/longy/OneDrive/Projects/AFRI/windows/chromedriver.exe' #for windows
# driver_path = '/Users/Yanxu/OneDrive/Projects/AgPolicyCoding/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
#os.chdir('/Users/long/OneDrive/Projects/AFRI')
os.chdir('C:/Users/longy/OneDrive/Projects/AFRI')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
# for mac
#dnldpath = {"download.default_directory" : "/Users/long/OneDrive/Projects/AFRI/data/SD/"}
# for windows
dnldpath = {"download.default_directory" :"C:/Users/longy/OneDrive/Projects/AFRI/data/SD/"}
chromeOptions.add_experimental_option("prefs",dnldpath)
driver = webdriver.Chrome(executable_path=driver_path, options=chromeOptions)

print("Current working directory: {0}".format(cwd))



urls=[]
sessionurls = []
driver.get("https://www.sdlegislature.gov/Session/Archived")
#sessions = driver.find_elements_by_link_text("Session Laws")
sessions = driver.find_elements_by_css_selector('tbody tr td:nth-child(7) a')

for session in sessions:
    sessionurl = session.get_attribute('href')
    sessionurls.append(sessionurl)

for sessionurl in sessionurls:
    driver.get(sessionurl)
    try:
        list = driver.find_elements_by_css_selector("tbody tr td:nth-child(1) span a")
        for row in list:
            url = row.get_attribute('href')
            urls.append(url)
    except:
        print(sessionurl)
print("urls are done")

acttxts = []
for url in urls:
    driver.get(url)
    acttxt = driver.find_element_by_css_selector("body").text
    acttxts.append(acttxt)
    sleep(1)
    #sleep(randint(1,2))
datasource = pd.DataFrame({
    'Link to full text':urls,
    'Full text': acttxts
})
datasource.to_excel('SD_Leginfo.xlsx')
datasource.to_csv('SD_Leginfo.csv')
datasource.to_pickle('SD_Leginfo.pkl')
datasource.to_json('SD_Leginfo.json')
# End of SD_Leginfo.py


# Start of IA_Leginfo.py

# import libraries
## selenium
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import inflect
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import fitz
import re

# set up working directory
working_directory = '/Volumes/SSD/AFRI/'
print(os.path.abspath(working_directory))

# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./Data/Raw_Data/IA/PDFs"

print(os.path.abspath(download_dir))


# Specify the path to the Firefox binary
firefox_binary_path = '/Applications/Firefox.app/Contents/MacOS/firefox'

# Create a Firefox profile and configure preferences
options = Options()
options.binary_location = firefox_binary_path

# Initialize the Firefox driver with the specified options
executable_path = './Data/geckodriver'
print(os.path.abspath(executable_path))

# Initialize the WebDriver with the specified path
service = Service(executable_path)
driver = webdriver.Firefox(service=service)


import subprocess

def get_mdls_where_from(file_path):
    try:
        result = subprocess.run(['mdls', '-name', 'kMDItemWhereFroms', '-raw', file_path],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            where_from_info = result.stdout.strip()
            if where_from_info:
                # The output might be a JSON-like list, so we print it directly
                return where_from_info
            else:
                print("No 'Where from' information available.")
                return None
        else:
            print(f"Error running mdls command: {result.stderr}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

def extract_text_from_pdf(file_path):
    try:
        # Open the PDF file
        pdf_document = fitz.open(file_path)
        text = ""

        # Iterate through each page and extract text
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            text += page.get_text()

        return text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


urls=[]
acttxts = []
session_nos = []
session_names = []
for session in range(1,166):

    session_url = "https://www.legis.iowa.gov/law/statutory/acts/actsChapter?ssid=" + str(session)

    # Navigate to the website
    driver.get(session_url)

    # Wait for the links to be loaded on the page
    timeout = 5  # Adjust the timeout value as needed

    try:
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.grid_9:nth-child(2) > h2:nth-child(1)')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        driver.quit()

    time.sleep(3)

    session_no = int(driver.find_element(By.CSS_SELECTOR, 'div.grid_9:nth-child(2) > h2:nth-child(1)').text.replace(' ', '').split('(')[0].replace('IowaActsChapters-', ''))

    session_name = driver.find_element(By.CSS_SELECTOR, 'div.grid_9:nth-child(2) > h2:nth-child(1)').text

    if 65 < session_no < 90:
        print(session_no)
        try:
            chapters = driver.find_elements(By.PARTIAL_LINK_TEXT,'Chapter')
            for chapter in chapters:
                print(chapter.text)
                url = chapter.get_attribute('href')
                urls.append(url)
                chapter.click()
                session_nos.append(session_no)
                session_names.append(session_name)
        except:
            print("no Chapter found")
            print(url)
    else:
        pass


folder_path = "./Data/Raw_Data/IA/PDFs"
list_of_files = glob.glob(os.path.join(folder_path, '*.pdf'))
#latest_file = max(list_of_files, key=os.path.getctime)
acttxts = []
urls = []
file_names = []
for idx,file in enumerate(list_of_files):
    print(idx)
    acttxt = extract_text_from_pdf(file)
    acttxts.append(acttxt)
    url = get_mdls_where_from(file).strip('()\n ').strip('"').strip('\n').replace(' ', '').replace('",\n', '')
    urls.append(url)

datasource = pd.DataFrame({
    'original_text': acttxts,
    'link': urlss,
    'file_name': list_of_files,
    'session_number': session_no,
    'session_name': session_names
})
s
datasource['year'] = datasource['session_name'].apply(lambda x: x.split('(')[1][:4])

datasource['original_act_num'] = datasource['file_name'].apply(lambda x: x.split('PDFs/')[1][:6])

datasource['act_num'] = "IA" + datasource['year'] + datasource['original_act_num']

datasource = datasource.assign(state='IA')

datasource.to_csv('./Data/Raw_Data/IA/IA_Leginfo.csv')
datasource.to_pickle('./Data/Raw_Data/IA/IA_Leginfo.pkl')
datasource.to_json('./Data/Raw_Data/IA/IA_Leginfo.json')
# End of IA_Leginfo.py


# Start of ME_Leginfo.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import fitz
from selenium.webdriver.chrome.options import Options

#driver_path = 'C:/Users/longy/OneDrive/Projects/AFRI/windows/chromedriver.exe' #for windows
driver_path = '/Users/long/OneDrive/Projects/AFRI/chromedriver'
driver_path = '/Volumes/SSD/AFRI/Data/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
os.chdir('/Volumes/SSD/AFRI/Data/Raw_Data/ME')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_directory = "/Volumes/SSD/AFRI/Data/Raw_Data/ME"

# Set Chrome options to automatically download files to the specified directory
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (without opening the browser window)
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_directory,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  # Open PDF files in an external PDF viewer
})

# Create the WebDriver instance
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)



# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import fitz
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import requests


#driver_path = 'C:/Users/longy/OneDrive/Projects/AFRI/windows/chromedriver.exe' #for windows
driver_path = '/Users/long/OneDrive/Projects/AFRI/chromedriver'
driver_path = '/Volumes/SSD/AFRI/Data/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
os.chdir('/Volumes/SSD/AFRI/Data/Raw_Data/ME')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "/Volumes/SSD/AFRI/Data/Raw_Data/ME"

# Create a Firefox profile and configure preferences
firefox_profile = webdriver.FirefoxProfile()

# Set the download directory
firefox_profile.set_preference("browser.download.folderList", 2)
firefox_profile.set_preference("browser.download.dir", download_dir)

# Disable the download dialog and set MIME types for automatic download
firefox_profile.set_preference("browser.download.manager.showWhenStarting", False)
firefox_profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")
firefox_profile.set_preference("browser.helperApps.neverAsk.openFile", "application/pdf")

# Disable Firefox's built-in PDF viewer
firefox_profile.set_preference("pdfjs.disabled", True)

# Start Firefox with the customized profile

driver = webdriver.Firefox(executable_path='/Volumes/SSD/AFRI/Data/geckodriver',firefox_profile=firefox_profile)


driver.get('http://lldc.mainelegislature.org/Open/Laws/')

sessions = ['1975', '1977', '1979', '1981', '1983', '1985', '1987', '1989', '1991', '1993', '1995', '1997', '1999', '2001', '2003', '2005', '2007', '2009', '2011', '2013', '2015', '2017', '2019', '2021']

urls = []
for session in sessions:
    driver.get('http://lldc.mainelegislature.org/Open/Laws/')
    driver.find_element(By.PARTIAL_LINK_TEXT, session).click()
    pdfs = driver.find_elements(By.PARTIAL_LINK_TEXT, session)
    for pdf in pdfs:
        pdf.get_attribute('href')
        url = pdf.get_attribute('href')
        if "_PL_" in url:
            urls.append(url)
        else:
            pass
        urls.append(url)


bad_act_urls = []

path = "/Volumes/SSD/AFRI/Data/Raw_Data/ME/PDFs"

for url in urls:
    response = requests.get(url)
    # Ensure the request was successful
    if response.status_code == 200:
        # Write the content of the response to a file
        with open(path, 'wb') as file:
            file.write(response.content)
    else:
        print("Failed to fetch PDF. Status code:", response.status_code)
        bad_act_urls.append(url)




act_txts = []
pgtxts = []

path = "/Volumes/SSD/AFRI/Data/Raw_Data/ME/PDFs"
files = glob.glob(os.path.join(path, '*.pdf'))

for file in files:
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        acttxt = ' '.join(pgtxts)
    act_txts.append(acttxt)
    doc.close()



datasource = pd.DataFrame({
    'links':urls,
    'original_text': act_txts,
    'file_name': files
})

datasource.to_excel('ME_Leginfo.xlsx')
datasource.to_csv('ME_Leginfo.csv')
datasource.to_pickle('ME_Leginfo.pkl')
datasource.to_json('ME_Leginfo.json')


# End of ME_Leginfo.py


# Start of IL_Leginfo.py

# import libraries
## selenium
import selenium

print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
## others
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
import fitz
import re
from bs4 import BeautifulSoup
from datetime import datetime

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/IL'
print(os.path.abspath(working_directory))


# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Initialize the Firefox driver with the specified options
executable_path = '../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service=service)

years = []
subtitles = []
bill_nums = []
filenames = []
act_urls = []
sessions = []
broken_act_urls = []
act_txts = []
bill_status_urls = []
bill_status_pages = []
effective_dates = []
orginal_act_nums = []

for num in range(90, 103):
    if num < 93:
        print(num)
        for act_num in range(1, 1300):
            print(act_num)
            start_year = 1817 + num * 2
            end_year = 1818 + num * 2
            session = f"({start_year}-{end_year})"
            sessions.append(session)

            url = 'https://www.ilga.gov/legislation/publicacts/pubact' + str(num) + "/acts/" + str(num) + "-" + str(
                act_num).zfill(4) + ".html"

            act_urls.append(url)

            try:
                # Navigate to the webpage
                driver.get(url)
            except:
                break

            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
            )
            act_txts.append(driver.find_element(By.CSS_SELECTOR, "body").text)

            try:
                bill_status.append(driver.find_element(By.LINK_TEXT, "Bill Status").get_attribute('href'))
            except:
                bill_status.append('NA')

            try:
                act_pdf_urls.append(driver.find_element(By.LINK_TEXT, "PDF").get_attribute('href'))
            except:
                act_pdf_urls.append('NA')

    else:
        print(num)
        for act_num in range(1, 1300):
            print(act_num)
            start_year = 1817 + num * 2
            end_year = 1818 + num * 2
            session = f"({start_year}-{end_year})"
            sessions.append(session)

            url = 'https://www.ilga.gov/legislation/publicacts/fulltext.asp?Name=' + str(num).zfill(3) + "-" + str(
                act_num).zfill(4) + "&GA=" + str(num)

            act_urls.append(url)

            try:
                # Navigate to the webpage
                driver.get(url)
            except:
                break

            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
            )
            act_txts.append(driver.find_element(By.CSS_SELECTOR, "body").text)

            try:
                bill_status_urls.append(driver.find_element(By.LINK_TEXT, "Bill Status").get_attribute('href'))
            except:
                bill_status_urls.append('NA')

            try:
                act_pdf_urls.append(driver.find_element(By.LINK_TEXT, "PDF").get_attribute('href'))
            except:
                act_pdf_urls.append('NA')

datasource = pd.DataFrame({
    'session': sessions,
    'original_text': act_txts,
    'link': act_urls,
    'bill_status_urls': bill_status_urls,
    'pdf_url': act_pdf_urls,
    'state': 'IL',
    'original_text': act_txts
})



datasource = datasource['original_text'].str.replace['Translate Website\n\n\n  Home     Legislation & Laws     Senate     House     My Legislation     Site Map    \n\nPrevious General Assemblies\n\n','']

datasource['original_text'] = datasource['original_text'].str.replace('Translate Website\n\n\n  Home     Legislation & Laws     Senate     House     My Legislation     Site Map    \n\nPrevious General Assemblies\n\n','',regex=False)

datasource['original_text'] = datasource['original_text'].str.replace('State of Illinois\nPublic Acts\n90th General Assembly\n[ Home ] [ Public Acts ] [ ILCS ] [ Search ] [ Bottom ]\n','',regex=False)

datasource['original_text'] = datasource['original_text'].str.replace('State of Illinois\n91st General Assembly\nPublic Acts\n[ Home ]  [ ILCS ] [ Search ] [ Bottom ]\n [ Other General Assemblies ]\n','', regex=False)

datasource = datasource[~datasource['original_text'].str.contains("No LegislationDocumentId returned.")]

for index,url in enumerate(datasource['bill_status']):
    try:
        driver.get(url)
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
        )
        bill_status_pages.append(driver.find_element(By.CSS_SELECTOR, "body").text)
    except:
        bill_status_pages.append('NA')

datasource['bill_status_pages'] = bill_status_pages

datasource = datasource[~datasource['bill_status_pages'].str.contains("Request Not Found")]

# Reset the index
datasource = datasource.reset_index(drop=True)

for index, text in enumerate(datasource['bill_status_pages']):
    print(index)
    # Regular expression pattern to find dates in the format MM/DD/YYYY
    year_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'

    # Find all occurrences of the pattern in the text
    dates_found = re.findall(pattern, text)

    # Convert the string dates to datetime objects and find the earliest
    if dates_found:
        # Convert each found date string to a datetime object
        date_objects = [datetime.strptime(date, '%B %d, %Y') for date in dates_found]
        # Find the earliest date
        earliest_date = min(date_objects)
        print(f"Earliest date found: {earliest_date.strftime('%m/%d/%Y')}")
        year = earliest_date.year
        years.append(year)
    else:
        print("No dates found in the text.")
        years.append('')

    effective_date_pattern = r'Effective Date (\w+ \d{1,2}, \d{4})'
    # Search for the pattern in the text
    effective_date_match = re.search(effective_date_pattern, text)

    # Check if a match was found
    if effective_date_match:
        # Extract the date string
        date_str = effective_date_match.group(1)

        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, '%B %d, %Y')  # '%B %d, %Y' is the format like 'March 17, 2023'

        # Convert the datetime object to the desired format MM/DD/YYYY
        effective_date = date_obj.strftime('%m/%d/%Y')
        effective_dates.append(effective_date)
    else:
        print("No date found after 'Effective Date' in the given text.")
        effective_dates.append('')

datasource['year'] = years
datasource['effective_date'] = effective_dates

# PA(90-0564) 12-22-97
# PA(91-0676) 12-23-99
# PA(92-0505) 12-20-01

for text in datasource['original_text']:
    #Regular expression pattern to find numbers after 'Public Act'
    pattern = r'Public Act (\d+-\d+)'

    # Search for the pattern in the text
    match = re.search(pattern, text)
    # Check if any matches were found and print them
    if match:
        orginal_act_nums.append(match.group(1))
    else:
        print("No numbers found after 'Public Act' in the given text.")
        orginal_act_nums.append('')

datasource['orginal_act_num'] = orginal_act_nums

for index, row in datasource.iterrows():
    num = row['orginal_act_num']
    if row['year'] == '':
        parts = num.split('-')
        if int(parts[0]) == 90:
            if int(parts[1]) <= 564:
                datasource.at[index, 'year'] = 1997
            else:
                datasource.at[index, 'year'] = 1998
        elif int(parts[0]) == 91:
            if int(parts[1]) <= 676:
                datasource.at[index, 'year'] = 1999
            else:
                datasource.at[index, 'year'] = 2000
        else:
            if int(parts[1]) <= 505:
                datasource.at[index, 'year'] = 2001
            else:
                datasource.at[index, 'year'] = 2002



datasource = datasource[~datasource['original_text'].str.contains('404 - File or directory not found')]

datasource.to_excel('IL_Leginfo.xlsx')
datasource.to_csv('IL_Leginfo.csv')
datasource.to_pickle('IL_Leginfo.pkl')
datasource.to_json('IL_Leginfo.json')


# End of IL_Leginfo.py


# Start of KY_Leginfo.py

# import libraries
import fitz
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options
from time import sleep
import pandas as pd
#import datefinder
import calendar
import os
import unittest
from random import randint
#import PyPDF2
import glob
import pickle
import time
import pdfplumber
import PyPDF2
import requests
import re
# Set the working directory
os.chdir('/Volumes/SSD/AFRI/Data/Raw_Data/KY')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))



#driver_path = r'C:/Users/longy/OneDrive/Projects/AFRI/windows/chromedriver.exe' #for windows
driver_path = '/Users/long/Library/CloudStorage/OneDrive-Personal/Projects/AFRI/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_experimental_option('prefs', {
    "download.default_directory": "/Volumes/SSD/AFRI/Data/Raw_Data/KY", #Change default directory for downloads
    "download.prompt_for_download": True, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True, #It will not show PDF directly in chrome
    "--enable-javascript":True})

driver = webdriver.Chrome(executable_path=driver_path)



# driver = webdriver.Firefox(executable_path='/path_to_geckodriver_folder/geckodriver')
# If geckodriver is in your PATH, you can initiate it directly
#driver = webdriver.Firefox()


# Set up browser options to auto-download PDFs
options = Options()
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", '/Volumes/SSD/AFRI/Data/Raw_Data/KY/acts_PDFs')

driver = webdriver.Firefox(options=options)


session_url = []
act_urls = []
bad_act_urls = []
bill_num = []
driver.get("https://legislature.ky.gov/Law/Pages/KyActs.aspx")
sessions = driver.find_elements(By.PARTIAL_LINK_TEXT, "Session")
for index,session in enumerate(sessions[:37]):
    session_url.append(session.get_attribute('href'))

for index,url in enumerate(session_url):
    if index < 19:
        driver.get(url)
        driver.find_element(By.LINK_TEXT, "Acts Table by Chapter").click()
        sleep(3)
        bills = driver.find_elements(By.CSS_SELECTOR, ".table > tbody:nth-child(1) > tr > td:nth-child(2)")
        for bill in bills:
            bill_num.append(bill.text)
        acts = driver.find_elements(By.CSS_SELECTOR, ".table > tbody:nth-child(1) > tr > td:nth-child(1) > a:nth-child(1)")
        for act in acts:
            act_url = act.get_attribute('href')
            act_urls.append(act_url)
            try:
                name = str(act_url).split("acts/")[1].split(".pdf")[0].replace("/","").replace("documents","")
            except:
                name = str(act_url).split("ACTS")[1].replace("/","").replace(".pdf","")
            path = "/Volumes/SSD/AFRI/Data/Raw_Data/KY/acts_PDFs/" + name + ".pdf"
            # Send a GET request to the URL
            response = requests.get(act_url)
            # Ensure the request was successful
            if response.status_code == 200:
                # Write the content of the response to a file
                with open(path, 'wb') as file:
                    file.write(response.content)
            else:
                print("Failed to fetch PDF. Status code:", response.status_code)
                bad_act_urls.append(act_url)
    elif index < 22:
        driver.get(url)
        driver.find_element(By.LINK_TEXT, "Acts Table by Chapter").click()
        sleep(3)
        bills = driver.find_elements(By.CSS_SELECTOR,
                                     "body > table:nth-child(6) > tbody:nth-child(1) > tr > td:nth-child(2)")
        for bill in bills:
            bill_num.append(bill.text)
        acts = driver.find_elements(By.CSS_SELECTOR,
                                    "body > table:nth-child(6) > tbody:nth-child(1) > tr > td:nth-child(1) > a:nth-child(1)")
        for act in acts:
            act_url = act.get_attribute('href')
            act_urls.append(act_url)
            name = str(act_url).split("acts/")[1].split(".pdf")[0].replace("/", "").replace("documents", "")
            path = "/Volumes/SSD/AFRI/Data/Raw_Data/KY/acts_PDFs/" + name + ".pdf"
            # Send a GET request to the URL
            response = requests.get(act_url)
            # Ensure the request was successful
            if response.status_code == 200:
                # Write the content of the response to a file
                with open(path, 'wb') as file:
                    file.write(response.content)
            else:
                print("Failed to fetch PDF. Status code:", response.status_code)
                bad_act_urls.append(act_url)
    elif index == 22:
        driver.get(url)
        driver.find_element(By.LINK_TEXT, "Acts Table by Chapter").click()
        sleep(3)
        bills = driver.find_elements(By.CSS_SELECTOR, "body > table > tbody > tr> td:nth-child(1)")
        for bill in bills:
            bill_num.append(bill.text)
        acts = driver.find_elements(By.CSS_SELECTOR,
                                    "body > table > tbody > tr > td:nth-child(2) > a")
        for act in acts:
            act_url = act.get_attribute('href')
            act_urls.append(act_url)
            try:
                name = str(act_url).split("acts/")[1].split(".pdf")[0].replace("/", "").replace("documents", "")
            except:
                name = str(act_url).split("ACTS")[1].replace("/", "").replace(".pdf", "")
            path = "/Volumes/SSD/AFRI/Data/Raw_Data/KY/acts_PDFs/" + name + ".pdf"
            # Send a GET request to the URL
            response = requests.get(act_url)
            # Ensure the request was successful
            if response.status_code == 200:
                # Write the content of the response to a file
                with open(path, 'wb') as file:
                    file.write(response.content)
            else:
                print("Failed to fetch PDF. Status code:", response.status_code)
                bad_act_urls.append(act_url)
    elif index < 33:
        driver.get(url)
        driver.find_element(By.LINK_TEXT, "Acts Table by Chapter").click()
        sleep(3)
        bills = driver.find_elements(By.CSS_SELECTOR,
                                     "body > table:nth-child(6) > tbody:nth-child(1) > tr > td:nth-child(2)")
        for bill in bills:
            bill_num.append(bill.text)
        acts = driver.find_elements(By.CSS_SELECTOR,
                                    "body > table:nth-child(6) > tbody:nth-child(1) > tr > td:nth-child(1) > a:nth-child(1)")
        for act in acts:
            act_url = act.get_attribute('href')
            act_urls.append(act_url)
            name = str(act_url).split("acts/")[1].split(".pdf")[0].replace("/", "").replace("documents", "")
            path = "/Volumes/SSD/AFRI/Data/Raw_Data/KY/acts_PDFs/" + name + ".pdf"
            # Send a GET request to the URL
            response = requests.get(act_url)
            # Ensure the request was successful
            if response.status_code == 200:
                # Write the content of the response to a file
                with open(path, 'wb') as file:
                    file.write(response.content)
            else:
                print("Failed to fetch PDF. Status code:", response.status_code)
                bad_act_urls.append(act_url)
    else:
        driver.get(url)
        driver.find_element(By.LINK_TEXT, "Acts Table by Chapter").click()
        sleep(3)
        bills = driver.find_elements(By.CSS_SELECTOR, "body > table > tbody > tr> td:nth-child(2)")
        for bill in bills:
            bill_num.append(bill.text)
        acts = driver.find_elements(By.CSS_SELECTOR, "body > table > tbody > tr> td:nth-child(1) > a")
        for act in acts:
            act_url = act.get_attribute('href')
            act_urls.append(act_url)
            try:
                name = str(act_url).split("acts/")[1].split(".pdf")[0].replace("/","").replace("documents","")
            except:
                name = str(act_url).split("ACTS")[1].replace("/","").replace(".pdf","")
            path = "/Volumes/SSD/AFRI/Data/Raw_Data/KY/acts_PDFs/" + name + ".pdf"
            # Send a GET request to the URL
            response = requests.get(act_url)
            # Ensure the request was successful
            if response.status_code == 200:
                # Write the content of the response to a file
                with open(path, 'wb') as file:
                    file.write(response.content)
            else:
                print("Failed to fetch PDF. Status code:", response.status_code)
                bad_act_urls.append(act_url)

for index, url in enumerate(session_url[33:36]):
    driver.get(url)
    driver.find_element(By.LINK_TEXT, "Acts Table by Chapter").click()
    sleep(3)
    bills = driver.find_elements(By.CSS_SELECTOR, "body > table > tbody:nth-child(1) > tr > td:nth-child(2)")
    for bill in bills:
        bill_num.append(bill.text)
    acts = driver.find_elements(By.CSS_SELECTOR,
                                "body > table > tbody:nth-child(1) > tr > td:nth-child(1) > a:nth-child(1)")
    for act in acts:
        act_url = act.get_attribute('href')
        act_urls.append(act_url)
        name = str(act_url).split("acts/")[1].split(".pdf")[0].replace("/", "").replace("documents", "")
        path = "/Volumes/SSD/AFRI/Data/Raw_Data/KY/acts_PDFs/" + name + ".pdf"
        # Send a GET request to the URL
        response = requests.get(act_url)
        # Ensure the request was successful
        if response.status_code == 200:
            # Write the content of the response to a file
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to fetch PDF. Status code:", response.status_code)
            bad_act_urls.append(act_url)

for index, url in enumerate(session_url[35:36]):
    driver.get(url)
    driver.find_element(By.LINK_TEXT, "Acts Table by Chapter").click()
    sleep(3)
    bills = driver.find_elements(By.CSS_SELECTOR, "body > table > tbody:nth-child(1) > tr > td:nth-child(2)")
    for bill in bills:
        bill_num.append(bill.text)
    acts = driver.find_elements(By.CSS_SELECTOR,
                                "body > table > tbody:nth-child(1) > tr > td:nth-child(1) > a:nth-child(1)")
    for act in acts:
        act_url = act.get_attribute('href')
        act_urls.append(act_url)
        name = str(act_url).split("acts/")[1].split(".pdf")[0].replace("/", "").replace("documents", "")
        path = "/Volumes/SSD/AFRI/Data/Raw_Data/KY/acts_PDFs/" + name + ".pdf"
        # Send a GET request to the URL
        response = requests.get(act_url)
        # Ensure the request was successful
        if response.status_code == 200:
            # Write the content of the response to a file
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to fetch PDF. Status code:", response.status_code)
            bad_act_urls.append(act_url)


act_txts = []

act_urls1 = list(set(act_urls))

datasource = pd.DataFrame({
    'bill_num':bill_num,
    'act_urls': act_urls
})

# Remove duplicates based on 'act_urls'
datasource = datasource.drop_duplicates(subset='act_urls', keep='first')

datasource = datasource[datasource['act_urls'] != "http://www.lrc.ky.gov/Statrev/ACTS2003/0194.pdf"]

act_txts = []

bad_files = []
path = "/Volumes/SSD/AFRI/Data/Raw_Data/KY/acts_PDFs"
files = glob.glob(os.path.join(path, '*.pdf'))

for file in files:
    try:
        doc = fitz.open(file)
        pgtxts = []
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)
            acttxt = ' '.join(pgtxts)
        act_txts.append(acttxt)
        doc.close()
    except:
        print(file)
        bad_files.append(file)

# Replace 'path_to_pdf' with the path to your PDF file
file = "./KY_1998.pdf"

doc = fitz.open(file)
pgtxts = []
for page in doc:
    pgtxt = page.get_text()
    pgtxts.append(pgtxt)
    acttxt = ' '.join(pgtxts)
acttxt_backup = acttxt
year_1998_acts = acttxt.split("Section 1.   ")


def split_by_chapter(text):
    # Define the pattern to split on
    pattern = re.compile(r'CHAPTER \d+ \n\(')

    # Split the text using the pattern
    chapters = re.split(pattern, text)

    # Filter out empty strings
    chapters = [chapter for chapter in chapters if chapter]

    return chapters


# Example usage:
chapters = split_by_chapter(acttxt)

year_1998_chapters =  chapters[1:]

year_1998_chapters = [f"Chapter {i+1} {item}" for i, item in enumerate(year_1998_chapters)]

year_1998_bill_num = []

for act in chapters[1:]:
    year_1998_bill_num.append(act.split(") \n")[0].replace(" ", ""))

# Creating DataFrame
df_1998 = pd.DataFrame({'original_text': year_1998_chapters, 'bill_num': year_1998_bill_num})
df_1998['act_urls'] = 'https://apps.legislature.ky.gov/law/acts/98RS/actsmas.pdf'
df_1998['file_name'] = "KY_1998.pdf"

# Read the CSV file into a DataFrame
df = pd.read_csv('./KY_Leginfo.csv', index_col=0)

df = pd.concat([df, df_1998], ignore_index=True)

df['state'] = "KY"


df.to_excel('KY_Leginfo.xlsx')
df.to_csv('KY_Leginfo.csv')
df.to_pickle('KY_Leginfo.pkl')
df.to_json('KY_Leginfo.json')


# End of KY_Leginfo.py


# Start of NJ_Leginfo.py

# import libraries
## selenium
import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
## others
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
import fitz
import re

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/NJ'
print(os.path.abspath(working_directory))


# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Initialize the Firefox driver with the specified options
executable_path = '../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)

# empty list to store info

act_pdf_urls = []
act_html_urls = []
bill_nums = []
chapter_nums = []
years = []
sessions = []
broken_act_urls = []
act_txts = []
act_names = []
names = []
#approved_dates = []
file_names = []


# New Jersey Legislature Website: https://www.njleg.state.nj.us/chapter-laws

driver.get('https://www.njleg.state.nj.us/chapter-laws')

WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.chapter-laws-filter_yearSelect__G_DM8'))
)

select_session = Select(driver.find_element(By.CSS_SELECTOR,'.chapter-laws-filter_yearSelect__G_DM8'))

sessions = driver.find_elements(By.CSS_SELECTOR,'.chapter-laws-filter_yearSelect__G_DM8  > option')

for session in sessions:
    if int(session.text) < 2022:
        print(session.text)
        select_session.select_by_visible_text(session.text)

        WebDriverWait(driver, 60).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.chapter-laws-filter_filterButtons__NFfv4:nth-child(1)'))
        )

        driver.find_element(By.CSS_SELECTOR,'button.chapter-laws-filter_filterButtons__NFfv4:nth-child(1)').click()

        total_page = int(driver.find_element(By.CSS_SELECTOR, 'div.chapter-laws-pagination_pagination__ulkGd:nth-child(1) > div:nth-child(1)').text.split('of ')[1])

        for page in range(0,total_page):
            print(page)

            chapter_num_elements = driver.find_elements(By.CSS_SELECTOR,
                                                        'div.chapter-laws-list_chapterLaw__KotAo > div:nth-child(1) > div:nth-child(1) > span:nth-child(1)')

            # Iterate over each element and get the text
            for element in chapter_num_elements:
                chapter_nums.append(element.text)


            bill_num_elements = driver.find_elements(By.CSS_SELECTOR,
                                                     'div.chapter-laws-list_chapterLaw__KotAo > div:nth-child(1) > div:nth-child(1) > a:nth-child(2)')

            # Iterate over each element and get the text
            for element in bill_num_elements:
                bill_nums.append(element.text)


            summmary_elements = driver.find_elements(By.CSS_SELECTOR,
                                                     'div.chapter-laws-list_chapterLaw__KotAo > div:nth-child(2) > p:nth-child(1)')

            # Iterate over each element and get the text
            for element in summmary_elements:
                names.append(element.text)


            act_pdf_url_elements = driver.find_elements(By.CSS_SELECTOR,
                                                        'div.chapter-laws-list_chapterLaw__KotAo > div:nth-child(1) > div:nth-child(2) > a:nth-child(1)')

            # Iterate over each element and get the text
            for element in act_pdf_url_elements:
                act_pdf_urls.append(element.get_attribute('href'))

            #act_html_url_elements = driver.find_elements(By.CSS_SELECTOR,
            # 'div.chapter-laws-list_chapterLaw__KotAo > div:nth-child(1) > div:nth-child(2) > a:nth-child(2)')

            # Iterate over each element and get the text
            #for element in act_html_url_elements:
            #    act_html_urls.append(element.get_attribute('href'))

            #approved_date_elements = driver.find_elements(By.CSS_SELECTOR,
            #  'div.chapter-laws-list_chapterLaw__KotAo > div:nth-child(1) > div:nth-child(2) > p:nth-child(3)')

            # Iterate over each element and get the text
            #for element in approved_date_elements:
            #    approved_dates.append(element.text)

            try:
                driver.find_element(By.CSS_SELECTOR,'div.chapter-laws-pagination_pagination__ulkGd:nth-child(1) > div:nth-child(2) > a:last-child  > svg:nth-child(1) > path:nth-child(1)').click()
            except:
                pass
            time.sleep(2)




# download corresponding pdf file

for act_url in act_pdf_urls:
    print(index)
    print(act_url)
    name = act_url.replace('/','').split('nj.usBills')[1]
    file_names.append(name)
    print(name)

    path = download_dir + name
    print(os.path.abspath(path))

    time.sleep(randint(1, 100) * 0.005)

    response = requests.get(act_url)
    # Ensure the request was successful
    if response.status_code == 200:
        # Write the content of the response to a file
        with open(path, 'wb') as file:
            file.write(response.content)
    else:
        print("Failed to fetch PDF. Status code:", response.status_code)
        broken_act_urls.append(act_url)
datasource= pd.DataFrame({
    'bill_num': bill_nums,
    'chapter_num': chapter_nums,
    'link': act_pdf_urls,
    'state': "NJ",
    'name': names
})
datasource['file_name'] = datasource['link'].apply(lambda x: x.replace('/', '').split('nj.usBills')[1])

#read pdfs
folder_path = "./PDFs"
print(os.path.abspath(folder_path))

files = glob.glob(os.path.join(folder_path, '*.PDF'))

broken_files = []

for file in files:
    try:
        doc = fitz.open(file)
        pgtxts = []
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)
            acttxt = ' '.join(pgtxts)
        act_txts.append(acttxt)
        doc.close()
    except:
        act_txts.append('NA')
        broken_files.append(file)

file_names = []

for file in files:
    file_names.append(file.split("./PDFs/")[1])

datasource_pdf = pd.DataFrame({
    'original_text': act_txts,
    'file_name': file_names,
})


datasource = pd.merge(datasource,datasource_pdf,on='file_name', how='left')

# delete one act with missing pdf from the url https://pub.njleg.state.nj.us/Bills/2010/PL11/209_.PDF
datasource = datasource.dropna(subset=['original_text'])
# delete duplicates
datasource = datasource.drop_duplicates(subset=['original_text'])


datasource.to_excel('NJ_Leginfo.xlsx')
datasource.to_csv('NJ_Leginfo.csv')
datasource.to_pickle('NJ_Leginfo.pkl')
datasource.to_json('NJ_Leginfo.json')

# End of NJ_Leginfo.py


# Start of WV_Leginfo.py

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import time
import json
import re
import fitz


#driver_path = 'I:\Projects\AFRI/chromedriver.exe' #for windows
driver_path = '/Users/long/Library/CloudStorage/OneDrive-Personal/Projects/AFRI/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
#os.chdir('/Users/long/OneDrive/Projects/AFRI')
os.chdir('/Volumes/SSD/AFRI/Data/Raw_Data/WV')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_experimental_option('prefs', {
    "download.default_directory": '/Users/long/Library/CloudStorage/OneDrive-Personal/Projects/AFRI/data/WV', #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})

driver = webdriver.Chrome(executable_path=driver_path, options=chromeOptions)

act_urls = []
checklist = []
session_acts = []
acttxts = []

driver.get("https://www.wvlegislature.gov/Educational/publications.cfm#acts")

#most sessions except 2003

sessions = driver.find_elements_by_partial_link_text("Vol.")

for session in sessions:
    url = session.get_attribute("href")
    driver.get(url)
    time.sleep(1)

othersessions = ["https://www.wvlegislature.gov/legisdocs/publications/acts/Acts_2003_Vol_1.pdf","https://www.wvlegislature.gov/legisdocs/publications/acts/Acts_2003_Vol_2.pdf","https://www.wvlegislature.gov/legisdocs/publications/acts/Acts_2003_2ES.pdf"]
for url in othersessions:
    driver.get(url)
    time.sleep(1)


acttxts = []
path = "/Volumes/SSD/AFRI/Data/Raw_Data/WV/PDFs"
files = glob.glob(os.path.join(path, '*.pdf'))
for file in files:
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        acttxt = ' '.join(pgtxts)
    acttxts.append(acttxt)
    doc.close()


datasource = pd.DataFrame({
    'original_text': acttxts,
    'file_name': files
})

# save bill info into files
datasource.to_excel('WV_Leginfo.xlsx')
datasource.to_csv('WV_Leginfo.csv')
datasource.to_pickle('WV_Leginfo.pkl')
datasource.to_json('WV_Leginfo.json')
# End of WV_Leginfo.py


# Start of ID_Leginfo.py

# import libraries
## selenium
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import inflect
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import fitz
import re


state = "ID"
# Read the CSV file into a DataFrame
work_dir = f'/Volumes/SSD/AFRI/Data/Raw_Data/{state}/'
# Change the working directory
os.chdir(work_dir)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))


# Specify the path to the Firefox binary
firefox_binary_path = '/Applications/Firefox.app/Contents/MacOS/firefox'

# Create a Firefox profile and configure preferences
options = Options()
options.binary_location = firefox_binary_path

# Initialize the Firefox driver with the specified options
executable_path = './Data/geckodriver'
print(os.path.abspath(executable_path))

# Initialize the WebDriver with the specified path
service = Service(executable_path)
driver = webdriver.Firefox(service=service)



driver.get('https://legislature.idaho.gov/statutesrules/sessionlaws/')

sessions = driver.find_elements(By.PARTIAL_LINK_TEXT, "Volume")

session_urls = []

for session in sessions:
    session_url = session.get_attribute('href')
    session_urls.append(session_url)
for idx, url in enumerate(session_urls):
    driver.get(url)
    print(idx)
    sleep(6)


path = "PDFs"
files = glob.glob(os.path.join(path, '*.pdf'))

acttxts = []
year = []
chapter_num = []
for file in files:
    pgtxts = []
    doc = fitz.open(file)  # Open the file explicitly
    try:
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)

        acttxt = '  '.join(pgtxts)
    finally:
        doc.close()  # Ensure the file is closed after processing
    acttxt = acttxt.split('RESOLUTIONS \n')[0]
    parts = re.split(r'\nCHAPTER \d+ \n', acttxt)
    acttxts.extend(parts[1:])
    chapter_num.extend(re.findall(r'\nCHAPTER (\d+) \n', acttxt))
    # Append the same number of rows (for demonstration, we'll append a placeholder)
    for i in range(len(parts[1:])):
        year.append(file[-8:-4])

datasource = pd.DataFrame({
    'original_text': acttxts,
    'year': year,
    'chapter_num': chapter_num,
    'state': "ID"
})

datasource.to_excel('ID_Leginfo.xlsx')
datasource.to_csv('ID_Leginfo.csv')
datasource.to_pickle('ID_Leginfo.pkl')
datasource.to_json('ID_Leginfo.json')


# End of ID_Leginfo.py


# Start of LA_Leginfo.py

# import libraries
## selenium
import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
## others
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
import fitz
import re
import math

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/LA'
print(os.path.abspath(working_directory))


# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Initialize the Firefox driver with the specified options
executable_path = '../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)


driver.get('https://www.legis.la.gov/Legis/SessionInfo/SessionInfo.aspx')


acts = []
act_urls = []
act_names = []
act_nums = []
bill_nums = []
years = []
records = []

years = ['97RS','971ES','972ES','973ES','98RS','981ES','982ES','983ES','99RS','991ES','992ES','993ES','00RS','001ES','002ES','003ES','01RS','011ES','012ES','013ES','02RS','021ES','022ES','023ES','03RS','031ES','032ES','033ES','04RS','041ES','042ES','043ES','05RS','051ES','052ES','053ES','06RS','061ES','062ES','063ES','07RS','071ES','072ES','073ES','08RS','081ES','082ES','083ES','09RS','091ES','092ES','093ES','10RS','101ES','102ES','103ES','11RS','111ES','112ES','113ES','12RS','121ES','122ES','123ES','13RS','131ES','132ES','133ES','14RS','141ES','142ES','143ES','15RS','151ES','152ES','153ES','16RS','161ES','162ES','163ES','17RS','171ES','172ES','173ES','18RS','181ES','182ES','183ES','19RS','191ES','192ES','193ES','20RS','201ES','202ES','203ES','21RS','211ES','212ES','213ES']

for year in years[1:]:
    url = 'https://www.legis.la.gov/Legis/ActNumbers.aspx?sid=' + year
    try:
        driver.get(url)

        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
        )

        act_elements = driver.find_elements(By.CSS_SELECTOR,'.ResultsListTable > tbody:nth-child(2) > *')

        for element in act_elements:

            act_nums.append(element.find_element(By.CSS_SELECTOR, ':nth-child(1)').text)

            bill_nums.append(element.find_element(By.CSS_SELECTOR, ':nth-child(2)').text)

            act_urls.append(element.find_element(By.PARTIAL_LINK_TEXT, 'B').get_attribute('href'))

            act_names.append(element.find_element(By.CSS_SELECTOR, ':nth-child(4)').text)

        num_pages = math.floor(int(re.findall(r'\d+', driver.find_element(By.CSS_SELECTOR,'#ctl00_ctl00_PageBody_PageContent_LabelTotalInstruments').text)[0])/100) + 1

        if num_pages > 1:
            for n in range(2,num_pages+1):

                print(n)

                if n == 11:
                    next_page = driver.find_element(By.LINK_TEXT,'...').click()
                else:
                    next_page = driver.find_element(By.LINK_TEXT,f"{n}").click()

                time.sleep(3)

                act_elements = driver.find_elements(By.CSS_SELECTOR, '.ResultsListTable > tbody:nth-child(2) > *')

                for element in act_elements:
                    act_nums.append(element.find_element(By.CSS_SELECTOR, ':nth-child(1)').text)

                    bill_nums.append(element.find_element(By.CSS_SELECTOR, ':nth-child(2)').text)

                    act_urls.append(element.find_element(By.PARTIAL_LINK_TEXT, 'B').get_attribute('href'))

                    act_names.append(element.find_element(By.CSS_SELECTOR, ':nth-child(4)').text)
    except:
        pass

pdf_urls = []
for act_url in act_urls:
    driver.get(act_url)
    WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
    )
    year = driver.find_element(By.CSS_SELECTOR, "#ctl00_PageBody_LabelSession").text.split(' REGULAR')[0]
    years.append(year)
    records.append(driver.find_element(By.CSS_SELECTOR, "#ctl00_PageBody_PanelBillInfo").text)

    driver.get(driver.find_element(By.LINK_TEXT,'Text').get_attribute('href'))
    WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
    )
    pdf_urls.append(driver.find_element(By.CSS_SELECTOR, "a").get_attribute('href'))

    time.sleep(randint(1, 100)*0.005)


file_names = []

for index, link in enumerate(pdf_urls):
    print(index)
    name = str(link).split("d=")[1]
    file_names.append(name + ".pdf")
    path = download_dir + name + ".pdf"
    # Send a GET request to the URL
    response = requests.get(link)
    # Ensure the request was successful
    if response.status_code == 200:
        # Write the content of the response to a file
        with open(path, 'wb') as file:
            file.write(response.content)
    else:
        print("Failed to fetch PDF. Status code:", response.status_code)
        bad_act_urls.append(link)
    time.sleep(randint(1, 100)*0.005)


datasource = pd.DataFrame({
    'act_name': act_names,
    'bill_num': bill_nums,
    'act_num': act_nums,
    'link': act_urls,
    'pdf_url': pdf_urls,
    'file_name': file_names,
    'year': years,
    'record': records
})



files = glob.glob(os.path.join(download_dir, '*.pdf'))

file_names = []

broken_files = []

act_txts = []

for file in files:
    try:
        print(file)
        doc = fitz.open(file)
        pgtxts = []
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)
            act_txt = ' '.join(pgtxts)
        act_txts.append(act_txt)
        doc.close()
        file_names.append(file.split('PDFs/')[1])
    except:
        broken_files.append(file)



file_df = pd.DataFrame({
    'file_name': file_names,
    'original_text': act_txts
})





datasource.to_csv('LA_Leginfo.csv', index=False)
datasource.to_excel('LA_Leginfo.xlsx', index=False)
datasource.to_pickle('LA_Leginfo.pkl')
datasource.to_json('LA_Leginfo.json', index=False)

#datasource = pd.read_csv('LA_Leginfo.csv', index_col=False)
# End of LA_Leginfo.py


# Start of MT_Leginfo.py

# import libraries
## selenium
import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
## others
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
import fitz
import re
# Counting the occurrence of each item in the list
from collections import Counter

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/MT'
print(os.path.abspath(working_directory))


# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Initialize the Firefox driver with the specified options
executable_path = '../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)

# empty list to store info

act_pdf_urls = []
act_html_urls = []
pdf_file_names = []
html_file_names = []
bill_nums = []
chapter_nums = []
years = []
sessions = []
broken_act_urls = []
act_txts = []
act_names = []
names = []
file_names = []
summaries = []
session_urls = []
statuss = []
rowtxts = []

# Montana Legislature Website: https://laws.leg.mt.gov/legprd/law0203w$.startup

for i in ['20211', '20191', '20171', '20172', '20151', '20131', '20111', '20091', '20071', '20072', '20051', '20051', '20031', '20011', '19991']:
    print(i)
    url = 'https://laws.leg.mt.gov/legprd/LAW0217W$BAIV.return_all_bills?P_SESS=' + i
    session_urls.append(url)
    driver.get(url)

    WebDriverWait(driver, 60).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'body'))
    )

    rows = driver.find_elements(By.CSS_SELECTOR,'body > table:nth-child(13) > tbody:nth-child(1) > tr')

    for row in rows[1:]:
        if "*" not in row.text:

            print(row.text)
            rowtxts.append(row.text)

            act_html_url = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > a:nth-child(2)').get_attribute('href')
            act_html_urls.append(act_html_url)

            act_pdf_url = row.find_element(By.CSS_SELECTOR,'td:nth-child(1) > a:nth-child(3)').get_attribute('href')
            act_pdf_urls.append(act_pdf_url)

            bill_num = row.find_element(By.CSS_SELECTOR,'td:nth-child(1) > a:nth-child(1)').text
            bill_nums.append(bill_num.replace(' ',''))

            chapter_num = row.find_element(By.CSS_SELECTOR,'td:nth-child(2)').text
            chapter_nums.append(chapter_num)

            status = row.find_element(By.CSS_SELECTOR,'td:nth-child(4)').text
            statuss.append(status)

            year = row.find_element(By.CSS_SELECTOR,'td:nth-child(5)').text
            years.append(year[-4:])

            name = row.find_element(By.CSS_SELECTOR,'td:nth-child(6)').text
            names.append(name)

            time.sleep(randint(1, 3)*0.001)



datasource = pd.DataFrame({
    'bill_num': rowtxts,
    'chapter_num': chapter_nums,
    'act_pdf_url': act_pdf_urls,
    'act_html_url': act_html_urls,
    'state': "MT",
    'name':names,
    'year': years,
    'status':statuss
})

file_names = []

for url in datasource['act_pdf_url']:
    file_names.append(url.split('/billpdf')[0][-4:] + url.split('billpdf/')[1].split('.htm')[0])

datasource['file_name'] = file_names

act_txts = []
file_names = []
for act_url in act_pdf_urls:
    print(act_url)
    name = act_url.split('/billpdf')[0][-4:] + act_url.split('billpdf/')[1].split('.htm')[0]
    file_names.append(name)
    print(name)

    path = download_dir + name
    print(os.path.abspath(path))

    time.sleep(randint(1, 100) * 0.005)

    response = requests.get(act_url)
    # Ensure the request was successful
    if response.status_code == 200:
        # Write the content of the response to a file
        with open(path, 'wb') as file:
            file.write(response.content)
    else:
        print("Failed to fetch PDF. Status code:", response.status_code)
        broken_act_urls.append(act_url)

folder_path = "./PDFs"
print(os.path.abspath(folder_path))

files = glob.glob(os.path.join(folder_path, '*.pdf'))

broken_files = []

for file in files:
    try:
        doc = fitz.open(file)
        pgtxts = []
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)
            acttxt = ' '.join(pgtxts)
        act_txts.append(acttxt)
        doc.close()
    except:
        act_txts.append('NA')
        broken_files.append(file)

file_names = []

for file in files:
    file_names.append(file.split("./PDFs/")[1])

datasource_pdf = pd.DataFrame({
    'original_text': act_txts,
    'file_name': file_names,
})


datasource = pd.merge(datasource, datasource_pdf, on='file_name', how= 'left')



# missing url 'http://leg.mt.gov/bills/2007/billpdf/SB0382.pdf'
datasource = datasource.dropna(subset=['original_text'])
# delete duplicates
datasource = datasource.drop_duplicates(subset=['original_text'])

datasource.to_excel('MT_Leginfo.xlsx')
datasource.to_csv('MT_Leginfo.csv')
datasource.to_pickle('MT_Leginfo.pkl')
datasource.to_json('MT_Leginfo.json')

# End of MT_Leginfo.py


# Start of FL_Leginfo.py

# import libraries
## selenium
import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
## others
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
import fitz
import re

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/FL'
#working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/FL'
print(os.path.abspath(working_directory))


# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Initialize the Firefox driver with the specified options
executable_path = '../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)

# empty list to store info
file_names = []
act_urls = []
bill_nums = []
chapter_nums = []
years = []
sessions = []
broken_act_urls = []
act_txts = []
act_names = []

#for 1975-1996
session_urls = []
years_1975_1996 = []
file_names_1975_1996 = []
# there are two sources to web-scrape for FL leginfo.
# for 1975-1996

# 1975 special session
name = '1975' + '_' + 'special.pdf'
file_names.append(name)
path = download_dir + name
years_1975_1996.append(1975)
response = requests.get('http://edocs.dlis.state.fl.us/fldocs/leg/actsflorida/1975/LOF1975V1Ch001-306.pdf')
session_urls.append('http://edocs.dlis.state.fl.us/fldocs/leg/actsflorida/1975/LOF1975V1Ch001-306.pdf')

# Ensure the request was successful
# Write the content of the response to a file
with open(path, 'wb') as file:
    file.write(response.content)

driver.get('http://edocs.dlis.state.fl.us/fldocs/leg/actsflorida/index.htm')
# regular sessions in 1975-1996
for index, year in enumerate(range(1975, 1997)):

    print(year)

    # get the last two digits of the year
    partial_link = f"{year % 100}-"

    session_elements = WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, partial_link))
    )

    for index, element in enumerate(session_elements):
        session_url = element.get_attribute("href")
        print(session_url)
        session_urls.append(session_url)

        name = str(year) + '_' + str(index+1) + '.pdf'
        file_names_1975_1996.append(name)
        years_1975_1996.append(year)


        path = download_dir + '/1975-1996/'+ name
        print(os.path.abspath(path))

        time.sleep(randint(1, 100) * 0.005)

        response = requests.get(pdf_url)
        # Ensure the request was successful
        if response.status_code == 200:
            # Write the content of the response to a file
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to fetch PDF. Status code:", response.status_code)
            broken_act_urls.append(pdf_url)


datasource_1975_1996 = pd.DataFrame({
    'file_name': file_names,
    'year':years,
    'link':session_urls
})

folder_path = "./PDFs/1975_1996/"
print(os.path.abspath(folder_path))

files = glob.glob(os.path.join(folder_path, '*.pdf'))

broken_files = []

for file in files:
    try:
        doc = fitz.open(file)
        pgtxts = []
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)
            session_txt = ' '.join(pgtxts)
        session_txts.append(session_txt)
        doc.close()
    except:
        print('NA')
        session_txts.append('NA')
        broken_files.append(file)

#creat a key to match
file_names = []

for file in files:
    file_names.append(file.split("1975_1996/")[1])

datasource_1975_1996_new = pd.DataFrame({
    'original_text': session_txts,
    'file_name': file_names
})

datasource_1975_1996 = pd.merge(datasource_1975_1996,datasource_1975_1996_new, on='file_name', how='left')

act_txts = []
years = []
file_names = []
act_urls = []

for index, row in datasource_1975_1996.iterrows():
    text = row['original_text']
    year = row['year']
    file_name = row['file_name']
    act_url = row['link']

    page_header_pattern = r'LAWS OF FLORIDA\s+\n+CHAPTER | CHAPTER+.*?\s+\n+LAWS OF FLORIDA | CHAPTER+.*?\s+\n+LAWS OF FLORIDA\s+\n+CHAPTER '
    # Finding all occurrences of the pattern in the text
    matches = re.findall(page_header_pattern, text)

    # Print the matches
    for match in matches:
        text = text.replace(match, '')

    # split the chapters
    chapters =[]
    chap_pattern  = r'Filed in Office Secretary of State \b(?:January|February|March|April|May|June|July|August|September|October|November|December)\b \d{1,2}, \d{4}\.'
    # Split the text using the modified pattern
    chapters = re.split(chap_pattern, text)
    for i,chapter in enumerate(chapters):
        try:
            act_txts.append('CHAPTER' + chapter.split('CHAPTER')[1])
        except:
            act_txts.append(chapter)
        years.append(year)
        file_names.append(file_name)
        act_urls.append(act_url)

datasource_1975_1996 = pd.DataFrame({
    'original_text': act_txts,
    'year': years,
    'link':act_urls,
    'file_name':file_names,
    'state': 'FL'
})

# for 1997-2021

driver.get('https://laws.flrules.org/node?field_list_year_nid=8184')

session_id_elements = driver.find_elements(By.CSS_SELECTOR,'#edit-field-list-year-nid option')

# Assuming session_id_elements is a list of elements
session_id_dict = {}

for index, element in enumerate(session_id_elements):
    year = element.text
    session_id = element.get_attribute("value")
    session_id_dict[index] = (year, session_id)

for index, (year, session_id) in session_id_dict.items():
    # Now you can use index, year, and session_id within your loop
    print(f"Index: {index}, Year: {year}, Session ID: {session_id}")
    if int(year) < 2022:
        print(year)
        session_url = 'https://laws.flrules.org/node?field_list_year_nid=' + session_id
        driver.get(session_url)

        act_elements = driver.find_elements(By.CSS_SELECTOR,'table.views-table:nth-child(1) > tbody:nth-child(3) > tr')

        for index, element in enumerate(act_elements):
            #print(element.text)
            if 'Ch' in element.text:
                print(element.text)

                act_url = element.find_element(By.PARTIAL_LINK_TEXT,'.pdf').get_attribute("href")
                act_urls.append(act_url)
                act_name = element.find_element(By.CSS_SELECTOR,':nth-child(3)').text
                act_names.append(act_name)
                bill_num = element.find_element(By.CSS_SELECTOR,':nth-child(4)').text.replace(' ','')
                bill_nums.append(bill_num)

                years.append(year)

                # download corresponding pdf file
                name =  element.find_element(By.CSS_SELECTOR,':nth-child(1)').text
                file_names.append(name)
                print(name)

                path = download_dir + name
                print(os.path.abspath(path))

                time.sleep(randint(1, 100) * 0.005)

                response = requests.get(act_url)
                # Ensure the request was successful
                if response.status_code == 200:
                    # Write the content of the response to a file
                    with open(path, 'wb') as file:
                        file.write(response.content)
                else:
                    print("Failed to fetch PDF. Status code:", response.status_code)
                    broken_act_urls.append(act_url)
            else:
                pass
        else:
            pass



folder_path = "./PDFs"
print(os.path.abspath(folder_path))

files = glob.glob(os.path.join(folder_path, '*.pdf'))

broken_files = []

for file in files:
    try:
        doc = fitz.open(file)
        pgtxts = []
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)
            acttxt = ' '.join(pgtxts)
        act_txts.append(acttxt)
        doc.close()
    except:
        act_txts.append('NA')
        broken_files.append(file)

file_names = []

for file in files:
    file_names.append(file.split("./PDFs/")[1])

file_names_new = []

for file in file_names:
    # Remove 'Ch_' and '.pdf', then replace '_' with '-' if it exists
    file = file.replace('Ch_', '').replace('.pdf', '').replace('_', '-')
    parts = file.split('-')  # Split at '-'
    year = parts[0]
    number = int(parts[1])  # Convert to int to remove leading zeros
    file_names_new.append(f"{year}/{number}")

datasource_1997_2021_pdf = pd.DataFrame({
    'original_text': act_txts,
    'file_name': file_names_new
})


datasource_1997_2021 = pd.DataFrame({
    'bill_num': bill_nums,
    'link': act_urls,
    'state': "FL",
    'act_name': act_names
})

# delete the duplicates from the websites, like https://laws.flrules.org/node?field_list_year_nid=5235
datasource_1997_2021 = datasource.drop_duplicates(keep='first')

datasource_1997_2021['file_name'] = datasource_1997_2021['link'].str.split('https://laws.flrules.org/').str[1]

datasource_1997_2021 = pd.merge(datasource_1997_2021_pdf, datasource_1997_2021, on='file_name', how='inner')

datasource = pd.concat([datasource_1975_1996, datasource_1997_2021], ignore_index=True)

datasource['year'] = datasource['file_name'].str[:4]

# Filter out the rows where the specified text is found
datasource = datasource[~datasource['original_text'].str.contains("\nHouse Concurrent Resolution No.")].reset_index(drop=True)

datasource.to_excel('FL_Leginfo.xlsx')
datasource.to_csv('FL_Leginfo.csv')
datasource.to_pickle('FL_Leginfo.pkl')
datasource.to_json('FL_Leginfo.json')
# End of FL_Leginfo.py


# Start of CA_leginfo_Galina.py

# Import libraries
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import openai
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import json
import nltk
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from nltk.stem import WordNetLemmatizer
import nltk
import inflection
from nltk.stem import WordNetLemmatizer
import pandas as pd
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import os
import unittest
from random import randint
#import PyPDF2
#import glob
import time
import json


# working path
os.chdir('/Users/long/Library/CloudStorage/GoogleDrive-long0555@umn.edu/Shared drives/UCSC-UMN AFRI project/Data/Raw_Data/CA/')
os.chdir('/Users/long/Documents/AFRI_AI/Inputs')
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))
driver_path = '/Users/long/Documents/AFRI_AI/chromedriver.exe'
chromeOptions = webdriver.ChromeOptions()
dnldpath = {"download.default_directory" :"/Users/long/Documents/AFRI_AI/Inputs"}
chromeOptions.add_experimental_option("prefs",dnldpath)
chromeOptions.add_argument('--no-sandbox')
chromeOptions.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path=driver_path, options=chromeOptions)


acts_Galina = pd.read_excel("StateLawsCA_by year_by category.xls")



# set up list titles
years = []
states = []
sessionyears = []
billnumbers = []
titles = []
briefsummarys = []
introduceddates = []
signingdates = []
effectivedates = []
expireddates = []
leadauthors = []
textlinks = []
fulltexts = []
fulltextpages = []

urls = acts_Galina["Link to full text"]

# extract bill information on each bill

for url in urls:
    driver.get(url)
    state = "CA"
    states.append(state)

    billnumber = driver.find_element(By.CSS_SELECTOR,'#bill_header > div:nth-child(3) > h1')
    billnumbers.append(billnumber.text.rsplit(" ")[0])

    title = driver.find_element(By.CSS_SELECTOR,'#bill_header > div:nth-child(3) > h1')
    title = title.text.split(" ",1)[1]
    titles.append((title.rsplit("(",1)[0]))

    textlinks.append(url)
    fulltext = driver.find_element(By.CSS_SELECTOR,"#bill_all").text
    fulltexts.append(fulltext)
    fulltextpage = driver.find_element(By.CSS_SELECTOR,"#centercolumn").text
    fulltextpages.append(fulltextpage)
    sleeptime = randint(1,11)*0.01
    time.sleep(sleeptime)

datasource = pd.DataFrame({
    'State': states,
    'Bill Number': billnumbers,
    'Name': titles,
    'Full text': fulltexts,
    'Link to full text':textlinks
})



# Python program to read
# json file



# Opening JSON file

os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/CA/')
f = open('CA_Leginfo.json')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))


#datasource = json.load(f)

#datasource = pd.read_pickle("CA_Leginfo.pkl")
#datasource = datasource.drop(['authors','Introduced Date','Date effective','Expiration date'], axis = 1)

# save bill info into files
datasource.to_excel('CA_Leginfo_Galina.xlsx')
datasource.to_csv('CA_Leginfo_Galina.csv')
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
datasource.to_sql('CA_Leginfo.sql',con=engine)
datasource.to_pickle('CA_Leginfo.pkl')
datasource.to_json('CA_Leginfo.json')


datasource.to_stata('CA_Leginfo.dta')

print("download finished")
# End of CA_leginfo_Galina.py


# Start of NASS.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
#import PyPDF2
#import glob
import time
import json


driver_path = 'C:/Users/longy/OneDrive/Projects/AFRI/windows/chromedriver.exe' #for windows
# driver_path = '/Users/Yanxu/OneDrive/Projects/AgPolicyCoding/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
#os.chdir('/Users/long/OneDrive/Projects/AFRI')
os.chdir('C:/Users/longy/OneDrive/Projects/AFRI')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
# for mac
#dnldpath = {"download.default_directory" : "/Users/long/OneDrive/Projects/AFRI/data/SD/"}
# for windows
dnldpath = {"download.default_directory" :"C:/Users/longy/OneDrive/Projects/AFRI/data/CA/"}
chromeOptions.add_experimental_option("prefs",dnldpath)
chromeOptions.add_argument('--no-sandbox')
chromeOptions.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path=driver_path, options=chromeOptions)

print("Current working directory: {0}".format(cwd))

driver = webdriver.Firefox()



sessions = ['2021 - 2022', '2019 - 2020', '2017 - 2018', '2015 - 2016', '2013 - 2014', '2011 - 2012', '2009 - 2010', '2007 - 2008', '2005 - 2006', '2003 - 2004', '2001 - 2002', '1999 - 2000']

urls = []
measures = []
subjects = []
authors = []
statuss = []
subjects1 = []
authors1 = []
statuss1 = []
# search bills by iterating sessionyears first and then keywords
for session in sessions:
    print(session)
    driver.get("https://leginfo.legislature.ca.gov/faces/billSearchClient.xhtml")
    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#attrSearch"))
    )
    select_session = Select(driver.find_element_by_css_selector('select#session_year'))
    select_session.select_by_visible_text(session)
    search = driver.find_element_by_css_selector('#attrSearch')
    search.click()

    WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody tr")))
    bills = driver.find_elements_by_css_selector('tbody tr')
    for bill in bills:
        if bill.find_element_by_css_selector("td:nth-child(4)").text == 'Chaptered':
            url = bill.find_element_by_css_selector("td:nth-child(1) a").get_attribute('href')
            urls.append(url)
            measure = bill.find_element_by_css_selector("td:nth-child(1) a").text
            measures.append(measure)
            subject = bill.find_element_by_css_selector("td:nth-child(2)").text
            subjects.append(subject)
            author = bill.find_element_by_css_selector("td:nth-child(3)").text
            authors.append(author)
            status = bill.find_element_by_css_selector("td:nth-child(4)").text
            statuss.append(status)
        else:
            print("Do not pass")
        time.sleep(0.01)
print("url done")

# set up list titles
years = []
states = []
sessionyears = []
billnumbers = []
titles = []
briefsummarys = []
introduceddates = []
signingdates = []
effectivedates = []
expireddates = []
leadauthors = []
textlinks = []
fulltexts = []
fulltextpages = []

# extract bill information on each bill

for url in urls[6313:]:
    driver.get(url)
    state = "CA"
    states.append(state)

    billnumber = driver.find_element_by_css_selector('#bill_header > div:nth-child(3) > h1')
    billnumbers.append(billnumber.text.rsplit(" ")[0])

    title = driver.find_element_by_css_selector('#bill_header > div:nth-child(3) > h1')
    title = title.text.split(" ",1)[1]
    titles.append((title.rsplit("(",1)[0]))

    textlinks.append(url)
    fulltext = driver.find_element_by_css_selector("#bill_all").text
    fulltexts.append(fulltext)
    fulltextpage = driver.find_element_by_css_selector("#centercolumn").text
    fulltextpages.append(fulltextpage)
    sleeptime = randint(1,11)*0.1
    time.sleep(sleeptime)

datasource = pd.DataFrame({
    'Year': years,
    'State': states,
    'Session Year': sessionyears,
    'Bill Number': billnumbers,
    'Name': titles,
    'Brief Summary': briefsummarys,
    'Introduced Date': introduceddates,
    'Date it was signed':signingdates,
    'Date effective':effectivedates,
    'Expiration date':expireddates,
    'Introducer':leadauthors,
    'Full text': fulltexts,
    'Link to full text':textlinks
})

datasource = pd.DataFrame({
    'Year': years,
    'State': states,
    'Session Year': sessionyears,
    'Bill Number': billnumbers,
    'Name': titles,
    'authors':authors,
    'Brief Summary': briefsummarys,
    'Introduced Date': introduceddates,
    'Date effective':effectivedates,
    'Expiration date':expireddates,
    'Full text': fulltexts,
    'Link to full text':textlinks
})


# Python program to read
# json file



# Opening JSON file

os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/CA/')
f = open('CA_Leginfo.json')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))


#datasource = json.load(f)

#datasource = pd.read_pickle("CA_Leginfo.pkl")
#datasource = datasource.drop(['authors','Introduced Date','Date effective','Expiration date'], axis = 1)

# save bill info into files
datasource.to_excel('CA_Leginfo.xlsx')
datasource.to_csv('CA_Leginfo.csv')
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
datasource.to_sql('CA_Leginfo.sql',con=engine)
datasource.to_pickle('CA_Leginfo.pkl')
datasource.to_json('CA_Leginfo.json')


datasource.to_stata('CA_Leginfo.dta')

print("download finished")
# End of NASS.py


# Start of CA_leginfo_v3.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
#import PyPDF2
#import glob
import time
import json


driver_path = 'C:/Users/longy/OneDrive/Projects/AFRI/windows/chromedriver.exe' #for windows
# driver_path = '/Users/Yanxu/OneDrive/Projects/AgPolicyCoding/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
#os.chdir('/Users/long/OneDrive/Projects/AFRI')
os.chdir('C:/Users/longy/OneDrive/Projects/AFRI')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
# for mac
#dnldpath = {"download.default_directory" : "/Users/long/OneDrive/Projects/AFRI/data/SD/"}
# for windows
dnldpath = {"download.default_directory" :"C:/Users/longy/OneDrive/Projects/AFRI/data/CA/"}
chromeOptions.add_experimental_option("prefs",dnldpath)
chromeOptions.add_argument('--no-sandbox')
chromeOptions.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path=driver_path, options=chromeOptions)

print("Current working directory: {0}".format(cwd))

driver = webdriver.Firefox()



sessions = ['2021 - 2022', '2019 - 2020', '2017 - 2018', '2015 - 2016', '2013 - 2014', '2011 - 2012', '2009 - 2010', '2007 - 2008', '2005 - 2006', '2003 - 2004', '2001 - 2002', '1999 - 2000']

urls = []
measures = []
subjects = []
authors = []
statuss = []
subjects1 = []
authors1 = []
statuss1 = []
# search bills by iterating sessionyears first and then keywords
for session in sessions:
    print(session)
    driver.get("https://leginfo.legislature.ca.gov/faces/billSearchClient.xhtml")
    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#attrSearch"))
    )
    select_session = Select(driver.find_element_by_css_selector('select#session_year'))
    select_session.select_by_visible_text(session)
    search = driver.find_element_by_css_selector('#attrSearch')
    search.click()

    WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody tr")))
    bills = driver.find_elements_by_css_selector('tbody tr')
    for bill in bills:
        if bill.find_element_by_css_selector("td:nth-child(4)").text == 'Chaptered':
            url = bill.find_element_by_css_selector("td:nth-child(1) a").get_attribute('href')
            urls.append(url)
            measure = bill.find_element_by_css_selector("td:nth-child(1) a").text
            measures.append(measure)
            subject = bill.find_element_by_css_selector("td:nth-child(2)").text
            subjects.append(subject)
            author = bill.find_element_by_css_selector("td:nth-child(3)").text
            authors.append(author)
            status = bill.find_element_by_css_selector("td:nth-child(4)").text
            statuss.append(status)
        else:
            print("Do not pass")
        time.sleep(0.01)
print("url done")

# set up list titles
years = []
states = []
sessionyears = []
billnumbers = []
titles = []
briefsummarys = []
introduceddates = []
signingdates = []
effectivedates = []
expireddates = []
leadauthors = []
textlinks = []
fulltexts = []
fulltextpages = []

# extract bill information on each bill

for url in urls[6313:]:
    driver.get(url)
    state = "CA"
    states.append(state)

    billnumber = driver.find_element_by_css_selector('#bill_header > div:nth-child(3) > h1')
    billnumbers.append(billnumber.text.rsplit(" ")[0])

    title = driver.find_element_by_css_selector('#bill_header > div:nth-child(3) > h1')
    title = title.text.split(" ",1)[1]
    titles.append((title.rsplit("(",1)[0]))

    textlinks.append(url)
    fulltext = driver.find_element_by_css_selector("#bill_all").text
    fulltexts.append(fulltext)
    fulltextpage = driver.find_element_by_css_selector("#centercolumn").text
    fulltextpages.append(fulltextpage)
    sleeptime = randint(1,11)*0.1
    time.sleep(sleeptime)

datasource = pd.DataFrame({
    'Year': years,
    'State': states,
    'Session Year': sessionyears,
    'Bill Number': billnumbers,
    'Name': titles,
    'Brief Summary': briefsummarys,
    'Introduced Date': introduceddates,
    'Date it was signed':signingdates,
    'Date effective':effectivedates,
    'Expiration date':expireddates,
    'Introducer':leadauthors,
    'Full text': fulltexts,
    'Link to full text':textlinks
})

datasource = pd.DataFrame({
    'Year': years,
    'State': states,
    'Session Year': sessionyears,
    'Bill Number': billnumbers,
    'Name': titles,
    'authors':authors,
    'Brief Summary': briefsummarys,
    'Introduced Date': introduceddates,
    'Date effective':effectivedates,
    'Expiration date':expireddates,
    'Full text': fulltexts,
    'Link to full text':textlinks
})


# Python program to read
# json file



# Opening JSON file

os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/CA/')
f = open('CA_Leginfo.json')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))


#datasource = json.load(f)

#datasource = pd.read_pickle("CA_Leginfo.pkl")
#datasource = datasource.drop(['authors','Introduced Date','Date effective','Expiration date'], axis = 1)

# save bill info into files
datasource.to_excel('CA_Leginfo.xlsx')
datasource.to_csv('CA_Leginfo.csv')
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
datasource.to_sql('CA_Leginfo.sql',con=engine)
datasource.to_pickle('CA_Leginfo.pkl')
datasource.to_json('CA_Leginfo.json')


datasource.to_stata('CA_Leginfo.dta')

print("download finished")
# End of CA_leginfo_v3.py


# Start of NM_Leginfo.py

# import libraries
## selenium
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import inflect
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import fitz
import re

# set up working directory
working_directory = '/Volumes/SSD/AFRI/'
print(os.path.abspath(working_directory))

# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./Data/Raw_Data/NM/PDFs"

print(os.path.abspath(download_dir))



# Specify the path to the Firefox binary
firefox_binary_path = '/Applications/Firefox.app/Contents/MacOS/firefox'

# Create a Firefox profile and configure preferences
options = Options()
options.binary_location = firefox_binary_path

# Initialize the Firefox driver with the specified options
executable_path = './Data/geckodriver'
print(os.path.abspath(executable_path))

# Initialize the WebDriver with the specified path
service = Service(executable_path)
driver = webdriver.Firefox(service=service)


driver.get('https://www.nmlegis.gov/Legislation/BillFinder/Chaptered')

session_info_elements = driver.find_elements(By.CSS_SELECTOR, '#MainContent_gridViewLegislation > tbody > tr:')

sessions_info = []

for session_element in session_info_elements:
    sessions_info.append(session_element.text)

sessions_info[1]

bill_nums = []

bill_elements = driver.find_elements(By.CSS_SELECTOR, '#MainContent_gridViewLegislation > tbody > tr > td:nth-child(1)')

for element in bill_elements:
    bill_nums.append(element.text)

len(bill_nums)

#MainContent_gridViewLegislation_lblSession_0
#MainContent_gridViewLegislation > tbody:nth-child(2) > tr:nth-child(1) > td:nth-child(5)
#MainContent_gridViewLegislation > tbody:nth-child(2) > tr:nth-child(1) > td:nth-child(5)
sessions = []

session_elements = driver.find_elements(By.CSS_SELECTOR, '#MainContent_gridViewLegislation > tbody > tr > td:nth-child(5)')

for session_element in session_elements:
    print(session_element.text)
    sessions.append(session_element.text)

len(sessions)


df = pd.DataFrame({
    'bill_num': bill_nums,
    'session': sessions
})

df.to_csv("/Volumes/SSD/AFRI/Data/Raw_Data/NM/NM_leginfo_re.csv")

df = pd.read_csv("/Volumes/SSD/AFRI/Data/Raw_Data/NM/NM_leginfo_re.csv")
df = df.dropna(subset='session')
df.head
urls = []


for index, row in df.iterrows():
    year = int(row['session'][:4])
    session_part = row['session'][2:].replace('1st','').replace('2nd','')
    bill_num = row['bill_num'].replace('*', '')
    type = bill_num[:2]
    bill_part = bill_num.split(' ')[1].zfill(4)
    if year > 2003:
        # Construct the URL based on conditions
        urls.append(f'https://www.nmlegis.gov/Sessions/{session_part}/final/{type}{bill_part}.pdf')
        urls.append(f'https://www.nmlegis.gov/Sessions/{session_part}2/final/{type}{bill_part}.pdf')
    else:
        urls.append(f'https://www.nmlegis.gov/Sessions/{session_part}/FinalVersions/{type}{bill_part}.pdf')

for url in urls:
    print(url)

# Specify the path to the Firefox binary
firefox_binary_path = '/Applications/Firefox.app/Contents/MacOS/firefox'

# Create a Firefox profile and configure preferences
options = Options()
options.binary_location = firefox_binary_path

# Initialize the Firefox driver with the specified options
executable_path = './geckodriver'
print(os.path.abspath(executable_path))

# Initialize the WebDriver with the specified path
service = Service(executable_path)
driver = webdriver.Firefox(service=service)


for url in urls:
    print(f"Processing URL: {url}")
    time.sleep(randint(1,5)*0.001)
    # Navigate to the URL using JavaScript
    driver.execute_script(f"window.location.href = '{url}'")
    driver.execute_script("window.location.reload();")








# End of NM_Leginfo.py


# Start of ML_test.py

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import re


class LitAutoEncoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 64),
            nn.ReLU(),
            nn.Linear(64, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 28 * 28))

    def forward(self, x):
        embedding = self.encoder(x)
        return embedding

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('val_loss', loss)


# data
dataset = MNIST('', train=True, download=True, transform=transforms.ToTensor())
mnist_train, mnist_val = random_split(dataset, [55000, 5000])

train_loader = DataLoader(mnist_train, batch_size=32)
val_loader = DataLoader(mnist_val, batch_size=32)

# model
model = LitAutoEncoder()

# training
trainer = pl.Trainer(gpus=4, num_nodes=8, precision=16, limit_train_batches=0.5)
trainer.fit(model, train_loader, val_loader)


# End of ML_test.py


# Start of OR_Leginfo.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import pytest
import unittest
from random import randint
import PyPDF2
import glob
import requests
from bs4 import BeautifulSoup


# driver_path = 'C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe' #for windows
# driver_path = '/Users/Yanxu/OneDrive/Projects/AgPolicyCoding/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
os.chdir('/Users/long/OneDrive/Projects/AFRI')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
dnldpath = {"download.default_directory" : "/Users/long/OneDrive/Projects/AFRI/data/OR/"}
chromeOptions.add_experimental_option("prefs",dnldpath)
driver = webdriver.Chrome(executable_path='./chromedriver', options=chromeOptions)
driver.get("https://www.oregonlegislature.gov/bills_laws/Pages/Oregon-Laws.aspx")


urls=[]
acttxts = []

sessionlist = driver.find_elements_by_css_selector('span.ms-commentexpand-iconouter')

for session in sessionlist:
    session.click()
    sleep(1)

chapters = driver.find_elements_by_css_selector('td.ms-cellstyle.ms-vb2 a')
for chapter in chapters:
    url = chapter.get_attribute('href')
    urls.append(url)

urlslist = list(set(urls))

urlslist.sort()

for idx,url in enumerate(urlslist):
    driver.get(url)
    sleep(1)
    if '.pdf' in url:
        path = "/Users/long/OneDrive/Projects/AFRI/data/OR/"
        list_of_files = glob.glob(os.path.join(path, '*.pdf'))
        latest_file = max(list_of_files, key=os.path.getctime)
        print(latest_file)
        with open(os.path.join(os.getcwd(), latest_file), 'r') as f:  # open in readonly mode
            # creating a pdf File object of original pdf
            pdfFileObj = open(latest_file, 'rb')
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
            pagenumber = pdfReader.numPages
            acttxt = []
            pgtxts = []
            for p in range(pagenumber):
                pageObj = pdfReader.getPage(p)
                pgtxt = pageObj.extractText()
                pgtxts.append(pgtxt)
                acttxt = '\n'.join(pgtxts)
    else:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, "html.parser")
        tag = soup.body
        acttxt = []
        for string in tag.strings:
            acttxt.append(string)
    acttxts.append(acttxt)
datasource = pd.DataFrame({
    'Session Laws': acttxts,
    'Link to full text': urls,
})

datasource.to_excel('OR_Leginfo.xlsx')
datasource.to_csv('OR_Leginfo.csv')


print("Web-scrapting finished")
# End of OR_Leginfo.py


# Start of CA_Leginfo.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
#import calendar
import os

#driver_path = 'C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe' #for windows
#driver_path = '/Users/Yanxu/OneDrive/Projects/AgPolicyCoding/chromedriver'
#driver = webdriver.Chrome(executable_path=driver_path)

#Change the working directory
os.chdir('/Users/long/OneDrive/Projects/AFRI')
#Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))


# set up webdriver by short path
#driver = webdriver.Chrome('/Users/Yanxu/OneDrive/Projects/AgPolicyCoding/chromedriver')
driver = webdriver.Chrome('./chromedriver')

# input keyword(s)
keywords = ['agriculture']

# choose sessionyears
#input_sessionyears = ['2021 - 2022']
input_sessionyears = ['2015 - 2016', '2013 - 2014', '2011 - 2012', '2009 - 2010', '2007 - 2008']
#input_sessionyears = ['2015 - 2016', '2013 - 2014', '2011 - 2012', '2009 - 2010', '2007 - 2008']
#input_sessionyears = ['2021 - 2022', '2019 - 2020', '2017 - 2018', '2015 - 2016', '2013 - 2014', '2011 - 2012', '2009 - 2010', '2007 - 2008', '2005 - 2006', '2003 - 2004', '2001 - 2002', '1999 - 2000']


urls = []
# search bills by iterating sessionyears first and then keywords
for input_sessionyear in input_sessionyears:
    for keyword in keywords:
        driver.get("https://leginfo.legislature.ca.gov/faces/billSearchClient.xhtml")

        # select session year
        select_sessionyear = Select(driver.find_element_by_css_selector('select#session_year'))
        select_sessionyear.select_by_visible_text(input_sessionyear)

        # search by keywords
        input_keywords = driver.find_element_by_css_selector("input#keyword.keyword_text")
        input_keywords.send_keys(keyword)
        search_button = driver.find_element_by_css_selector('input#attrSearch')
        search_button.click()

        no_bills_returned = int(driver.find_element_by_css_selector("#text_bill_returned").text.split(":")[1].split("Bills")[0])

        if no_bills_returned >0:
            # get urls in the first page
            searchresult = driver.find_elements_by_css_selector('div#searchresult tbody a')
            basicinfo = driver.find_elements_by_css_selector('div.commdataRow')

            for i in range(len(searchresult)):
                #for i in range(10):
                url = searchresult[i].get_attribute('href')
                urls.append(url)

            #%%
            # get urls in the middle pages
            text_bill_returned= driver.find_element_by_css_selector('div#text_bill_returned').text
            n_pages = int(text_bill_returned.rsplit('of ')[1].rsplit(" ")[0])
            page_index = int(text_bill_returned.rsplit('Page ')[1].rsplit(' of')[0])
            while page_index < n_pages:
                WebDriverWait(driver, 300).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "input#nextTen"))
                )
                next_page = driver.find_element_by_css_selector('input#nextTen')
                next_page.click()
                WebDriverWait(driver, 300).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div#searchresult tbody a"))
                )
                searchresult = driver.find_elements_by_css_selector('div#searchresult tbody a')
                basicinfo = driver.find_elements_by_css_selector('div.commdataRow')
                for i in range(len(searchresult)):
                    url = searchresult[i].get_attribute('href')
                    urls.append(url)
                WebDriverWait(driver, 300).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div#text_bill_returned"))
                )
                text_bill_returned = driver.find_element_by_css_selector('div#text_bill_returned').text
                page_index = int(text_bill_returned.rsplit('Page ')[1].rsplit(' of')[0])
                sleep(3)
print("done with urls")


# set up list titles
years = []
states = []
sessionyears = []
billnumbers = []
titles = []
briefsummarys = []
introduceddates = []
signingdates = []
effectivedates = []
expireddates = []
leadauthors = []
textlinks = []


# extract bill information on each bill

for url in urls:
    driver.get(url)
    version = driver.find_element_by_css_selector("select#version.bill_version_select")

    if "Chaptered" in version.text:
        introduceddate = version.text.split("- Introduced")[0]
        introduceddate = introduceddate.split(" ")[-2]
        introduceddates.append(introduceddate)

        state = "CA"
        states.append(state)

        billnumber = driver.find_element_by_css_selector('div#bill_title')
        billnumbers.append(billnumber.text.rsplit(" ")[0])

        title = driver.find_element_by_css_selector('div#bill_title h2')
        title = title.text.split(" ",1)[1]
        titles.append((title.rsplit("(",1)[0]))

        try:
            briefsummaryresolution = driver.find_element_by_css_selector('span.Resolution').text
        except:
            briefsummaryresolution = "N/A"

        briefsummarydigest = driver.find_element_by_css_selector('span#digesttext').text

        if not briefsummarydigest:
            briefsummary = briefsummaryresolution
        else:
            briefsummary = briefsummarydigest

        briefsummarys.append(briefsummary)

        if not version.text.find("Chaptered"):
            signingdate = 'N/A'
        else:
            signingdate = version.text.split("- Chaptered")[0].split(" ")[-2]

        signingdates.append(signingdate)

        signingdate_str = str(signingdate)
        signingdate_length = len(signingdate_str)
        shortyear = int(str(signingdate)[signingdate_length-2: signingdate_length])
        if shortyear > 22:
            f"{shortyear:02}"
        else:
            if shortyear < 10:
                year = "200" + str(shortyear)
            else:
                year = "20" + str(shortyear)
        years.append(year)

        if (int(year) % 2) == 0:
            sessionyear = str(int(year)-1)+"-"+str(year)
        else:
            sessionyear = str(year)+str("-")+str(int(year)+1)
        sessionyears.append(sessionyear)

        if "take effect immediately" in briefsummary:
            effectivedate = signingdate
        else:
            if "take effect on or before" in briefsummary:
                effectivedate = briefsummary.split("take effect on or before")[1].split(".")[0]
            elif "take effect on" in briefsummary:
                effectivedate = briefsummary.split("take effect on")[1].split(".")[0]
            else:
                effectivedate = "Need to check"

        effectivedates.append(effectivedate)

        if datefinder.find_dates(briefsummary):
            dates = datefinder.find_dates(briefsummary)
            datelist = []
            try:
                for date in dates:
                    datelist.append(date.date())
                exdtkeys = ['expired', 'repeal', 'extend', 'repealed', 'repeals']
                if any(key in briefsummary for key in exdtkeys):
                    expireddate = max(datelist)
                else:
                    expireddate = 'N/A'
            except:
                expireddate = 'Need to check'
        else:
            expireddate = 'N/A'

        expireddates.append(expireddate)

        driver.find_element_by_css_selector("#nav_bar_top_status > span").click()
        leadauthor = driver.find_element_by_css_selector('#leadAuthors').text
        #try:
        #    leadauthor = driver.find_element_by_css_selector('#leadAuthors').text.replace(" (A)","")
        #except:
        #    leadauthor = driver.find_element_by_css_selector('#leadAuthors').text.replace(" (S)","")
        leadauthors.append(leadauthor)

        textlinks.append(url)
        sleep(5)

datasource = pd.DataFrame({
    'Year': years,
    'State': states,
    'Session Year': sessionyears,
    'Bill Number': billnumbers,
    'Name': titles,
    'Brief Summary': briefsummarys,
    'Introduced Date': introduceddates,
    'Date it was signed':signingdates,
    'Date effective':effectivedates,
    'Expiration date':expireddates,
    'Introducer':leadauthors,
    'Link to full text':textlinks
})

# drop duplicates by "Official ID"
datasource.drop_duplicates(subset = ['Bill Number', 'Year'],
                           keep = 'first', inplace = True, ignore_index= True)

# save bill info into excel file
datasource.to_excel('CA_Leginfo_food.xlsx')
print("download finished")
# End of CA_Leginfo.py


# Start of NASS_data.py

# import libraries
## selenium
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import inflect
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/'
print(os.path.abspath(working_directory))

# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./NASS_Commodities"

print(os.path.abspath(download_dir))


# Specify the path to the Firefox binary
firefox_binary_path = '/Applications/Firefox.app/Contents/MacOS/firefox'

# Create a Firefox profile and configure preferences
options = Options()
options.binary_location = firefox_binary_path

# Initialize the Firefox driver with the specified options
executable_path = './geckodriver'
print(os.path.abspath(executable_path))

# Initialize the WebDriver with the specified path
service = Service(executable_path)
driver = webdriver.Firefox(service=service)



driver.get('https://quickstats.nass.usda.gov/')

states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

commodities = []

commodity_elements = driver.find_elements(By.CSS_SELECTOR, '#commodity_desc > option')

for element in commodity_elements:
    commodities.append(element.text)

df = pd.DataFrame(commodities, columns=['Commodity'])

df.to_csv('NASS_commodities.csv', index=False)

# Replace 'your_api_key' with your actual QuickStats API key
api_key = '07D7A7C0-032B-3C59-BCD4-3ECB111B0BE6'
url = 'https://quickstats.nass.usda.gov/api/api_GET/'

commodity_list = []
state_list = []
download_status = []

#index = commodities.index('CRUSTACEANS')

for commodity in commodities:
    commodity_list.append(commodity)
    # Loop over states
    for state in states:
        state_list.append(state)
        # Example parameters for the API request
        params = {
            'key': api_key,
            'commodity_desc': commodity,
            #'year': '2022',
            'state_alpha': state,
            'format': 'CSV'  # Request data in CSV format
        }

        # Make the API request
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Specify the path to save the CSV file
            csv_file_path = os.path.join(download_dir, f'quickstats_data_{commodity.replace(" ", "_").lower()}_{state.lower()}.csv')

            # Save the CSV data to a file
            with open(csv_file_path, mode='wb') as file:
                file.write(response.content)

            print(f"Data for {commodity} in {state} saved to {csv_file_path}")
            download_status.append('Yes')
        else:
            print(f"Failed to fetch data for {commodity} in {state}. Status code: {response.status_code}")
            download_status.append('No')


# List all CSV files in the directory
files = [file for file in os.listdir(download_dir) if file.endswith('.csv') and not file.startswith('._')]

# Initialize an empty list to hold DataFrames
data_frames = []

# Specify the dtype for problematic columns (replace 'object' with appropriate dtype if known)
dtype_spec = {
    18: 'object',
    21: 'object',
    35: 'object',
    38: 'object'
}

# Loop through the list of CSV files and read each into a DataFrame, then append to the list
for file in files:
    file_path = os.path.join(download_dir, file)
    df = pd.read_csv(file_path)
    data_frames.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
appended_df = pd.concat(data_frames, ignore_index=True)

appended_df.to_csv("./NASS_Commodities/NASS_commodities_data.csv", index=False)


df = pd.read_csv("./NASS_Commodities/NASS_commodities_data.csv")

df = df[(df['agg_level_desc'] == 'STATE') & (df['freq_desc'] == 'ANNUAL')]

df = df.loc[:, ~(df == 'nan').all()]

df['Value'] = df['Value'].fillna('')

# Filter out rows where 'Value' contains '(D)'
df = df[~df['Value'].str.contains(r'\(D\)')]

df = df[df['sector_desc'].isin(['ANIMALS & PRODUCTS', 'CROPS'])]

commodities_list = df[df['sector_desc'].isin(['ANIMALS & PRODUCTS', 'CROPS'])]
commodities_list = commodities_list['commodity_desc'].drop_duplicates().tolist()
commodities_list = [commodity.replace(' TOTALS', '') for commodity in commodities_list]

commodities_list = list(set(commodities_list))

commodities_list = pd.DataFrame(commodities_list)

commodities_dict_list = []

# Define the list to hold the dictionary values
commodity_dicts = []


# Initialize an inflect engine
p = inflect.engine()

# Iterate over each commodity description in the DataFrame
for commodity in df['commodity_desc']:
    # Convert the commodity description to lowercase
    commodity_lower = commodity.lower()

    # Check if the commodity is in singular form
    if p.singular_noun(commodity_lower):
        # If it's plural, we generate singular form and use the original as plural form
        original_commodity = commodity_lower
        revised_commodity = commodity_lower.replace(', other', '')
        plural_commodity = commodity_lower
        singular_commodity = p.singular_noun(commodity_lower)
    else:
        # If it's singular, we generate plural form and use the original as singular form
        original_commodity = commodity_lower
        revised_commodity = commodity_lower.replace(', other', '')
        singular_commodity = commodity_lower
        plural_commodity = p.plural(commodity_lower)


    # Perform transformations and store results in a list
    commodity_forms = [
        commodity_lower,  # original_commodity
        commodity_lower.replace(', other', ''),  # revised_commodity
        commodity_lower,  # singular_commodity
        p.plural(commodity_lower)  # plural_commodity
    ]

    # Create a dictionary with the original and generated forms
    #commodity_dict = {
    #    'original': original_commodity,
    #    'singular': singular_commodity,
    #    'plural': plural_commodity,
    #    'revised': revised_commodity
    #}

    # Append the dictionary to the list
    commodity_dicts.append(commodity_dict)

# Create the new column in the DataFrame
df['commodity_forms'] = commodity_dicts

df.to_csv('NASS_commodities_animal_crop.csv', index=False)

df = pd.read_csv("./NASS_Commodities/NASS_commodities_animal_crop.csv")

# Group by 'STATE' and save each group to a separate CSV file
for state, group in df.groupby('state_alpha'):
    filename = f"./NASS_Commodities/{state}_commodity_price.csv"
    group.to_csv(filename, index=False)
    print(f"Saved {filename}")

















# Iterate over each commodity description in commodities_list
for commodity in df['commodity_desc']:
    # Convert the commodity description to lowercase
    commodity_lower = commodity.lower()

    # Check if the commodity is in singular form
    if p.singular_noun(commodity_lower):
        # Generate plural form
        original_commodity = commodity_lower
        revised_commodity = commodity_lower.replace(', other', '')
        plural_commodity = p.plural(commodity_lower)
        singular_commodity = commodity_lower
    else:
        # Generate singular form
        original_commodity = commodity_lower
        revised_commodity = commodity_lower.replace(', other', '')
        singular_commodity = p.singular_noun(commodity_lower)
        plural_commodity = commodity_lower

    # Create a dictionary with the original and generated forms
    commodity_dict = {'original':commodity_lower, 'singular': singular_commodity, 'plural': plural_commodity, 'revised': revised_commodity}

    # Append the dictionary to the list
    commodities_dict_list.append(commodity_dict)

commodities_list['commodities_dict'] = commodities_dict_list


# Convert commodities_list to DataFrame with a column name 'commodity_desc'
commodities_list_df = pd.DataFrame({'commodity_desc': commodities_list})

# Combine both DataFrames horizontally
commodities_df = pd.concat([commodities_dict_df, commodities_list_df], axis=1)


df.to_csv("NASS_commodities_state_annual.csv", index=False)

df = df.drop_duplicates()



df.columns()

print(df.columns.tolist())


# List of columns to keep
columns_to_keep = ['source_desc', 'sector_desc', 'group_desc', 'commodity_desc']

# Keep only the specified columns using filter
df_filtered = df.filter(items=columns_to_keep)

df_filtered.to_csv("NASS_commodities_list.csv", index=False)



df = pd.read_csv('/Volumes/SSD/AFRI/Data/NASS_Commodities/combined_data/NASS_commodities_state.csv')



df = df.drop_duplicates(subset=['commodity_desc', 'class_desc', 'year', 'state_alpha'])


df = df[df['class_desc'] != 'ALL CLASSES']

df_country = df.drop_duplicates(subset=['commodity_desc', 'class_desc', 'year'])

df_country.to_csv('us_commodity_classes.csv', index=False)

df_apple = df_country[df_country['commodity_desc'] == 'APPLES']
df_apple = df[df['commodity_desc'] == 'APPLES']

df_apple.to_excel('apple.xlsx', index=False)

df_wheat = df[df['commodity_desc'] == 'WHEAT']

df_wheat.to_excel('us_commodity_classes_state.xlsx', index=False)




for state in states:
    # Define the parameters for the API request
    params = {
        'key': api_key,
        'commodity_desc': 'APPLES',  # Request data for APPLES
        #'year': '2022',
        'state_alpha': state,
        'format': 'CSV'  # Request data in CSV format
    }

    # Define the URL for the API request
    url = 'https://quickstats.nass.usda.gov/api/api_GET/'  # Replace with the actual API endpoint URL

    # Make the API request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Specify the path to save the CSV file
        csv_file_path = os.path.join(f'apples_{state.lower()}.csv')

        # Save the CSV data to a file
        with open(csv_file_path, mode='wb') as file:
            file.write(response.content)

        print(f"Data in {state} saved to {csv_file_path}")
    else:
        print(f"Failed to fetch data for {commodity} in {state}. Status code: {response.status_code}")

data_frames = []

download_dir = '/Volumes/SSD/AFRI/Data/apple'

# List all CSV files in the directory
files = [file for file in os.listdir(download_dir) if file.endswith('.csv') and not file.startswith('._')]


# Loop through the list of CSV files and read each into a DataFrame, then append to the list
for file in files:
    file_path = os.path.join(download_dir, file)
    df = pd.read_csv(file_path)
    data_frames.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
apple = pd.concat(data_frames, ignore_index=True)

apple.to_excel('apple.xlsx', index=False)

apple_country = apple.drop_duplicates(subset=['commodity_desc', 'class_desc', 'year'])
apple_country.to_excel('apple.xlsx', index=False)




df = pd.read_csv('/Volumes/SSD/AFRI/Data/NASS_Commodities/combined_data/NASS_commodities_state.csv')

df_survey = df[df['source_desc'] == 'SURVEY' | df['source_desc'] == 'CROPS']

df_survey = df_survey[df_survey['freq_desc'] == 'ANNUAL']

df_survey = df_survey[df_survey['year'] != 2024]

df_survey.to_csv('NASS_commodities_state_survey.csv')

df_survey_us =  df_survey.drop_duplicates(subset=['commodity_desc', 'class_desc', 'year'])

df_survey_us.to_excel('NASS_commodities_survey_US.xlsx', index=False)

# Step 1: Group by 'commodity_desc' and 'year' and count unique 'class_desc'
grouped = df_survey_us.groupby(['commodity_desc', 'year'])['class_desc'].nunique().reset_index(name='unique_class_count')

# Step 2: Identify groups with more than 1 unique 'class_desc'
groups_with_multiple_classes = grouped[grouped['unique_class_count'] > 2]

# Step 3: Filter out 'ALL CLASSES' rows for these identified groups
# Merge to find the rows that should be filtered out
result_df = df_survey_us.merge(groups_with_multiple_classes[['commodity_desc', 'year']], on=['commodity_desc', 'year'], how='left', indicator=True)

# Filter out rows where 'ALL CLASSES' should be removed
df_survey_us_filtered = result_df[~((result_df['_merge'] == 'both') & (result_df['class_desc'] == 'ALL CLASSES'))].drop(columns=['_merge'])


df_survey_us_filtered.to_excel('NASS_commodities_US_filtered.xlsx')

# Group by 'year' and 'commodity_desc', then count the unique 'class_desc'
results_table = df_survey_us_filtered.groupby(['year', 'commodity_desc'])['class_desc'].nunique().reset_index()

# Rename the 'class_desc' column to 'count_class_desc' to reflect the counts
results_table = results_table.rename(columns={'class_desc': 'count_class_desc'})

results_table.to_csv('NASS_commodities_US_varieties.csv')
results_table.to_excel('NASS_commodities_US_varieties.xlsx', index=False)

# Plotting
fig, ax = plt.subplots(figsize=(60, 40))

# Iterate over unique 'commodity_desc' to plot each as a separate line
for commodity in results_table['commodity_desc'].unique():
    subset = results_table[results_table['commodity_desc'] == commodity]
    ax.plot(subset['year'], subset['count_class_desc'], marker='o', label=commodity)

# Add labels and title
ax.set_xlabel('Year')
ax.set_ylabel('Count of Unique Class Desc')
ax.set_title('Number of Unique Class Descriptions for Each Commodity by Year')
ax.legend(title='Commodity')

# Show the plot
plt.show()



df_census = df[df['source_desc'] == 'CENSUS']

df_survey = df[(df['sector_desc'] == 'ANIMALS & PRODUCTS') | (df['sector_desc'] == 'CROPS')]

df_census = df_census[df_census['freq_desc'] == 'ANNUAL']

df_census = df_census[df_census['year'] != 2024]

df_census.to_csv('NASS_commodities_state_census.csv')

df_census_us =  df_census.drop_duplicates(subset=['commodity_desc', 'class_desc', 'year'])

df_census_us.to_excel('NASS_commodities_census_US.xlsx', index=False)

# Step 1: Group by 'commodity_desc' and 'year' and count unique 'class_desc'
grouped = df_census_us.groupby(['commodity_desc', 'year'])['class_desc'].nunique().reset_index(name='unique_class_count')

# Step 2: Identify groups with more than 1 unique 'class_desc'
groups_with_multiple_classes = grouped[grouped['unique_class_count'] > 2]

# Step 3: Filter out 'ALL CLASSES' rows for these identified groups
# Merge to find the rows that should be filtered out
result_df = df_census_us.merge(groups_with_multiple_classes[['commodity_desc', 'year']], on=['commodity_desc', 'year'], how='left', indicator=True)

# Filter out rows where 'ALL CLASSES' should be removed
df_census_us_filtered = result_df[~((result_df['_merge'] == 'both') & (result_df['class_desc'] == 'ALL CLASSES'))].drop(columns=['_merge'])


df_census_us_filtered.to_excel('NASS_commodities_US_census_filtered.xlsx')

# Group by 'year' and 'commodity_desc', then count the unique 'class_desc'
results_table = df_census_us_filtered.groupby(['year', 'commodity_desc'])['class_desc'].nunique().reset_index()

# Rename the 'class_desc' column to 'count_class_desc' to reflect the counts
results_table = results_table.rename(columns={'class_desc': 'count_class_desc'})

results_table.to_csv('NASS_commodities_US_varieties_census.csv')
results_table.to_excel('NASS_commodities_US_varieties_census.xlsx', index=False)









# End of NASS_data.py


# Start of NE_Leginfo.py

# import libraries
import pandas as pd
import os
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from word_forms.word_forms import get_word_forms
import datefinder
from dateutil import parser
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import inflect
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import fitz


# Specify the path to the Firefox binary
firefox_binary_path = '/Applications/Firefox.app/Contents/MacOS/firefox'

# Create a Firefox profile and configure preferences
options = Options()
options.binary_location = firefox_binary_path

# Initialize the Firefox driver with the specified options
executable_path = './geckodriver'
print(os.path.abspath(executable_path))

# Initialize the WebDriver with the specified path
service = Service(executable_path)
driver = webdriver.Firefox(service=service)




urls=[]
sessionurls = []
section_urls = []
chs = list(range(1,91))
for ch in chs:
    ch = str(ch)
    ch_url = "https://nebraskalegislature.gov/laws/browse-chapters.php?chapter=" + ch
    driver.get(ch_url)
    sections = driver.find_elements_by_css_selector("span.col-md-2.col-sm-3.my-auto:nth-child(1) a")
    for section in sections:
        section_url = section.get_attribute('href')
        section_urls.append(section_url)

sections = []
bill_urls = []
for section_url in section_urls:
    driver.get(section_url)
    section_text = driver.find_element_by_css_selector("div.card-body").text
    sections.append(section_text)
    try:
        bills = driver.find_elements_by_partial_link_text("Laws ")
        for bill in bills:
            bill_url = bill.get_attribute('href')
            bill_urls.append(bill_url)
    except:
        print("no source")
print("done with bill urls")

billurls = list(set(bill_urls))


for url in billurls:
    print(f"Processing URL: {url}")

    # Navigate to the URL using JavaScript
    driver.execute_script(f"window.location.href = '{url}'")
    wait_for_page_load(driver)  # Wait for the page to load

    time.sleep(randint(1, 5) * 0.01)  # Additional sleep to simulate user delay

    try:
        # Refresh the page
        driver.execute_script("window.location.reload();")
        wait_for_page_load(driver)  # Wait again after refresh
        time.sleep(randint(1, 5) * 0.01)  # Wait to ensure the page is properly loaded after refresh
    except Exception as e:
        print(f"An error occurred while refreshing the page: {e}")

act_txts = []
urls = []
path = f"Raw_Data/{state}/PDFs"
files = glob.glob(os.path.join(path, '*.pdf'))
for file in files:
    print(file)
    urls.append(get_mdls_where_from(file))
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        act_txt = ' '.join(pgtxts)
    act_txts.append(act_txt)
    doc.close()


datasource = pd.DataFrame({
    'original_text': acttxts,
    'link':billurls
})

datasource['session'] = datasource['link'].apply(lambda x: x.split('FloorDocs/')[1].split('/PDF')[0])

datasource['year'] = datasource['original_text'].apply(lambda x: x.split('AN\nACT')[0].split('BILL')[1].replace('\n', ' ').split('Introduced')[0].replace('(corrected)','').replace('(CORRECTED)','').replace('Corrected Copy','').split('the ')[1][-8:].replace(' ','').replace('.','')[-4:])

datasource['year'][1645] = '2001'
datasource['year'][707] = '2000'
datasource['year'][762] = '2017'

datasource['year'].astype(int)

datasource['year'] = datasource['year'].astype(int)

datasource = datasource[(datasource['year'] >= 1975) & (datasource['year'] <= 2021)]


# Save the DataFrame to an Excel file
datasource.to_excel(f"{state}_Leginfo.xlsx", index=False)

# Save the DataFrame to a CSV file
datasource.to_csv(f'{state}_Leginfo.csv', index=False)

# Save the DataFrame to a Pickle file
datasource.to_pickle(f'{state}_Leginfo.pkl')

# Save the DataFrame to a JSON file
datasource.to_json(f'{state}_Leginfo.json', orient='records')

# End of NE_Leginfo.py


# Start of KS_Leginfo.py

# import libraries
## selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.firefox.service import Service
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import fitz
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import requests
from bs4 import BeautifulSoup

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/KS'
print(os.path.abspath(working_directory))

# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Create a Firefox profile and configure preferences
options = Options()


# Initialize the Firefox driver with the specified options
executable_path='../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)

years = []
act_urls = []
acttxts = []
bad_act_urls = []

for year in range(1996,2011):
    session_url = "https://kslegislature.org/historical_data/sessionlaws/" + str(year) + "/"
    driver.get(session_url)
    time.sleep(0.001)

    if year < 2004:
        acts = driver.find_elements(By.PARTIAL_LINK_TEXT, "html")
    else:
        acts = driver.find_elements(By.PARTIAL_LINK_TEXT, "pdf")

    for act in acts:
        years.append(year)
        act_url = act.get_attribute('href')
        act_urls.append(act_url)
        name = str(act_url).replace("/", "").split("sessionlaws")[1]
        path = download_dir + name
        print(print(os.path.abspath(path)))
        # Send a GET request to the URL
        response = requests.get(act_url)
        # Ensure the request was successful
        if response.status_code == 200:
            # Write the content of the response to a file
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to fetch PDF. Status code:", response.status_code)
            bad_act_urls.append(act_url)

bill_urls_2011_2022 = []
for session_url in ["http://www.kslegislature.org/li_2022/b2021_22/measures/bills/",
                    "http://www.kslegislature.org/li_2021s/b2021s/measures/bills/",
                    "http://www.kslegislature.org/li_2020s/b2020s/measures/bills/",
                    "http://www.kslegislature.org/li_2020/b2019_20/measures/bills/",
                    "http://www.kslegislature.org/li_2018/b2017_18/measures/bills/",
                    "http://www.kslegislature.org/li_2016s/b2015_16/measures/bills/",
                    "http://www.kslegislature.org/li_2016/b2015_16/measures/bills/",
                    "http://www.kslegislature.org/li_2014/b2013_14/measures/bills/",
                    "http://www.kslegislature.org/li_2013s/b2013_14/measures/bills/",
                    "http://www.kslegislature.org/li_2012/b2011_12/measures/bills/"]:
    print(session_url)
    driver.get(session_url)
    time.sleep(1)

    try:
        page_no = driver.find_element(By.CSS_SELECTOR,"#tab-disp").text.split("of ")[1]
        "for more than 1 pages of bills"
        for page in range(1,int(page_no)+1):
            print(page)
            page_url = session_url + "#" + str(page)
            #print(page_url)
            driver.get(page_url)
            time.sleep(0.01)
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#bill-tab-1"))
            )
            bills = driver.find_elements(By.CSS_SELECTOR, f"#bill-tab-{page} li a")

            for bill in bills:
                bill_url = bill.get_attribute("href")
                bill_urls_2011_2022.append(bill_url)
    except:
        bills = driver.find_elements(By.CSS_SELECTOR, "#bill-tab-1 > li a")
        for bill in bills:
            bill_url = bill.get_attribute("href")
            bill_urls_2011_2022.append(bill_url)


not_found_urls = []

for url in bill_urls_2011_2022:
    print(url)
    try:
        driver.get(url)
        version = driver.find_element(By.CSS_SELECTOR, "#version-tab-1 > tr").text

        if "Enrolled" in version:
            act_url = driver.find_element(By.CSS_SELECTOR,
                                          "#version-tab-1 > tr:nth-child(1) > td:nth-child(2) > a").get_attribute('href')

            act_urls.append(act_url)

            # get the act year
            try:
                year = driver.find_element(By.CSS_SELECTOR,
                                           "#history-tab-1 > tr:nth-child(1) > td:nth-child(1)").text[-4:]
            except:
                year = driver.find_element(By.CSS_SELECTOR,
                                           "#version-tab-1 > tr:nth-child(1) > td:nth-child(1)").text[-4:]
            years.append(year)

            name = year + str(act_url).replace("/", "").split("documents")[1]
            path = download_dir + name
            #print(os.path.abspath(path))
            # Send a GET request to the URL
            response = requests.get(act_url)
            # Ensure the request was successful
            if response.status_code == 200:
                # Write the content of the response to a file
                with open(path, 'wb') as file:
                    file.write(response.content)
            else:
                print("Failed to fetch PDF. Status code:", response.status_code)
                bad_act_urls.append(act_url)
        else:
            pass
    except:
        not_found_urls.append(url)
# missing url
# http://www.kslegislature.org/li_2014/b2013_14/measures/hb2070/

# Get index of the element
#index = bill_urls_2011_2022.index("http://www.kslegislature.org/li_2014/b2013_14/measures/hb2070/")

act_txts = []
file_names = []

badfiles = []

folder_path = "./PDFs"
print(os.path.abspath(folder_path))

files = glob.glob(os.path.join(folder_path, '*'))

for file in files:
    file_names.append(file.replace("./PDFs/", "")[4:].replace(".pdf","").replace(".html",""))
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        acttxt = ' '.join(pgtxts)
    act_txts.append(acttxt)
    doc.close()


datasource = pd.DataFrame({
    'original_text': act_txts,
    'year': years,
    'state': "KS",
    'file_name': file_names,
    'link':act_urls
})



datasource.to_excel('KS_Leginfo.xlsx')
datasource.to_csv('KS_Leginfo.csv')
datasource.to_pickle('KS_Leginfo.pkl')
datasource.to_json('KS_Leginfo.json')



# End of KS_Leginfo.py


# Start of NV_Leginfo.py

# import libraries
## selenium
import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
## others
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
import fitz
import re

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/NV'
print(os.path.abspath(working_directory))


# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Initialize the Firefox driver with the specified options
executable_path = '../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)

# empty list to store info

act_pdf_urls = []
act_html_urls = []
bill_nums = []
chapter_nums = []
years = []
sessions = []
broken_act_urls = []
act_txts = []
act_names = []
names = []
file_names = []


# New Jersey Legislature Website: https://www.njleg.state.nj.us/chapter-laws

driver.get('https://www.leg.state.nv.us/law1.html')

WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.col-md-2 > div'))
)

divs = driver.find_elements(By.CSS_SELECTOR,'div.col-md-2 > div')
session_elements = []
session_names = []
for div in divs:
    match = re.search(r"Session \((\d{4})\)", div.text)
    if match:
        year = int(match.group(1))
        if year > 1974 and year < 2022:
            session_elements.append(div)
            session_names.append(div.text.split('Bills')[0])

session_content_list_urls = []
for session in session_elements:
    session_content_list_urls.append(session.find_element(By.PARTIAL_LINK_TEXT, 'Bills by Chapter Number').get_attribute('href'))


datasource_session = pd.DataFrame({
    'session_name':session_names,
    'session_content_list_url': session_content_list_urls
})


session_content_lists = []
act_html_urls = []
session_nums = []

for index,row in datasource_session.iterrows():
    session_num = row['session_name']
    print(session_num)
    url = row['session_content_list_url']
    print(url)
    driver.get(url)
    WebDriverWait(driver, 60).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'body'))
    )
    session_content_list = driver.find_element(By.CSS_SELECTOR,'body').text

    rows = driver.find_elements(By.CSS_SELECTOR,'.MsoNormalTable > tbody:nth-child(1) > tr')[2:]
    for index, row in enumerate(rows):
        #print(row.text)
        try:
            page = int(row.find_element(By.CSS_SELECTOR,':nth-child(3)').text)
            act_html_url = driver.find_element(By.LINK_TEXT, f'{page}').get_attribute('href')
            act_html_urls.append(act_html_url)
        except:
            act_html_urls.append('NA')
        session_content_lists.append(session_content_list)
        session_nums.append(session_num)

datasource_session_content = pd.DataFrame({
    'act_html_url':act_html_urls,
    'session_num':session_nums,
    'session_content_list': session_content_lists
})

datasource_session_content = datasource_session_content[~(datasource_session_content['act_html_url'] == 'NA')]

session_txts = []
act_html_urls = []
for index,row in datasource_session_content.iterrows():
    url = row['act_html_url']
    act_html_urls.append(url)
    print(url)
    driver.get(url)
    WebDriverWait(driver, 60).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'body'))
    )
    session_txts.append(driver.find_element(By.CSS_SELECTOR,'body').text)

datasource_session_content['session_txt'] = session_txts

datasource_session_content = datasource_session_content.drop_duplicates(subset=['session_txt'])
datasource_session_content = datasource_session_content.reset_index(drop=True)

datasource_session_content = pd.read_csv('NV_Leginfo_sessionclean.csv',index_col=0)

act_html_urls = []
session_nums = []
session_content_lists = []
act_txts = []

for index,row in datasource_session_content.iterrows():
    acts = row['session_txt'].split('________')
    for act in acts:
        act_txts.append(act)
        session_nums.append(row['session_num'])
        act_html_urls.append(row['act_html_url'])
        session_content_lists.append(row['session_content_list'])

datasource = pd.DataFrame({
    'act_url':act_html_urls,
    'session_num':session_nums,
    'session_content_list': session_content_lists,
    'original_text':act_txts
})

datasource = datasource.drop_duplicates(subset=['original_text'])
datasource = datasource.reset_index(drop=True)

datasource['session_num'] = datasource['session_num'].str.replace('Statutes of Nevada','')
datasource['session_num'] = datasource['session_num'].str.replace('\n','')

datasource['year'] = datasource['session_num'].str.split('Session').str[1].str[2:6]

datasource['original_text'] = datasource['original_text'].str.replace('\n \n…………………………………………………………………………………………………………………','')
datasource['original_text'] = datasource['original_text'].str.replace('…………………………………………………………………………………………………………………','')

datasource = datasource[~datasource['original_text'].str.contains("SESSION OF THE LEGISLATURE")]

def keep_after_bill(text):
    for phrase in ["Assembly Bill No.", "Senate Bill No."]:
        if phrase in text:
            parts = text.split(phrase)
            return phrase + parts[-1]  # Return the part after the phrase, including the phrase itself
    return text  # Return the original text if none of the phrases are found

# Apply the function to each row in the 'original_text' column
datasource['original_text'] = datasource['original_text'].apply(keep_after_bill)

datasource = datasource[datasource['original_text'].str.contains("Assembly Bill No.|Senate Bill No.")]

def extract_bill_number(text):
    # Pattern to find "Assembly Bill No. <number>" or "Senate Bill No. <number>"
    pattern = r'(Assembly Bill No\.|Senate Bill No\.) (\d+)'
    match = re.search(pattern, text)

    if match:
        bill_prefix = 'AB' if 'Assembly' in match.group(1) else 'SB'
        bill_number = match.group(2)
        return bill_prefix + bill_number
    else:
        return None  # or some default value if needed

# Apply the function to the 'original_text' column and create a new 'bill_num' column
datasource['bill_num'] = datasource['original_text'].apply(extract_bill_number)
datasource = datasource.drop(columns=['session_content_list'])


datasource.to_excel('NV_Leginfo.xlsx')
datasource.to_csv('NV_Leginfo.csv')
datasource.to_pickle('NV_Leginfo.pkl')
datasource.to_json('NV_Leginfo.json')

datasource = pd.read_csv('NV_Leginfo.csv',index_col=0)

# End of NV_Leginfo.py


# Start of NH_Leginfo.py

# import libraries
## selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
from bs4 import BeautifulSoup
from collections import Counter


# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/NH'
print(os.path.abspath(working_directory))

# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Initialize the Firefox driver with the specified options
executable_path='../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)

# session years: 1989-2021

years = []
act_urls = []
original_act_num = []
bad_act_urls = []

for year in range(1989,2022):
    print(year)
    driver.get('https://www.gencourt.state.nh.us/bill_status/legacy/bs2016/')
    # Find the input element by its css "#txtsessionyear"
    sessionyear = driver.find_element(By.CSS_SELECTOR, "input#txtsessionyear")
    # Clear the existing value in the input field
    sessionyear.clear()

    # Input the current year into the input field
    sessionyear.send_keys(str(year))

    driver.find_element(By.CSS_SELECTOR, "#Table1 > tbody:nth-child(1) > tr:nth-child(13) > td:nth-child(1) > input:nth-child(1)").click()

    # click submit
    driver.find_element(By.CSS_SELECTOR, "#cmdsubmit").click()

    # Wait for a brief moment (you may adjust the sleep time as needed)
    time.sleep(randint(1, 10))

    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#btnsh'))
    )

    time.sleep(randint(1, 10)*0.01)

    chapters = driver.find_elements(By.XPATH,
                                    "//*[contains(text(), 'SIGNED BY GOVERNOR') or contains(text(), 'LAW WITHOUT SIGNATURE') or contains(text(), 'VETO OVERRIDDEN')]/../../../../../..")

    print(len(chapters))

    for index, chap in enumerate(chapters):

        print(index)

        time.sleep(randint(1, 10) * 0.01)

        print(index)

        original_act_num.append(chap.find_element(By.XPATH, "./*[1]/*[1]").text)

        years.append(year)

        try:

            act_url = chap.find_element(By.XPATH, "./*[1]//*[contains(text(), 'Bill Text')]").get_attribute('href')

        except:

            act_url = chap.find_element(By.XPATH, "./*[1]//*[contains(text(), 'HTML')]").get_attribute('href')


        act_urls.append(act_url)

        name = str(datasource['year'][index]) + str(datasource['original_act_num'][index]) + ".html"
        path = download_dir + name
        print(os.path.abspath(path))

        time.sleep(randint(1, 100) * 0.02)

        response = requests.get(act_url)
        # Ensure the request was successful
        if response.status_code == 200:
            # Write the content of the response to a file
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to fetch PDF. Status code:", response.status_code)
            bad_act_urls.append(act_url)




# Calculating the frequency of each element in the list
frequency = Counter(years)

# Printing the frequency of each element
for element, count in frequency.items():
    print(f"Element {element} appears {count} times")

datasource = pd.DataFrame({
    #'original_text': act_txts,
    'year': years,
    'state': "NH",
    'link':act_urls,
    'original_act_num':original_act_num
})


datasource = pd.DataFrame({
    #'original_text': act_txts,
    'name': names,
})

names = []

datasource = pd.read_excel('./extra_urls.xlsx')

for index, act_url in enumerate(datasource['link']):
    print(index)
    name = str(datasource['year'][index]) + str(datasource['original_act_num'][index]).split('-FN')[0].replace(' ','') + ".html"
    path = download_dir + name
    names.append(name)
    print(os.path.abspath(path))

    driver.get(act_url)

    time.sleep(randint(1, 3)*0.01)

    response = requests.get(act_url)
    # Ensure the request was successful
    if response.status_code == 200:
        # Write the content of the response to a file
        with open(path, 'wb') as file:
            file.write(response.content)
    else:
        print("Failed to fetch PDF. Status code:", response.status_code)
        bad_act_urls.append(act_url)

    WebDriverWait(driver, 60).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'body'))
    )


def read_html_files(folder_path):
    html_files = [f for f in os.listdir(folder_path) if f.endswith('.html')]
    extracted_info = []

    for file in html_files:
        with open(os.path.join(folder_path, file), 'r', encoding='ISO-8859-1') as f:
            soup = BeautifulSoup(f, 'lxml')
            # Example: Extracting all paragraphs
            paragraphs = soup.find_all('p')
            extracted_info.append({
                'original_text': [p.text for p in paragraphs]
            })

    return extracted_info

# Use the function
original_text = read_html_files(download_dir)

# Iterate over all files in the directory
for filename in os.listdir(download_dir):
    if filename.endswith(".html"):
        # Form the path to the HTML file
        filepath = os.path.join(download_dir, filename)

        # Open and read the HTML file
        with open(filepath, "r", encoding="utf-8") as file:
            html_content = file.read()

        # Parse HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract content as needed
        # Example: Extract all text content
        text_content = soup.get_text()

        # Example: Extract specific elements like <p>
        paragraphs = soup.find_all('p')
        for paragraph in paragraphs:
            print(paragraph.text)


file_name = glob.glob(os.path.join(download_dir, '*.html'))


datasource = pd.DataFrame({
    #'original_text': act_txts,
    'year': years,
    'state': "NH",
    'link':act_urls,
    'original_act_num':original_act_num,
    'file_name':names
})

datasource['file_name'] = names

datasource = datasource.drop_duplicates(subset='link', keep='first')

datasource.to_csv('NH_Leginfo_url.csv')

datasource.to_excel('NH_Leginfo.xlsx')
datasource.to_csv('NH_Leginfo.csv')
datasource.to_pickle('NH_Leginfo.pkl')
datasource.to_json('NH_Leginfo.json')




# End of NH_Leginfo.py


# Start of SC_Leginfo.py

# import libraries
import pandas as pd
import os
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from word_forms.word_forms import get_word_forms
import datefinder
from dateutil import parser
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import inflect
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import fitz

# Set the working directory
os.chdir('/Volumes/SSD/AFRI/Data/')

# Verify the current working directory
print(os.getcwd())


# Specify the path to the Firefox binary
firefox_binary_path = '/Applications/Firefox.app/Contents/MacOS/firefox'

# Create a Firefox profile and configure preferences
options = Options()
options.binary_location = firefox_binary_path

# Initialize the Firefox driver with the specified options
executable_path = './geckodriver'
print(os.path.abspath(executable_path))

# Initialize the WebDriver with the specified path
service = Service(executable_path)
driver = webdriver.Firefox(service=service)




# driver_path = 'C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe' #for windows
# driver_path = '/Users/Yanxu/OneDrive/Projects/AgPolicyCoding/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
os.chdir('/Users/long/OneDrive/Projects/AFRI')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# chromeOptions = webdriver.ChromeOptions()
dnldpath = {"download.default_directory" : "/Users/long/OneDrive/Projects/AFRI/data/SC/"}
chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_experimental_option('prefs', {
    "download.default_directory": dnldpath, #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,#It will not show PDF directly in chrome
    "--enable-javascript":True
})
driver = webdriver.Chrome(executable_path='./chromedriver', options = chromeOptions)
driver.get("https://www.scstatehouse.gov/aacts.php")

session_urls = []
urls =[]
docurls = []
acttxts = []
titles = []
yrs = []
billnums= []
billnumlist = []
specialyears = ['1995','1996']
sgurls = []
summarys = []


driver.get("https://www.scstatehouse.gov/aacts.php")

sessions = driver.find_elements_by_css_selector("div#contentsection a")

for idx, session in enumerate(sessions):
    sessionname = session.text
    if "Excel" in sessionname:
        print(sessionname)
    else:
        session_url = session.get_attribute('href')
        session_urls.append(session_url)
for idx, session_url  in enumerate(session_urls):
    driver.get(session_url )
    sleep(1)
    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "h2.barheader"))
    )
    yr = driver.find_element_by_css_selector("h2.barheader").text.rsplit("Acts")[1].rsplit("Session")[0].rsplit(" ")[0]
    if yr in specialyears:
        sessiongroup = driver.find_elements_by_css_selector("div#resultsbox dl dd a")
        for element in sessiongroup:
            sgurl = element.get_attribute('href')
            driver.get(sgurl)
            WebDriverWait(driver, 300).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div#contentsection"))
            )
            lawlist = driver.find_elements_by_css_selector("div#contentsection dl dt a")
            for element in lawlist:
                url = element.get_attribute('href')
                urls.append(url)
    else:
        WebDriverWait(driver, 300).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div#resultsbox dl dd"))
        )
        lawlist = driver.find_elements_by_xpath('//*[@id="resultsbox"]/ dl / dt / a[1]')
        for element in lawlist:
            url = element.get_attribute('href')
            urls.append(url)

for idx, url in enumerate(urls):
    try:
        driver.get(url)
        WebDriverWait(driver, 300).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div#resultsbox"))
        )
        summary = driver.find_element_by_css_selector("div#resultsbox").text
    except:
        WebDriverWait(driver, 300).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
        )
        summary = driver.find_element_by_css_selector("body").text
    # summarys
    summarys.append(acttxt)
    try:
        textloc = driver.find_element_by_css_selector("div.bill-list-item a.nodisplay")
        # docurls
        docurl = textloc.get_attribute('href')
        docurls.append(docurl)
        sleep(2)
        driver.get(docurl)
    except:
        docurls.append(url)
    body = driver.find_element_by_css_selector("body")
    # acttxts
    acttxt = body.text
    acttxts.append(acttxt)

sessions1 = [i.split('Bill')[0] for i in acttxts]
sessions2 = [i.split('Download')[0] for i in sessions1]
sessions = [i.split('\n')[1] for i in sessions2]
datasource = pd.DataFrame({
    #'Year': yrs,
    # 'State': sts,
    'Session Year': sessions,
    #'Bill Number': billnums,
    #'Title': titles,
    #'Brief Summary': summarys,
    # 'Introduced Date': introdts,
    #'Date it was signed':sigdts,
    # 'Date effective':effdts,
    'Full Text':acttxts,
    'Link to full text': docurls,
    'Link to bill': urls
})




driver.get('https://www.scstatehouse.gov/listofacts.php?O=&Y=2021')

url_elements = driver.find_elements(By.LINK_TEXT, 'Word')
urls = []
for element in url_elements:
    element.click()
    url = element.get_attribute('href')
    urls.append(url)

import textract
import docx2txt

# Define the folder path
folder_path = '/Volumes/SSD/AFRI/Data/Raw_Data/SC/2021'

# Function to read text from a .docx file using textract
# Function to read text from a .docx file using docx2txt
def read_docx(file_path):
    try:
        text = docx2txt.process(file_path)
        return text
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None



# List to store the content of all files
SC2021 = []

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.docx') and not filename.startswith('._'):
        file_path = os.path.join(folder_path, filename)
        file_content = read_docx(file_path)
        SC2021.append(file_content)

new_data = pd.DataFrame({
    'Full Text': SC2021,
    'Link to full text': urls,
    'year': 2021
})

datasource.to_excel('SC_Leginfo.xlsx')
datasource.to_csv('SC_Leginfo.csv')
print("Web-scrapting finished")
# End of SC_Leginfo.py


# Start of MO_Leginfo.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import numpy as np

#driver_path = 'C:/Users/longy/OneDrive/Projects/AFRI/windows/chromedriver.exe' #for windows
driver_path = '/Users/long/Library/CloudStorage/OneDrive-Personal/Projects/AFRI/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
os.chdir('/Users/long/OneDrive/Projects/AFRI')
#os.chdir('C:/Users/longy/OneDrive/Projects/AFRI')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
dnldpath = {"download.default_directory" : "/Users/long/OneDrive/Projects/AFRI/data/ND/"}
chromeOptions.add_experimental_option('prefs', {
    "download.default_directory": dnldpath, #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True, #It will not show PDF directly in chrome
    "--enable-javascript":True
})
driver = webdriver.Chrome(executable_path=driver_path, options=chromeOptions)

print("Current working directory: {0}".format(cwd))


def unique(elements):
    x = np.array(elements)
    print(np.unique(x))

urls=[]
sessionurls = []
billurls = []
elements = []
driver.get("https://house.mo.gov/LegislationSP.aspx?year=2022&code=R")
WebDriverWait(driver, 300).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "div.view-content ul li a"))
)
driver.find_element_by_css_selector('#Bills\ Truly\ Agreed\ and\ Finally\ Passed\ by\ Effective\ Date').click()

#reportgrid > tbody > tr > td
#reportgrid > tbody > tr > td

bills = driver.find_elements_by_css_selector('#reportgrid > tbody > tr:nth-child(2)')

bills = driver.find_elements_by_xpath('//*[@id="reportgrid"]/tbody/tr[2]')
bills = driver.find_elements_by



bills = driver.find_elements_by_xpath('/html/body/form/div[3]/div[1]/table/tbody/tr/th/table/tbody/tr')

bills = driver.find_elements_by_css_selector('tbody > tr:nth-child(2) > td:nth-child(1) > a')

bills = driver.find_elements_by_css_selector('table#reportgrid')

//*[@id="reportgrid"]/tbody/tr[2]

bills = driver.find_elements_by_css_selector('tr.reportbillinfo')

bills = driver.find_element_by_css_selector('#reportgrid > tbody > tr:nth-child(12) > td > a')

#reportgrid > tbody > tr:nth-child(7) > td:nth-child(1)

bills = driver.find_element_by_css_selector('#reportgrid > tbody > tr > td:nth-child(1) > a')


digits = ["0","1","2","3","4","5","6","7","8","9"]
for digit in digits:
    element = driver.find_elements_by_partial_link_text(digit)
    elements.extend(element)
    laws = list(set(elements))


c = driver.find_element_by_css_selector('#GridView1')

a = driver.find_elements_by_css_selector('tbody tr td a')

b = driver.find_elements_by_css_selector('tbody > tr > td > a')

a = driver.find_elements_by_css_selector('tbody > tr > td:nth-child(1) > a')

#reportgrid > tbody > tr:nth-child(2) > td:nth-child(1) > a

#reportgrid > tbody > tr:nth-child(2) > td:nth-child(1) > a

laws.click()

sessions = driver.find_elements_by_css_selector('div.view-content ul li a')
for index,session in enumerate(sessions):
    print(index)
    print(session.text)
    sessionurl = session.get_attribute('href')
    sessionurls.append(sessionurl)

for sessionurl in sessionurls[:33]:
    driver.get(sessionurl)
    sleep(1)
    driver.find_element_by_link_text("Chapter Categories").click()
    try:
        chapters = driver.find_elements_by_css_selector('div.field-items ul li a')
        for chapter in chapters:
            url = chapter.get_attribute('href')
            urls.append(url)
    except:
        print(sessionurl)
print("urls are done")

acttxts = []
for idx,url in enumerate(urls):
    driver.get(url)
    sleep(1)
    path = "C:/Users/longy/OneDrive/Projects/AFRI/data/ND/"
    list_of_files = glob.glob(os.path.join(path, '*.pdf'))
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    with open(os.path.join(os.getcwd(), latest_file), 'r') as f:  # open in readonly mode
        # creating a pdf File object of original pdf
        pdfFileObj = open(latest_file, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        pagenumber = pdfReader.numPages
        acttxt = []
        pgtxts = []
        for p in range(pagenumber):
            pageObj = pdfReader.getPage(p)
            pgtxt = pageObj.extractText()
            pgtxts.append(pgtxt)
            acttxt = '\n'.join(pgtxts)
    acttxts.append(acttxt)
path = "C:/Users/longy/OneDrive/Projects/AFRI/data/ND/"
list_of_files = glob.glob(os.path.join(path, '*.pdf'))
#latest_file = max(list_of_files, key=os.path.getctime)
acttxts = []
for idx,file in enumerate(list_of_files):
    with open(os.path.join(os.getcwd(), file), 'r') as f:  # open in readonly mode
        # creating a pdf File object of original pdf
        pdfFileObj = open(file, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        pagenumber = pdfReader.numPages
        acttxt = []
        pgtxts = []
        for p in range(pagenumber):
            pageObj = pdfReader.getPage(p)
            pgtxt = pageObj.extractText()
            pgtxts.append(pgtxt)
            acttxt = '  '.join(pgtxts)
    acttxts.append(acttxt)
datasource = pd.DataFrame({
    'Link to full text':urls,
    'Full text': acttxts
})

datasource = pd.DataFrame({
    'Full text': acttxts
})
datasource.to_excel('ND_Leginfo.xlsx')
datasource.to_csv('ND_Leginfo.csv')
datasource.to_pickle('ND_Leginfo.pkl')
datasource.to_json('ND_Leginfo.json')


with open('ND_Leginfo.pkl', 'rb') as k:
    data = pickle.load(k)

# End of MO_Leginfo.py


# Start of CA_leginfo_v2.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import pytest
import unittest
from random import randint
import PyPDF2
import glob


# driver_path = 'C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe' #for windows
# driver_path = '/Users/Yanxu/OneDrive/Projects/AgPolicyCoding/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
os.chdir('/Users/long/OneDrive/Projects/AFRI')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
dnldpath = {"download.default_directory" : "/Users/long/OneDrive/Projects/AFRI/data/CA/"}
chromeOptions.add_experimental_option("prefs",dnldpath)
driver = webdriver.Chrome(executable_path='./chromedriver', options=chromeOptions)

print("Current working directory: {0}".format(cwd))


# input keyword(s)
keywords = ['']

# choose sessionyears
#input_sessionyears = ['2021 - 2022']
#input_sessionyears = ['2015 - 2016', '2013 - 2014', '2011 - 2012', '2009 - 2010', '2007 - 2008']
#input_sessionyears = ['2015 - 2016', '2013 - 2014', '2011 - 2012', '2009 - 2010', '2007 - 2008']
input_sessionyears = ['2021 - 2022', '2019 - 2020', '2017 - 2018', '2015 - 2016', '2013 - 2014', '2011 - 2012', '2009 - 2010', '2007 - 2008', '2005 - 2006', '2003 - 2004', '2001 - 2002', '1999 - 2000']


urls = []
# search bills by iterating sessionyears first and then keywords
for input_sessionyear in input_sessionyears:
    driver.get("https://leginfo.legislature.ca.gov/faces/advance/advance.xhtml")

    # select session year
    select_sessionyear = Select(driver.find_element_by_css_selector('select#session_year'))
    select_sessionyear.select_by_visible_text(input_sessionyear)

    # search by keywords
    #input_keywords = driver.find_element_by_css_selector("input#keyword.keyword_text")
    #input_keywords.send_keys(keyword)

    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input#executeSearchBtn"))
    )
    search_button = driver.find_element_by_css_selector('input#executeSearchBtn')
    search_button.click()

    # get urls in the first page
    searchresult = driver.find_elements_by_css_selector('tbody a')
    basicinfo = driver.find_elements_by_css_selector('div.commdataRow')

    for i in range(len(searchresult)):
        #for i in range(10):
        url = searchresult[i].get_attribute('href')
        urls.append(url)

searchresultmiddle = driver.find_elements_by_css_selector('tbody a')
for i in range(len(searchresultmiddle)):
    url = searchresultmiddle[i].get_attribute('href')
urls.append(url)

# get urls in the middle pages
try:
    #driver.find_element_by_link_text("Next 1000 BIlls")
    # centercolumn > div:nth-child(10) > input[type=submit]
    next_page = driver.find_element_by_css_selector('div:nth-child(10) > input[type=submit]')
    next_page.click()
    searchresultmiddle = driver.find_elements_by_css_selector('tbody a')
    for i in range(len(searchresultmiddle)):
        url = searchresultmiddle[i].get_attribute('href')
    urls.append(url)


    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div#searchresult tbody a"))
    )
    searchresult = driver.find_elements_by_css_selector('div#searchresult tbody a')
    basicinfo = driver.find_elements_by_css_selector('div.commdataRow')
    for i in range(len(searchresult)):
        url = searchresult[i].get_attribute('href')
        urls.append(url)
    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div#text_bill_returned"))
    )
    text_bill_returned = driver.find_element_by_css_selector('div#text_bill_returned').text
    page_index = int(text_bill_returned.rsplit('Page ')[1].rsplit(' of')[0])
    sleep(3)

#print("done with urls")


# set up list titles
years = []
states = []
sessionyears = []
billnumbers = []
titles = []
briefsummarys = []
introduceddates = []
signingdates = []
effectivedates = []
expireddates = []
leadauthors = []
textlinks = []


# extract bill information on each bill

for url in urls:
    driver.get(url)
    version = driver.find_element_by_css_selector("select#version.bill_version_select")

    if "Chaptered" in version.text:
        introduceddate = version.text.split("- Introduced")[0]
        introduceddate = introduceddate.split(" ")[-2]
        introduceddates.append(introduceddate)

        state = "CA"
        states.append(state)

        billnumber = driver.find_element_by_css_selector('div#bill_title')
        billnumbers.append(billnumber.text.rsplit(" ")[0])

        title = driver.find_element_by_css_selector('div#bill_title h2')
        title = title.text.split(" ",1)[1]
        titles.append((title.rsplit("(",1)[0]))

        try:
            briefsummaryresolution = driver.find_element_by_css_selector('span.Resolution').text
        except:
            briefsummaryresolution = "N/A"

        briefsummarydigest = driver.find_element_by_css_selector('span#digesttext').text

        if not briefsummarydigest:
            briefsummary = briefsummaryresolution
        else:
            briefsummary = briefsummarydigest

        briefsummarys.append(briefsummary)

        if not version.text.find("Chaptered"):
            signingdate = 'N/A'
        else:
            signingdate = version.text.split("- Chaptered")[0].split(" ")[-2]

        signingdates.append(signingdate)

        signingdate_str = str(signingdate)
        signingdate_length = len(signingdate_str)
        shortyear = int(str(signingdate)[signingdate_length-2: signingdate_length])
        if shortyear > 22:
            f"{shortyear:02}"
        else:
            if shortyear < 10:
                year = "200" + str(shortyear)
            else:
                year = "20" + str(shortyear)
        years.append(year)

        if (int(year) % 2) == 0:
            sessionyear = str(int(year)-1)+"-"+str(year)
        else:
            sessionyear = str(year)+str("-")+str(int(year)+1)
        sessionyears.append(sessionyear)

        if "take effect immediately" in briefsummary:
            effectivedate = signingdate
        else:
            if "take effect on or before" in briefsummary:
                effectivedate = briefsummary.split("take effect on or before")[1].split(".")[0]
            elif "take effect on" in briefsummary:
                effectivedate = briefsummary.split("take effect on")[1].split(".")[0]
            else:
                effectivedate = "Need to check"

        effectivedates.append(effectivedate)

        if datefinder.find_dates(briefsummary):
            dates = datefinder.find_dates(briefsummary)
            datelist = []
            try:
                for date in dates:
                    datelist.append(date.date())
                exdtkeys = ['expired', 'repeal', 'extend', 'repealed', 'repeals']
                if any(key in briefsummary for key in exdtkeys):
                    expireddate = max(datelist)
                else:
                    expireddate = 'N/A'
            except:
                expireddate = 'Need to check'
        else:
            expireddate = 'N/A'

        expireddates.append(expireddate)

        driver.find_element_by_css_selector("#nav_bar_top_status > span").click()
        leadauthor = driver.find_element_by_css_selector('#leadAuthors').text
        #try:
        #    leadauthor = driver.find_element_by_css_selector('#leadAuthors').text.replace(" (A)","")
        #except:
        #    leadauthor = driver.find_element_by_css_selector('#leadAuthors').text.replace(" (S)","")
        leadauthors.append(leadauthor)

        textlinks.append(url)
        sleep(5)

datasource = pd.DataFrame({
    'Year': years,
    'State': states,
    'Session Year': sessionyears,
    'Bill Number': billnumbers,
    'Name': titles,
    'Brief Summary': briefsummarys,
    'Introduced Date': introduceddates,
    'Date it was signed':signingdates,
    'Date effective':effectivedates,
    'Expiration date':expireddates,
    'Introducer':leadauthors,
    'Link to full text':textlinks
})

# drop duplicates by "Official ID"
datasource.drop_duplicates(subset = ['Bill Number', 'Year'],
                           keep = 'first', inplace = True, ignore_index= True)

# save bill info into excel file
datasource.to_excel('CA_Leginfo.xlsx')
print("download finished")
# End of CA_leginfo_v2.py


# Start of CO_Leginfo.py

# import libraries
## selenium
import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
## others
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
import fitz
import re

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/CO'
print(os.path.abspath(working_directory))


# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Initialize the Firefox driver with the specified options
executable_path = '../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)

# empty list to store info
act_pdf_urls = []
bill_nums = []
chapter_nums = []
years = []
sessions = []
broken_act_urls = []
act_txts = []
act_names = []
names = []
file_names = []
effective_dates = []


# Colorado Legislature Website: https://leg.colorado.gov/session-laws?field_sessions_target_id=75371&sort_bef_combine=field_page_value%20ASC

driver.get('https://leg.colorado.gov/session-laws?field_sessions_target_id=92641&sort_bef_combine=field_page_value%20ASC')

WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '#edit-field-sessions-target-id'))
)

select_session = Select(driver.find_element(By.CSS_SELECTOR,'#edit-field-sessions-target-id'))

all_sessions = driver.find_elements(By.CSS_SELECTOR,'#edit-field-sessions-target-id  > option')

session_pages = []

session_elements = []

for session in all_sessions:
    if int(session.text[:4]) < 2022:
        if session.text != '2017 Extraordinary Session':
            print(session.text)
            select_session.select_by_visible_text(session.text)
            time.sleep(5)
            try:
                driver.find_element(By.CSS_SELECTOR, '.pager-last > a:nth-child(1)').click()
                time.sleep(5)
                session_pages.append(int(driver.find_element(By.CSS_SELECTOR, '.pager-current').text))
                session_elements.append(session)
            except:
                session_pages.append(1)
                session_elements.append(session)

for index,session in enumerate(session_elements):
    if int(session.text[:4]) < 2022:
        if session.text != '2017 Extraordinary Session':
            print(session.text)
            select_session.select_by_visible_text(session.text)
            time.sleep(5)

            WebDriverWait(driver, 300).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".views-table > tbody:nth-child(2) > tr "))
            )

            rows = driver.find_elements(By.CSS_SELECTOR, '.views-table > tbody:nth-child(2) > tr')
            for row in rows:

                years.append(int(session.text[:4]))

                bill_num = row.find_element(By.CSS_SELECTOR, ':nth-child(1)').text
                bill_nums.append(bill_num)

                name = row.find_element(By.CSS_SELECTOR, ':nth-child(2)').text
                names.append(name)

                effective_date = row.find_element(By.CSS_SELECTOR, ':nth-child(3)').text
                effective_dates.append(effective_date)

                chapter_num = row.find_element(By.CSS_SELECTOR, ':nth-child(5)').text
                chapter_nums.append(chapter_num)

                act_pdf_url = row.find_element(By.CSS_SELECTOR, ':nth-child(6) > a:nth-child(1)').get_attribute('href')
                act_pdf_urls.append(act_pdf_url)



            if session.text != '2020 Extraordinary Session':
                for i in range(1, session_pages[index]):

                    driver.find_element(By.CSS_SELECTOR, '.pager-next > a:nth-child(1)').click()
                    time.sleep(10)
                    rows = driver.find_elements(By.CSS_SELECTOR, '.views-table > tbody:nth-child(2) > tr')
                    for row in rows:

                        years.append(int(session.text[:4]))

                        bill_num = row.find_element(By.CSS_SELECTOR, ':nth-child(1)').text
                        bill_nums.append(bill_num)

                        name = row.find_element(By.CSS_SELECTOR, ':nth-child(2)').text
                        names.append(name)

                        effective_date = row.find_element(By.CSS_SELECTOR, ':nth-child(3)').text
                        effective_dates.append(effective_date)

                        chapter_num = row.find_element(By.CSS_SELECTOR, ':nth-child(5)').text
                        chapter_nums.append(chapter_num)

                        act_pdf_url = row.find_element(By.CSS_SELECTOR, ':nth-child(6) > a:nth-child(1)').get_attribute('href')
                        act_pdf_urls.append(act_pdf_url)





for year in range(1993, 2016):
    print(year)
    session_url = 'https://leg.colorado.gov/agencies/office-legislative-legal-services/session-laws-' + str(year)
    driver.get(session_url)

    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div > table > tbody > tr"))
    )

    rows = driver.find_elements(By.CSS_SELECTOR, 'div > table > tbody > tr')
    for row in rows[1:]:

        bill_num = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text
        print(bill_num)
        if "Extraordinary" in bill_num:
            continue
        bill_nums.append(bill_num)

        years.append(year)

        name = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2)').text
        names.append(name)

        effective_date = row.find_element(By.CSS_SELECTOR, 'td:nth-child(3)').text
        effective_dates.append(effective_date)

        chapter_num = row.find_element(By.CSS_SELECTOR, 'td:nth-child(5)').text
        chapter_nums.append(chapter_num)

        try:
            act_pdf_url = row.find_element(By.CSS_SELECTOR, 'td:nth-child(6) > a:nth-child(1)').get_attribute('href')
            act_pdf_urls.append(act_pdf_url)
        except:
            act_pdf_urls.append('NA')


datasource = pd.DataFrame({
    'bill_num': bill_nums,
    'chapter_num': chapter_nums,
    'link': act_pdf_urls,
    'state': "CO",
    'name': names,
    'effective_date':effective_dates,
})

#for these 'HB11-1130''SB11-024''HB11-1169''SB11-020' bills, I added links by hand.
datasource['link'][7136] = 'https://leg.colorado.gov/sites/default/files/images/olls/2011a_sl_106.pdf'
datasource['link'][7109] = 'https://leg.colorado.gov/sites/default/files/images/olls/2011a_sl_79.pdf'
datasource['link'][7149] = 'https://leg.colorado.gov/sites/default/files/images/olls/2011a_sl_119.pdf'
datasource['link'][7069] = 'https://leg.colorado.gov/sites/default/files/images/olls/2011a_sl_39.pdf'

#delete one bill since there is no pdf file.https://leg.colorado.gov/agencies/office-legislative-legal-services/session-laws-1994
# SB94-157 	Implementation Of Federal Mandates 	6/1/1994 	1829 	305
datasource = datasource[datasource['link'] != 'NA']

def convert_to_four_digit_year(two_digit_year):
    cutoff = 50  # Define a cutoff for deciding the century
    if two_digit_year < cutoff:
        return 2000 + two_digit_year
    else:
        return 1900 + two_digit_year

datasource.loc[:, 'year'] = datasource['bill_num'].apply(lambda x: convert_to_four_digit_year(int(x[2:4])))


# download corresponding pdf file
file_names = []
for index,row in datasource.iterrows():
    if int(row['year']) > 2015:
        act_url = row['link']
        print(act_url)
        file_name = act_url.split('sl/')[1]
        file_names.append(file_name )
        print(file_name)

        path = download_dir + file_name
        print(os.path.abspath(path))

        time.sleep(randint(1, 100) * 0.005)

        response = requests.get(act_url)
        # Ensure the request was successful
        if response.status_code == 200:
            # Write the content of the response to a file
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to fetch PDF. Status code:", response.status_code)
            broken_act_urls.append(act_url)
    else:
        act_url = row['link']
        print(act_url)
        file_name = act_url.split('olls/')[1]
        file_names.append(file_name )
        print(file_name)

        path = download_dir + file_name
        print(os.path.abspath(path))

        time.sleep(randint(1, 100) * 0.005)

        response = requests.get(act_url)
        # Ensure the request was successful
        if response.status_code == 200:
            # Write the content of the response to a file
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to fetch PDF. Status code:", response.status_code)
            broken_act_urls.append(act_url)

datasource['file_name'] = file_names

datasource = datasource.drop_duplicates()

#read pdfs
folder_path = "./PDFs"
print(os.path.abspath(folder_path))

files = glob.glob(os.path.join(folder_path, '*.pdf'))

broken_files = []

for file in files:
    try:
        doc = fitz.open(file)
        pgtxts = []
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)
            acttxt = ' '.join(pgtxts)
        act_txts.append(acttxt)
        doc.close()
    except:
        act_txts.append('NA')
        broken_files.append(file)

file_names = []

for file in files:
    file_names.append(file.split("./PDFs/")[1])

datasource_pdf = pd.DataFrame({
    'original_text': act_txts,
    'file_name': file_names
})


datasource = pd.merge(datasource,datasource_pdf,on='file_name', how='left')

datasource.to_excel('CO_Leginfo.xlsx')
datasource.to_csv('CO_Leginfo.csv')
datasource.to_pickle('CO_Leginfo.pkl')
datasource.to_json('CO_Leginfo.json')

# End of CO_Leginfo.py


# Start of TN_Leginfo.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
#import PyPDF2
#import glob

driver_path = 'C:/Users/longy/OneDrive/Projects/AFRI/windows/chromedriver.exe' #for windows
# driver_path = '/Users/Yanxu/OneDrive/Projects/AgPolicyCoding/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
#os.chdir('/Users/long/OneDrive/Projects/AFRI')
os.chdir('C:/Users/longy/OneDrive/Projects/AFRI')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
# for mac
#dnldpath = {"download.default_directory" : "/Users/long/OneDrive/Projects/AFRI/data/SD/"}
# for windows
dnldpath = {"download.default_directory" :"C:/Users/longy/OneDrive/Projects/AFRI/data/LA/"}
chromeOptions.add_experimental_option("prefs",dnldpath)
driver = webdriver.Chrome(executable_path=driver_path, options=chromeOptions)

print("Current working directory: {0}".format(cwd))


driver.get('https://sos.tn.gov/publications/services/acts-and-resolutions')

# End of TN_Leginfo.py


# Start of WI_Leginfo.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import time

#driver_path = 'C:/Users/longy/OneDrive/Projects/AFRI/windows/chromedriver.exe' #for windows
driver_path = '/Users/long/Library/CloudStorage/OneDrive-Personal/Projects/AFRI/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
os.chdir('/Users/long/Library/CloudStorage/OneDrive-Personal/Projects/AFRI')
#os.chdir('C:/Users/longy/OneDrive/Projects/AFRI')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
# for mac
#dnldpath = {"download.default_directory" : "/Users/long/OneDrive/Projects/AFRI/data/SD/"}
# for windows
chromeOptions.add_experimental_option('prefs', {
    "download.default_directory": "/Users/long/Library/CloudStorage/OneDrive-Personal/Projects/AFRI/data/WI/", #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True, #It will not show PDF directly in chrome
    "--enable-javascript":True})

driver = webdriver.Chrome(executable_path=driver_path, options=chromeOptions)
#river = webdriver.Firefox()

sessions_first = ['1975','1977','1979','1981','1983','1985','1987','1989','1991','1993']
sessions_first_act_urls = []
acttxts = []
sts = []

for session in sessions_first:
    session_url = "https://docs.legis.wisconsin.gov/" + str(session) + "/related/acts"
    print(session_url)
    driver.get(session_url)
    acts = driver.find_elements_by_partial_link_text("Wisconsin")
    for act in acts:
        act_url = act.get_attribute('href')
        act_urls.append(act_url)

for act in sessions_first_act_urls:
    driver.get(act)
    sleep(randint(1, 2))

path = "/Users/long/OneDrive/Projects/AFRI/data/WI/"
list_of_files = glob.glob(os.path.join(path, '*.pdf'))

for file in list_of_files:
    with open(os.path.join(os.getcwd(), file), 'r') as f:  # open in readonly mode
        # creating a pdf File object of original pdf
        pdfFileObj = open(latest_file, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        pagenumber =  pdfReader.numPages
        acttxt =[]
        pgtxts = []
        for p in range(pagenumber):
            pageObj = pdfReader.getPage(p)
            pgtxt = pageObj.extractText()
            pgtxts.append(pgtxt)
            acttxt = '\n'.join(pgtxts)
    acttxts.append(acttxt)
    st ="WI"
    sts.append(st)
    sleep(1)

act2_urls = []

sessions_second = ['1995','1997','1999','2001','2003','2005','2007','2009','2011','2013','2015','2017','2019','2021']

for session in sessions_second:
    session_url = "https://docs.legis.wisconsin.gov/" + str(session) + "/related/acts"
    print(session_url)
    driver.get(session_url)
    sleep(randint(1,2))
    acts = driver.find_elements_by_partial_link_text("Wisconsin Act")
    for act in acts:
        act2_url = act.get_attribute('href')
        act2_urls.append(act2_url)
        print(act2_url)
        t = 0.1 * randint(1, 9)
        time.sleep(t)

for index,url in enumerate(act2_urls):
    print(index)
    driver.get(url)
    t = 0.3* randint(1, 9)
    time.sleep(t)
    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div#document.acts"))
    )
    acttxt = driver.find_element_by_css_selector('div#document.acts').text
    acttxts.append(acttxt)
    st = "WI"
    sts.append(st)
    t = 0.1* randint(1, 9)
    time.sleep(t)

datasource = pd.DataFrame({
    #'Link to full text':session_urls,
    'Full text': acttxts
})

# drop duplicates by "Official ID"
datasource.drop_duplicates(subset = ['Full text'],
                           keep = 'first', inplace = True, ignore_index= True)


datasource.to_excel('WI_Leginfo.xlsx')
datasource.to_csv('WI_Leginfo.csv')
datasource.to_pickle('WI_Leginfo.pkl')
datasource.to_json('WI_Leginfo.json')
# End of WI_Leginfo.py


# Start of MI_Leginfo.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
from datetime import datetime
from dateutil.parser import parse

# driver_path = 'C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe' #for windows
# driver = webdriver.Chrome(executable_path=driver_path)

# set up webdriver by short path
driver = webdriver.Chrome('./chromedriver')


# choose sessionyears
input_sessionyears = ['2021-2022', '2019-2020', '2017-2018', '2015-2016', '2013-2014', '2011-2012', '2009-2010',
                      '2007-2008', '2005-2006', '2003-2004', '2001-2002', '1999-2000', '1997-1998']


urls = []

for input_sessionyear in input_sessionyears:
    driver.get("https://www.legislature.mi.gov/(S(uycj0anii1cfudf2mj0uyllp))/mileg.aspx?page=LegAdvancedSearch")

    select_public = driver.find_element_by_css_selector('#frg_legadvancedsearch_PublicActs')
    select_public.click()

    # select session year
    select_sessionyear = Select(
        driver.find_element_by_css_selector('#frg_legadvancedsearch_LegislativeSession_LegislativeSession'))
    select_sessionyear.select_by_visible_text(input_sessionyear)

    # search by keywords
    #input_keywords = driver.find_element_by_css_selector('#frg_legadvancedsearch_LegFullText')
    #input_keywords.send_keys(keyword)

    # click search button
    search_button = driver.find_element_by_css_selector('#frg_legadvancedsearch_SearchButton')
    search_button.click()

    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "#maincontent > table:nth-child(2) > tbody > tr:nth-child(1) > td > span"))
    )

    try:
        searchresult = driver.find_elements_by_css_selector(
            'table#frg_executesearch_SearchResults_Results  tbody  tr td a')
    except:
        continue
    sleep(3)

    for i in range(len(searchresult)):
        url = searchresult[i].get_attribute('href')
        urls.append(url)

print("done with urls")

# set up list titles
years = []
states = []
sessionyears = []
billnumbers = []
names = []
briefsummarys = []
introduceddates = []
signingdates = []
effectivedates = []
expirationdates = []
leadauthors = []
textlinks = []
acttxts = []

for url in urls:

    driver.get(url)

    textlink = driver.find_element_by_css_selector("#frg_print_HtmlDocIcon_DocumentImage").get_attribute('href')
    textlinks.append(textlink)

    state = "MI"
    states.append(state)

    year = driver.find_element_by_css_selector('#frg_print_DocumentName').text.split(" of ")[1]
    years.append(year)

    if (int(year) % 2) == 0:
        sessionyear = str(int(year) - 1) + "-" + str(year)
    else:
        sessionyear = str(year) + str("-") + str(int(year) + 1)
    sessionyears.append(sessionyear)

    briefsummary = driver.find_element_by_css_selector('#frg_print_DocumentDescription').text
    briefsummarys.append(briefsummary)

    text = driver.find_element_by_css_selector('#frg_print_HtmlDocIcon_DocumentImage > img')
    text.click()

    #try:
    #    billnumber = driver.find_element_by_xpath("//font[contains(text(),'SENATE BILL No.')]").text.split("No. ")[1]
    #    billnumber = "SB-" + str(billnumber)
    #except:
    #    billnumber = driver.find_element_by_xpath("//font[contains(text(),'HOUSE BILL No.')]").text.split("No. ")[1]
    #    billnumber = "HB-" + str(billnumber)
    #billnumbers.append(billnumber)

    #name = driver.find_element_by_xpath("//font[contains(text(),'AN ACT to')]").text
    #names.append(name)

    #leadauthor = driver.find_element_by_xpath("//font[contains(text(),'Introduced by')]").text.split("Introduced by ")[1]
    #leadauthors.append(leadauthor)

    #signingdate = driver.find_element_by_css_selector('body').text.split('Approved by the Governor')[1].split('Filed with the Secretary of State')[0]
    #dates = datefinder.find_dates(signingdate)
    #datelist = []
    #for date in dates:
    #    signingdate = date.date()
    #signingdates.append(signingdate)

    acttxt = driver.find_element_by_css_selector("body").text
    acttxts.append(acttxt)
    sleep(randint(1,2))
datasource = pd.DataFrame({
    'Year': years,
    'State': states,
    'Session Year': sessionyears,
    'Brief Summary': briefsummarys,
    'Link to full text':textlinks,
    'Full text': acttxts
})



# save bill info into excel file
datasource.to_excel('MI_Leginfo.xlsx')
datasource.to_csv('MI_Leginfo.csv')
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
datasource.to_sql('MI_Leginfo.sql',con=engine)
datasource.to_pickle('MI_Leginfo.pkl')
datasource.to_json('MI_Leginfo.json')


datasource.to_stata('MI_Leginfo.dta')

print("download finished")
# End of MI_Leginfo.py


# Start of TX_Leginfo.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import fitz
import time
# driver_path = r'C:\Users\longy\OneDrive\Projects\AFRI\windows\chromedriver.exe' #for windows
driver_path = '/Users/long/OneDrive/Projects/AFRI/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
os.chdir('/Users/long/OneDrive/Projects/AFRI')
# os.chdir('C:/Users/longy/OneDrive/Projects/AFRI/')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_experimental_option('prefs', {
    "download.default_directory": "C:/Users/longy/OneDrive/Projects/AFRI/data/TX/", #Change default directory for downloads
    "download.default_directory": "/Users/long/OneDrive/Projects/AFRI/data/TX/", #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True, #It will not show PDF directly in chrome
    "--enable-javascript":True})

driver = webdriver.Chrome(executable_path=driver_path, options=chromeOptions)

#64-86th session 2021
urls = []

for session in range(64,87):
    for sub in range(0,4):
        url = "https://lrl.texas.gov/legis/billsearch/searchchapter.cfm?legSession=" + str(session) +"-" + str(sub) + "&chapter=&submitbutton=Search+by+chapter"
        driver.get(url)
        sleep(1)
        act_urls = driver.find_elements_by_css_selector('td.results a')
        for i in act_urls:
            act_url = i.get_attribute('href')
            if 'pdf' in act_url:
                print(act_url)
                urls.append(act_url)


for act in urls:
    driver.get(act)
    t = 0.1* randint(1, 9)
    time.sleep(t)



path = "/Users/long/Downloads/TX2"
files = glob.glob(os.path.join(path, '*.pdf'))

acttxts = []
pgtxts = []
for file in files:
    with fitz.open(file) as doc:
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)
            acttxt = '  '.join(pgtxts)
        t = 0.03 * randint(1, 9)
        time.sleep(t)
    acttxts.append(acttxt)


datasource = pd.DataFrame({
    #'Link to full text':session_urls,
    'Full text': acttxts
})


datasource.to_string("TX_Leginfo")
datasource.to_excel('TX_Leginfo.xlsx')
datasource.to_csv('TX_Leginfo.csv')
datasource.to_pickle('TX_Leginfo.pkl')
datasource.to_json('TX_Leginfo.json')


# End of TX_Leginfo.py


# Start of NC_Leginfo.py

# import libraries
## selenium
import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
## others
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
import fitz
import re

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/NC'
print(os.path.abspath(working_directory))


# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Initialize the Firefox driver with the specified options
executable_path = '../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)

# empty list to store info

act_pdf_urls = []
act_html_urls = []
pdf_file_names = []
html_file_names = []
bill_nums = []
chapter_nums = []
years = []
sessions = []
broken_act_urls = []
act_txts = []
act_names = []
names = []
file_names = []
summaries = []


# North Caroline Legislature Website: https://www.ncleg.gov/Laws/SessionLaws/

driver.get('https://www.ncleg.gov/Laws/SessionLaws/')

WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'body'))
)

driver.find_element(By.CSS_SELECTOR,'#sessionDropdown_chosen > a:nth-child(1) > span:nth-child(1)').click()

session_elements = driver.find_elements(By.CSS_SELECTOR,'li.active-result')
session_list = []
for session in session_elements:
    session_list.append(session.text)
    print(session.text)

session_urls = []

for text in session_list:
    if int(text.split('-')[0]) <= 2021 and int(text.split('-')[0]) >= 1975:
        if 'Extra' in text:
            session_urls.append('https://www.ncleg.gov/Laws/SessionLaws/' + text.split('-')[0] + 'e' + text.split('Session ')[1])
        else:
            session_urls.append('https://www.ncleg.gov/Laws/SessionLaws/' + text.split('-')[0])

for url in session_urls:

    driver.get(url)

    WebDriverWait(driver, 60).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'body'))
    )

    rows = driver.find_elements(By.CSS_SELECTOR,'div.py-2')

    for row in rows:

        act_html_url = row.find_element(By.CSS_SELECTOR,
                                        'div:nth-child(1) > a:nth-child(1)').get_attribute('href')
        act_html_urls.append(act_html_url)

        act_pdf_url = row.find_element(By.CSS_SELECTOR,
                                       'div:nth-child(1) > a:nth-child(2)').get_attribute('href')
        act_pdf_urls.append(act_html_url)

        chapter_num = row.find_element(By.CSS_SELECTOR,
                                       'div:nth-child(1) > a:nth-child(3)').text
        chapter_nums.append(chapter_num)

        bill_num = row.find_element(By.CSS_SELECTOR,
                                    'div:nth-child(2)').text
        bill_nums.append(bill_num)

        summary = row.find_element(By.CSS_SELECTOR,
                                   'div:nth-child(3)').text
        summaries.append(summary)

datasource= pd.DataFrame({
    'bill_num': bill_nums,
    'chapter_num': chapter_nums,
    'act_pdf_url': act_pdf_urls,
    'act_html_url': act_html_urls,
    'state': "NC",
    'summary':summaries
})


# download corresponding pdf file

for act_url in act_html_urls:
    print(act_url)

    # Open the desired webpage
    driver.get(act_url)

    act_txts.append(driver.find_element(By.CSS_SELECTOR,'body').text)

    # Get the HTML content of the page
    html_content = driver.page_source

    name = act_url.split('SL')[1]
    html_file_names.append(name)
    print(name)

    path = download_dir + name
    print(os.path.abspath(path))

    # Write the HTML content to a file
    with open(path, 'w', encoding='utf-8') as file:
        file.write(html_content)

datasource['original_text'] = act_txts
datasource = datasource.dropna(subset=['original_text'])
# delete duplicates
datasource = datasource.drop_duplicates(subset=['original_text'])

#datasource.to_excel('NC_Leginfo.xlsx') IllegalCharacterError
datasource.to_csv('NC_Leginfo.csv')
datasource.to_pickle('NC_Leginfo.pkl')
datasource.to_json('NC_Leginfo.json')

# End of NC_Leginfo.py


# Start of WA_Leginfo.py

# import libraries
## selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.firefox.service import Service
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import fitz
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import requests

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/WA'
print(os.path.abspath(working_directory))

# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "../PDFs"
print(os.path.abspath(download_dir))

# Create a Firefox profile and configure preferences
options = Options()


# Initialize the Firefox driver with the specified options
executable_path='../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)


# The page you're looking for doesn't exist.
''' 
['https://leg.wa.gov/CodeReviser/documents/sessionlaw/1991c54.pdf',
 'https://leg.wa.gov/CodeReviser/documents/sessionlaw/1991ex1c1.pdf',
 'http://lawfilesext.leg.wa.gov/biennium/1991-92/Pdf/Bills/Session%20Laws/House/120.sl.pdf',
 'http://lawfilesext.leg.wa.gov/biennium/1991-92/Pdf/Bills/Session%20Laws/Senate/6347-S2.sl.pdf',
 'http://lawfilesext.leg.wa.gov/biennium/1995-96/Pdf/Bills/Session%20Laws/House/164.sl.pdf']
'''


driver.get('https://leg.wa.gov/CodeReviser/pages/session_laws.aspx')

# Create a list from range
years_list = list(range(1975, 2022))  # Note: 2022 is exclusive

# Remove the year 1978
years_list.remove(1978)

years = []
links = []

badfiles = []

for year in years_list:
    driver.get('https://leg.wa.gov/CodeReviser/pages/session_laws.aspx')
    print(year)
    driver.find_element(By.PARTIAL_LINK_TEXT,str(year)).click()
    chapters = driver.find_elements(By.PARTIAL_LINK_TEXT, "Ch.")
    for chapter in chapters:
        try:
            years.append(year)
            link = chapter.get_attribute("href")
            links.append(link)
            time.sleep(0.01*randint(1,2))
        except:
            badfiles.append(chapter)


df = pd.DataFrame({
    'year':years,
    'link': links
})

# Dropping duplicates based on 'links' column
df = df.drop_duplicates(subset='links')



# store some broken pages
bad_act_urls = []

file_names = []
# download pdfs from the above links scraped
for index, link in enumerate(df['links']):
    try:
        name = str(link).split("sessionlaw/")[1].split(".pdf")[0]
        path = "/Volumes/SSD/AFRI/Data/Raw_Data/WA/PDFs/" + name + ".pdf"
        # Send a GET request to the URL
        response = requests.get(link)
        # Ensure the request was successful
        if response.status_code == 200:
            # Write the content of the response to a file
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to fetch PDF. Status code:", response.status_code)
            bad_act_urls.append(link)
    except:
        # Find the position of the last '/'
        last_slash_pos = link.rfind('/')

        # Split the URL into two parts
        base_url = link[:last_slash_pos]
        name = str(link.split("biennium/")[1].split("/Pdf")[0]) + link[last_slash_pos + 1:].split(".pdf")[0]
        path = "/Volumes/SSD/AFRI/Data/Raw_Data/WA/PDFs/" + name + ".pdf"
        # Send a GET request to the URL
        response = requests.get(link)
        # Ensure the request was successful
        if response.status_code == 200:
            # Write the content of the response to a file
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to fetch PDF. Status code:", response.status_code)
            bad_act_urls.append(link)
    time.sleep(0.01)
    file_names.append(name)

df['file_name'] = file_names


act_txts = []
pgtxts = []
file_names_from_folder = []

path = "./PDFs"
files = glob.glob(os.path.join(path, '*.pdf'))
print(os.path.abspath(path))

for file in files:
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        acttxt = ' '.join(pgtxts)
    act_txts.append(acttxt)
    doc.close()
    file_names_from_folder.append(file.split("PDFs/")[1].replace(".pdf",""))

df_pdfs = pd.DataFrame({
    'original_text':act_txts,
    'file_name': file_names_from_folder
})


datasource = pd.merge(df, df_pdfs, on='file_name', how='inner')

datasource.to_excel('WA_Leginfo.xlsx')
datasource.to_csv('WA_Leginfo.csv')
datasource.to_pickle('WA_Leginfo.pkl')
datasource.to_json('WA_Leginfo.json')
# End of WA_Leginfo.py


# Start of AR_Leginfo.py

# import libraries
## selenium
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import fitz

import time
import requests
from bs4 import BeautifulSoup

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/AR'
print(os.path.abspath(working_directory))

# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Create a Firefox profile and configure preferences
options = Options()


# Initialize the Firefox driver with the specified options
executable_path='../../geckodriver'
print(os.path.abspath(executable_path))

# Initialize the WebDriver with the specified path
service = Service(executable_path)
driver = webdriver.Firefox(service=service)

years = []
chapter_nums = []
subtitles = []
bill_nums = []
filenames = []
act_urls = []
sessions = []
broken_act_urls = []

driver.get('https://www.arkleg.state.ar.us/Acts/SearchByRange?ddBienniumSession')
# accept cookies button
try:
    driver.find_element(By.CSS_SELECTOR, 'a.btn').click()
except:
    pass

session_elements = driver.find_elements(By.CSS_SELECTOR, '#ddBienniumSessionActsSearchByRange option')
session_texts = []
for session_text in session_elements:
    session_texts.append(session_text.text)

for session_text in session_texts[1:]:
    print(session_text)
    try:
        # Attempt to convert the value to an integer
        year = int(session_text.replace(' ','').split('-')[0])
        if year > 2021:
            pass
        else:
            select = Select(driver.find_element(By.CSS_SELECTOR, '#ddBienniumSessionActsSearchByRange'))
            select.select_by_visible_text(session_text)
            # clear starting act number and ending act number
            starting_num = driver.find_element(By.CSS_SELECTOR, '#startAct')
            starting_num.clear()
            starting_num.send_keys(1)
            ending_num = driver.find_element(By.CSS_SELECTOR, '#endAct')
            ending_num.clear()
            ending_num.send_keys(9999)
            # startAct
            search_button = driver.find_element(By.CSS_SELECTOR, 'div.form-row:nth-child(5) > button:nth-child(1)')
            search_button.click()

            time.sleep(3)

            # for page 1
            rows = driver.find_elements(By.CSS_SELECTOR, 'div.row.tableRow, div.row.tableRowAlt')
            for row in rows:
                #years.append(year)
                #sessions.append(session_text)
                chapter_num = row.find_element(By.CSS_SELECTOR, ':nth-child(1)').text
                chapter_nums.append(chapter_num)
                subtitles.append(row.text)
                bill_nums.append(row.find_element(By.CSS_SELECTOR, ':nth-child(3)').text)
                act_url = row.find_element(By.LINK_TEXT, 'PDF').get_attribute('href')
                act_urls.append(act_url)

                name = str(year) + str(session_text.split(',')[0].replace(' ', '').split('-')[1]) + "chap" + str(
                    chapter_num) + ".pdf"
                filenames.append(name)

                path = download_dir + name
                print(os.path.abspath(path))

                time.sleep(randint(1, 100) * 0.005)

                response = requests.get(act_url)
                # Ensure the request was successful
                if response.status_code == 200:
                    # Write the content of the response to a file
                    with open(path, 'wb') as file:
                        file.write(response.content)
                else:
                    print("Failed to fetch PDF. Status code:", response.status_code)
                    broken_act_urls.append(act_url)
            time.sleep(1)
            # for page 2-n
            page_num = int(driver.find_element(By.CSS_SELECTOR,'div.row:nth-child(25) > div:nth-child(1)').text.split(':')[0].split('of ')[1])
            for num in range(2,page_num+1):
                driver.find_element(By.LINK_TEXT,str(num)).click()
                time.sleep(1)
                rows = driver.find_elements(By.CSS_SELECTOR, 'div.row.tableRow, div.row.tableRowAlt')
                for row in rows:
                    years.append(year)
                    sessions.append(session_text)
                    chapter_num = row.find_element(By.CSS_SELECTOR, ':nth-child(1)').text
                    chapter_nums.append(chapter_num)
                    subtitles.append(row.find_element(By.CSS_SELECTOR, ':nth-child(2)').text)
                    bill_nums.append(row.find_element(By.CSS_SELECTOR, ':nth-child(3)').text)
                    act_url = row.find_element(By.LINK_TEXT, 'PDF').get_attribute('href')
                    act_urls.append(act_url)

                    name = str(year) + str(session_text.split(',')[0].replace(' ','').split('-')[1]) + "chap" + str(chapter_num) + ".pdf"
                    filenames.append(name)

                    path = download_dir + name
                    print(os.path.abspath(path))

                    time.sleep(randint(1, 100) * 0.005)

                    response = requests.get(act_url)
                    # Ensure the request was successful
                    if response.status_code == 200:
                        # Write the content of the response to a file
                        with open(path, 'wb') as file:
                            file.write(response.content)
                    else:
                        print("Failed to fetch PDF. Status code:", response.status_code)
                        broken_act_urls.append(act_url)
    except:
        pass



acttxts = []
pgtxts = []
path = "/Users/long/Downloads/AR/"
files = glob.glob(os.path.join(path, '*.pdf'))
for file in files[:20]:
    pdfFileObj = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    numpages = pdfReader.numPages
    for i in range(numpages):
        pgtxt = []
        pgtxt = pdfReader.getPage(i)
        acttxt = ' '.join(pgtxts)
    print(acttxt)
    acttxts.append(acttxt)
    pdfFileObj.close()


act_txts = []
path = "./PDFs"
files = glob.glob(os.path.join(path, '*.pdf'))
for file in files:
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        act_txt = ' '.join(pgtxts)
    act_txts.append(act_txt)
    doc.close()


datasource = pd.DataFrame({
    'original_text': act_txts,
    'year':years,
    'chapter_num':chapter_nums,
    'subtitle':subtitles,
    'bill_num':bill_nums,
    'filename':filenames,
    'act_url':act_urls,
    'session':sessions
})

# save bill info into files
datasource.to_excel('AR_Leginfo.xlsx')
datasource.to_csv('AR_Leginfo.csv')
datasource.to_pickle('AR_Leginfo.pkl')
datasource.to_json('AR_Leginfo.json')
print("done")
# End of AR_Leginfo.py


# Start of OH_Leginfo.py

# import libraries
## selenium
import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
## others
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
import fitz
import re
import datefinder

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/OH'
print(os.path.abspath(working_directory))


# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))
# Initialize the Firefox driver with the specified options
executable_path = '../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service=service)


# empty list to store info

act_urls_1997_2014 = []
act_pdf_urls = []
act_html_urls = []
pdf_file_names = []
html_file_names = []
bill_nums = []
chapter_nums = []
years = []
sessions = []
broken_act_urls = []
act_txts = []
act_names = []
names = []
file_names = []
summaries = []


# Ohio lature Website: http://archives.legislature.state.oh.us/acts.cfm?GenAssem=122 1997-2014
# https://www.legislature.ohio.gov/legislation/acts/131 after 2016

# for 1997-2014

for i in range(122,131):
    print(i)

    driver.get('http://archives.legislature.state.oh.us/acts.cfm?GenAssem=' + str(i))

    WebDriverWait(driver, 60).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'body'))
    )

    bill_elements = driver.find_elements(By.PARTIAL_LINK_TEXT,'Bill')

    for element in bill_elements:
        sessions.append(i)
        bill_nums.append(element.text)
        act_urls_1997_2014.append(element.get_attribute('href'))

act_pdf_urls = []
act_status_urls = []

for url in act_urls_1997_2014:
    driver.get(url)
    WebDriverWait(driver, 60).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'body'))
    )
    #act_txts.append(driver.find_element(By.CSS_SELECTOR,'.bigPanel').text)

    try:
        act_pdf_url = driver.find_element(By.PARTIAL_LINK_TEXT, 'PDF').get_attribute('href')
    except Exception as e:
        try:
            act_pdf_url = driver.find_element(By.PARTIAL_LINK_TEXT, 'pdf').get_attribute('href')
        except Exception as e2:
            try:
                act_pdf_url = driver.find_element(By.PARTIAL_LINK_TEXT, 'As Enrolled').get_attribute('href')
            except Exception as e3:
                act_pdf_url = url

    act_pdf_urls.append(act_pdf_url)

    act_status_url = driver.find_element(By.PARTIAL_LINK_TEXT,'Status ').get_attribute('href')

    act_status_urls.append(act_status_url)

datasource = pd.DataFrame({
    'bill_num': bill_nums,
    'session': sessions,
    'act_html_url': act_urls_1997_2014,
    'state': "OH",
    'act_pdf_url': act_pdf_urls,
    'act_status_url': act_status_urls
})

datasource.to_csv('OH_Leginfo_1997_2014.csv')

datasource = pd.read_csv('OH_Leginfo_1997_2014.csv', index_col=0)

for url in datasource['act_html_url']:
    driver.get(url)
    WebDriverWait(driver, 60).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.bigPanel'))
    )
    act_txts.append(driver.find_element(By.CSS_SELECTOR,'.bigPanel').text)
    time.sleep(randint(1,100)*0.005)

datasource['original_text'] =  act_txts

datasource.to_csv('OH_Leginfo_1997_2014.csv')

datasource = pd.read_csv('OH_Leginfo_1997_2014.csv', index_col=0)


act_urls_2016 = []
bill_nums_2016 = []
names_2016 = []
act_pdf_urls_2016 = []
sessions_2016 = []

for i in range(131,136):

    print(i)

    driver.get('https://www.legislature.ohio.gov/legislation/acts/' + str(i))

    WebDriverWait(driver, 60).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'body'))
    )

    bill_elements = driver.find_elements(By.PARTIAL_LINK_TEXT,'No.')

    for element in bill_elements:
        sessions_2016.append(i)
        bill_nums_2016.append(element.text)
        act_urls_2016.append(element.get_attribute('href'))

        # Find the next sibling of the element
        names_2016.append(element.find_element(By.XPATH, "following-sibling::*[2]").text)

        grandparent_element = element.find_element(By.XPATH, "../..")

        act_pdf_urls_2016.append(grandparent_element.find_element(By.PARTIAL_LINK_TEXT,'Download').get_attribute('href'))

datasource_2016 = pd.DataFrame({
    'session':sessions_2016,
    'bill_num': bill_nums_2016,
    'chapter_num': 'NA',
    'act_pdf_url': act_pdf_urls_2016,
    'act_html_url': act_urls_2016,
    'state': "OH"
})

datasource_2016.to_csv('OH_Leginfo_2016.csv')



datasource_2016 = pd.read_csv('OH_Leginfo_2016.csv', index_col= 0)

file_names_20162 = []
acttxts_20162 = []

for index,row in datasource_2016.iterrows():
    act_url = row['act_html_url']
    driver.get(act_url)
    time.sleep(5)
    try:
        driver.find_element(By.LINK_TEXT,'As Enrolled').click()
    except:
        driver.find_element(By.LINK_TEXT, 'As Adopted by Senate').click()
    time.sleep(5)
    path = "/Volumes/SSD/AFRI/Data/Raw_Data/OH/PDFs_2016new/"
    list_of_files = glob.glob(os.path.join(path, '*.pdf'))
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    # New file name
    new_pdf_file_path = path + act_url.replace('/','').split('legislation')[1] + '.pdf'
    file_names_20162.append(act_url.replace('/','').split('legislation')[1] + '.pdf')
    # Rename the file
    os.rename(latest_file, new_pdf_file_path)

file_names_2016 = []
for index,row in datasource_2016.iterrows():
    act_url = row['act_html_url']
    file_names_2016.append(act_url.replace('/','').split('legislation')[1] + '.pdf')

datasource_2016['file_name'] = file_names_2016


folder_path = "./PDFs_2016new"
print(os.path.abspath(folder_path))

files = glob.glob(os.path.join(folder_path, '*.pdf'))

act_txts_2016 = []

broken_files = []

for file in files:
    try:
        doc = fitz.open(file)
        pgtxts = []
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)
            acttxt = ' '.join(pgtxts)
        act_txts_2016.append(acttxt)
        doc.close()
    except:
        act_txts_2016.append('NA')
        broken_files.append(file)

file_names_2016 = []

for file in files:
    file_names_2016.append(file.split("PDFs_2016new/")[1])

datasource_2016_pdf = pd.DataFrame({
    'original_text':act_txts_2016,
    'file_name': file_names_2016
})

datasource_2016 =  pd.merge(datasource_2016, datasource_2016_pdf, on='file_name', how='left')
datasource_2016.to_csv('OH_Leginfo_2016.csv')
datasource_2016 =  pd.read_csv('OH_Leginfo_2016.csv',index_col=0)

datasource = pd.read_csv('OH_Leginfo_1997_2014.csv', index_col=0)
datasource = datasource._append(datasource_2016, ignore_index=True)

# cleaning
bill_nums = []

for num in datasource['bill_num']:
    print(num)
    if 'Bill' in num:
        if 'Senate' in num:
            bill_nums.append('SB' + num.split(' Bill ')[1])
        else:
            bill_nums.append('HB' + num.split(' Bill ')[1])
    else:
        bill_nums.append(num.replace('No.','').replace('.','').replace(' ',''))
datasource['bill_num'] = bill_nums

sessions = []
for index, row in datasource.iterrows():
    if pd.isna(row['session']):
        session = row['act_html_url'].split('legislation/')[1][:3]
        sessions.append(session)
    else:
        sessions.append(row['session'])

datasource['session'] = sessions

act_status_urls = []
for index, row in datasource.iterrows():
    if pd.isna(row['act_status_url']):
        act_status_urls.append(row['act_html_url'] + '/status')
    else:
        act_status_urls.append(row['act_status_url'])

datasource['act_status_url'] = act_status_urls

years = []
for session in datasource['session']:
    if 122 <= int(session) <= 135:
        # Calculate the year range based on the session number
        start_year = 1997 + (int(session) - 122) * 2
        end_year = start_year + 1
        year_range = f"{start_year}-{end_year}"
        years.append(year_range)
    else:
        years.append("Unknown Session")
datasource['year'] =  years


state = 'OH'
# Read the CSV file into a DataFrame
datasource = pd.read_csv(f'/Volumes/SSD/AFRI/Data/Cleaned/{state}_leginfo_clean.csv', index_col=0)

status_file_urls = []

for link in datasource['link']:
    print(link)
    if 'archives' in link:
        status_url = 'http://archives.legislature.state.oh.us/lsc/statusreport' + link.split('ID=')[1][:3] + '/' +  link.split('ID=')[1] + '_statusreport.pdf'
    else:
        status_url = link + '/votes'
    status_file_urls.append(status_url)
year_info = []
for url in status_file_urls:
    if 'archives' in url:
        print(url)
        driver.execute_script(f"window.location.href = '{url}'")
        time.sleep(1)
        year_info.append('')
    else:
        driver.get('https://www.legislature.ohio.gov/legislation/134/hb575/votes')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'body'))
        )
        year_info.append(driver.find_element(By.CSS_SELECTOR, 'body').text)

link = 'http://archives.legislature.state.oh.us/bills.cfm?ID=130_SB_378'
'https://www.legislature.ohio.gov/legislation/134/hb687/status'

#http://archives.legislature.state.oh.us/lsc/statusreport125/125_HB_1_statusreport.pdf
#http://archives.legislature.state.oh.us/lsc/statusreport122/122_HB_1_statusreport.pdf
#http://archives.legislature.state.oh.us/lsc/statusreport
#http://archives.legislature.state.oh.us/bills.cfm?ID=122_HB_1
#https://www.legislature.ohio.gov/legislation/134/hb687statusreport.pdf
#https://www.legislature.ohio.gov/legislation/134/hb575/votes

files = glob.glob(os.path.join('/Users/long/Downloads', '*.pdf'))

status_info = []

for file in files:
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        acttxt = ' '.join(pgtxts)
    status_info.append(acttxt)
    doc.close()

df_status_info = pd.DataFrame({
    'file_name': files,
    'status': status_info
})

df_status_info['file_name'] = df_status_info['file_name'].str.replace('/Users/long/Downloads/', '')


file_names = []
for url in status_file_urls:
    try:
        file_names.append(url.split('/statusreport')[1][4:])
    except:
        file_names.append(url)

datasource['file_name'] = file_names

datasource = datasource.merge(df_status_info, on='file_name', how='left')

datasource.drop(['status_x'], axis=1, inplace=True)



datasource.to_excel('OH_Leginfo.xlsx')
datasource.to_csv('OH_Leginfo.csv')
datasource.to_pickle('OH_Leginfo.pkl')
datasource.to_json('OH_Leginfo.json')

datasource.to_csv("OH_leginfo_clean.csv")

datasource = pd.read_csv('OH_Leginfo.csv')

datasource.drop()

datasource.drop('Unnamed: 0.2', axis=1, inplace=True)

for index, row in datasource.iterrows():
    if 'pdf' not in row['file_name']:
        print(row['file_name'])
        driver.get(row['file_name'])


status_df = pd.read_excel('status-report-of-legislation.xlsx')


datasource['key'] = datasource['link'].str.replace('https://www.legislature.ohio.gov/legislation/', '').str.replace('/','')

datasource = datasource.merge(status_df, on='key', how='left')


years = []

for status in datasource['status_y']:
    #print(status)
    try:
        year = status.split('Consideration')[1].replace('A','').replace('*','').replace(' ','')[:10].replace('\n','')[-2:]
        if int(year) > 30:
            years.append(int('19' + year))
        else:
            years.append(int('20' + year))
        print(year)
    except:
        years.append(status)

datasource['year'] = datasource['year'].astype('int')


datasource['year'] = years

datasource.loc[datasource['year'].isna(), 'year'] = datasource['introduced']


nan_df = datasource[datasource['year'].isna()]

datasource['year'][492] = '1999'
datasource['year'][493] = '1999'
datasource['year'][566] = '2001'

datasource = datasource.iloc[:, :-3]

for row in datasource[:10].iterrows():
    #print(row[1]['key'])
    try:
        row[1]('introduced') == row[1]('status_y').split('Consideration')[1].replace('A','').replace('*','').replace(' ','')[:10].replace(' ','')[-3:].replace('/','')
    except:
        print(row[1]('status_y')


        status_df['key'] = status_df['session'].astype(str) + status_df['session'].str.replace('.','').lower().astype(str) + status_df['Number'].astype(str)
        # End of OH_Leginfo.py


        # Start of ND_Leginfo.py

        # import libraries
        import pandas as pd
        import os
        import re
        from nltk.corpus import stopwords
        from nltk.tokenize import word_tokenize
        from word_forms.word_forms import get_word_forms
        import datefinder
        from dateutil import parser
        from selenium import webdriver
        from selenium.webdriver.firefox.service import Service
        from selenium.webdriver.firefox.options import Options
        from selenium.webdriver.common.by import By
        from selenium.webdriver.support.ui import WebDriverWait
        from selenium.webdriver.support import expected_conditions as EC
        from selenium.webdriver.common.keys import Keys
        from selenium.webdriver.chrome.options import Options
        from selenium.common.exceptions import NoSuchElementException
        from selenium.common.exceptions import TimeoutException
        from selenium.webdriver.support.ui import Select
        ## others
        from time import sleep
        import pandas as pd
        import datefinder
        import calendar
        import os
        import unittest
        from random import randint
        import PyPDF2
        import glob
        import pickle
        import inflect
        import time
        import requests
        from bs4 import BeautifulSoup
        import pandas as pd
        import matplotlib.pyplot as plt
        import fitz


        # Set the working directory
        os.chdir('/Volumes/SSD/AFRI/Data/')

        # Verify the current working directory
        print(os.getcwd())


        urls=[]
        sessionurls = []
        driver.get("https://ndlegis.gov/session-laws/")
        WebDriverWait(driver, 300).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.view-content ul li a"))
        )
        sessions = driver.find_elements_by_css_selector('div.view-content ul li a')
        for index,session in enumerate(sessions):
            print(index)
        print(session.text)
        sessionurl = session.get_attribute('href')
        sessionurls.append(sessionurl)

        for sessionurl in sessionurls[:33]:
            driver.get(sessionurl)
        sleep(1)
        driver.find_element_by_link_text("Chapter Categories").click()
        try:
            chapters = driver.find_elements_by_css_selector('div.field-items ul li a')
            for chapter in chapters:
                url = chapter.get_attribute('href')
                urls.append(url)
        except:
            print(sessionurl)
print("urls are done")

acttxts = []
for idx,url in enumerate(urls):
    driver.get(url)
    sleep(1)
    path = "C:/Users/longy/OneDrive/Projects/AFRI/data/ND/"
    list_of_files = glob.glob(os.path.join(path, '*.pdf'))
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    with open(os.path.join(os.getcwd(), latest_file), 'r') as f:  # open in readonly mode
        # creating a pdf File object of original pdf
        pdfFileObj = open(latest_file, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        pagenumber = pdfReader.numPages
        acttxt = []
        pgtxts = []
        for p in range(pagenumber):
            pageObj = pdfReader.getPage(p)
            pgtxt = pageObj.extractText()
            pgtxts.append(pgtxt)
            acttxt = '\n'.join(pgtxts)
    acttxts.append(acttxt)
path = "C:/Users/longy/OneDrive/Projects/AFRI/data/ND/"
list_of_files = glob.glob(os.path.join(path, '*.pdf'))
#latest_file = max(list_of_files, key=os.path.getctime)
acttxts = []
for idx,file in enumerate(list_of_files):
    with open(os.path.join(os.getcwd(), file), 'r') as f:  # open in readonly mode
        # creating a pdf File object of original pdf
        pdfFileObj = open(file, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        pagenumber = pdfReader.numPages
        acttxt = []
        pgtxts = []
        for p in range(pagenumber):
            pageObj = pdfReader.getPage(p)
            pgtxt = pageObj.extractText()
            pgtxts.append(pgtxt)
            acttxt = '  '.join(pgtxts)
    acttxts.append(acttxt)
datasource = pd.DataFrame({
    'Link to full text':urls,
    'Full text': acttxts
})

datasource = pd.DataFrame({
    'Full text': acttxts
})
datasource.to_excel('ND_Leginfo.xlsx')
datasource.to_csv('ND_Leginfo.csv')
datasource.to_pickle('ND_Leginfo.pkl')
datasource.to_json('ND_Leginfo.json')




with open('ND_Leginfo.pkl', 'rb') as k:
    data = pickle.load(k)




import subprocess

def get_mdls_where_from(file_path):
    try:
        result = subprocess.run(['mdls', '-name', 'kMDItemWhereFroms', '-raw', file_path],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            where_from_info = result.stdout.strip()
            if where_from_info:
                # The output might be a JSON-like list, so we print it directly
                return where_from_info
            else:
                print("No 'Where from' information available.")
                return None
        else:
            print(f"Error running mdls command: {result.stderr}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None


acttxts = []
urls = []
path = "Raw_Data/ND/PDFs"
files = glob.glob(os.path.join(path, '*.pdf'))
for file in files:
    print(file)
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        acttxt = ' '.join(pgtxts)
    acttxts.append(acttxt)
    doc.close()

datasource = pd.DataFrame({
    'original_text': acttxts,
})

datasource.to_csv('ND_Leginfo_AAEA.csv')

datasource = pd.read_csv('ND_Leginfo_AAEA.csv')


# New list to hold the split strings
acttxts = []

# Loop through each string in the example list
for file in datasource['original_text']:
    # Split the string by 'Filed'
    split_parts = file.split('\nFiled')
    # Extend the new list with the resulting parts
    acttxts.extend(split_parts)

# Print the new list
print(new_list)


acttxts


df = pd.DataFrame({
    'original_text': acttxts,
})

df['original_text'][0]

df['year'] = df['original_text'].apply(lambda x: x[-10:].replace('\n','').replace(' ','')[-4:])

df = df[df['original_text'].str.len() > 100]

df['year'] = pd.to_numeric(df['year'], errors='coerce')

df = df.dropna(subset=['year'])

df['year'] = df['year'].astype(int)

df = df[(df['year'] >= 1975) & (df['year'] <= 2021)]

df.to_csv("Raw_Data/Cleaned/ND_leginfo_clean.csv")

df.to_csv("Raw_Data/ND/ND_leginfo_clean.csv")


# End of ND_Leginfo.py


# Start of AK_Leginfo.py

# import libraries
## selenium
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
#import fitz

import time
import requests
from bs4 import BeautifulSoup

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/'
print(os.path.abspath(working_directory))

# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Create a Firefox profile and configure preferences
options = Options()


# Initialize the Firefox driver with the specified options
executable_path='./geckodriver'
print(os.path.abspath(executable_path))

# Initialize the WebDriver with the specified path
service = Service(executable_path)
driver = webdriver.Firefox(service=service)


years = []
chapter_nums = []
subtitles = []
bill_nums = []
filenames = []
act_urls = []
sessions = []
broken_act_urls = []

driver.get('https://www.arkleg.state.ar.us/Acts/SearchByRange?ddBienniumSession')
# accept cookies button
try:
    driver.find_element(By.CSS_SELECTOR, 'a.btn').click()
except:
    pass

session_elements = driver.find_elements(By.CSS_SELECTOR, '#ddBienniumSessionActsSearchByRange option')
session_texts = []
for session_text in session_elements:
    session_texts.append(session_text.text)

for session_text in session_texts[1:]:
    print(session_text)
    try:
        # Attempt to convert the value to an integer
        year = int(session_text.replace(' ','').split('-')[0])
        if year > 2021:
            pass
        else:
            select = Select(driver.find_element(By.CSS_SELECTOR, '#ddBienniumSessionActsSearchByRange'))
            select.select_by_visible_text(session_text)
            # clear starting act number and ending act number
            starting_num = driver.find_element(By.CSS_SELECTOR, '#startAct')
            starting_num.clear()
            starting_num.send_keys(1)
            ending_num = driver.find_element(By.CSS_SELECTOR, '#endAct')
            ending_num.clear()
            ending_num.send_keys(9999)
            # startAct
            search_button = driver.find_element(By.CSS_SELECTOR, 'div.form-row:nth-child(5) > button:nth-child(1)')
            search_button.click()

            time.sleep(3)

            # for page 1
            rows = driver.find_elements(By.CSS_SELECTOR, 'div.row.tableRow, div.row.tableRowAlt')
            for row in rows:
                #years.append(year)
                #sessions.append(session_text)
                chapter_num = row.find_element(By.CSS_SELECTOR, ':nth-child(1)').text
                chapter_nums.append(chapter_num)
                subtitles.append(row.text)
                bill_nums.append(row.find_element(By.CSS_SELECTOR, ':nth-child(3)').text)
                act_url = row.find_element(By.LINK_TEXT, 'PDF').get_attribute('href')
                act_urls.append(act_url)

                name = str(year) + str(session_text.split(',')[0].replace(' ', '').split('-')[1]) + "chap" + str(
                    chapter_num) + ".pdf"
                filenames.append(name)

                path = download_dir + name
                print(os.path.abspath(path))

                time.sleep(randint(1, 100) * 0.005)

                response = requests.get(act_url)
                # Ensure the request was successful
                if response.status_code == 200:
                    # Write the content of the response to a file
                    with open(path, 'wb') as file:
                        file.write(response.content)
                else:
                    print("Failed to fetch PDF. Status code:", response.status_code)
                    broken_act_urls.append(act_url)
            time.sleep(1)
            # for page 2-n
            page_num = int(driver.find_element(By.CSS_SELECTOR,'div.row:nth-child(25) > div:nth-child(1)').text.split(':')[0].split('of ')[1])
            for num in range(2,page_num+1):
                driver.find_element(By.LINK_TEXT,str(num)).click()
                time.sleep(1)
                rows = driver.find_elements(By.CSS_SELECTOR, 'div.row.tableRow, div.row.tableRowAlt')
                for row in rows:
                    years.append(year)
                    sessions.append(session_text)
                    chapter_num = row.find_element(By.CSS_SELECTOR, ':nth-child(1)').text
                    chapter_nums.append(chapter_num)
                    subtitles.append(row.find_element(By.CSS_SELECTOR, ':nth-child(2)').text)
                    bill_nums.append(row.find_element(By.CSS_SELECTOR, ':nth-child(3)').text)
                    act_url = row.find_element(By.LINK_TEXT, 'PDF').get_attribute('href')
                    act_urls.append(act_url)

                    name = str(year) + str(session_text.split(',')[0].replace(' ','').split('-')[1]) + "chap" + str(chapter_num) + ".pdf"
                    filenames.append(name)

                    path = download_dir + name
                    print(os.path.abspath(path))

                    time.sleep(randint(1, 100) * 0.005)

                    response = requests.get(act_url)
                    # Ensure the request was successful
                    if response.status_code == 200:
                        # Write the content of the response to a file
                        with open(path, 'wb') as file:
                            file.write(response.content)
                    else:
                        print("Failed to fetch PDF. Status code:", response.status_code)
                        broken_act_urls.append(act_url)
    except:
        pass
acttxts = []
pgtxts = []
path = "/Users/long/Downloads/AR/"
files = glob.glob(os.path.join(path, '*.pdf'))
for file in files[:20]:
    pdfFileObj = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    numpages = pdfReader.numPages
    for i in range(numpages):
        pgtxt = []
        pgtxt = pdfReader.getPage(i)
        acttxt = ' '.join(pgtxts)
    print(acttxt)
    acttxts.append(acttxt)
    pdfFileObj.close()


act_txts = []
path = "./PDFs"
files = glob.glob(os.path.join(path, '*.pdf'))
for file in files:
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        act_txt = ' '.join(pgtxts)
    act_txts.append(act_txt)
    doc.close()


datasource = pd.DataFrame({
    'original_text': act_txts,
    'year':years,
    'chapter_num':chapter_nums,
    'subtitle':subtitles,
    'bill_num':bill_nums,
    'filename':filenames,
    'act_url':act_urls,
    'session':sessions
})

# save bill info into files
datasource.to_excel('AR_Leginfo.xlsx')
datasource.to_csv('AR_Leginfo.csv')
datasource.to_pickle('AR_Leginfo.pkl')
datasource.to_json('AR_Leginfo.json')
print("done")
# End of AK_Leginfo.py


# Start of DC_Leginfo.py

# %%
# import libraries
## selenium
import selenium

print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
## others
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import time
import requests
import fitz
import re
from bs4 import BeautifulSoup

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/DC'
print(os.path.abspath(working_directory))

# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Initialize the Firefox driver with the specified options
executable_path = '../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service=service)

# %%

years = []
chapter_nums = []
subtitles = []
bill_nums = []
filenames = []
act_urls = []
sessions = []
broken_act_urls = []
act_txts = []
expired_dates = []

# go the website and select Bill under the Legislation Category; enated, expired, Official law under the status, click download
# https://lims.dccouncil.gov/searchresult/currentPage=1&documentSearch=false&refinBy[0]=statusId%3A%20Expired&refinBy[1]=statusId%3A%20Enacted&refinBy[2]=statusId%3A%20Official%20Law&refinBy[3]=legislationCategoryId%3A%20Bill&searchString=
# download '⁦/Volumes/SSD/AFRI/Data/Raw_Data/DC/PDFs⁩'


driver.get('https://lims.dccouncil.gov/searchresult/documentSearch=false&searchString=')

#More under Legislation Category
driver.find_element(By.CSS_SELECTOR, 'div.widget:nth-child(2) > div:nth-child(3) > a:nth-child(1) > span:nth-child(1)').click()
# select bill
driver.find_element(By.CSS_SELECTOR, 'div.widget:nth-child(2) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(2) > div:nth-child(1) > label:nth-child(2)').click()

#More under Status
driver.find_element(By.CSS_SELECTOR, 'div.widget:nth-child(4) > div:nth-child(3) > a:nth-child(1) > span:nth-child(1)').click()
# select Enacted
driver.find_element(By.CSS_SELECTOR, 'div.widget:nth-child(4) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > div:nth-child(1) > label:nth-child(2)').click()
# select Expired
driver.find_element(By.CSS_SELECTOR, 'div.widget:nth-child(4) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(2) > div:nth-child(1) > label:nth-child(2)').click()
# select Official Law
driver.find_element(By.CSS_SELECTOR, 'div.widget:nth-child(4) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(4) > div:nth-child(1) > label:nth-child(2)').click()

# more period
driver.find_element(By.CSS_SELECTOR, 'div.widget:nth-child(1) > div:nth-child(3) > a:nth-child(1) > span:nth-child(1)').click()

# select period and download

period_elements = driver.find_elements(By.CSS_SELECTOR,'div.widget:nth-child(1) > div:nth-child(2) > ul:nth-child(1) > li > div:nth-child(1) > label:nth-child(2)')
for element in period_elements:

    element.click()

    driver.find_element(By.LINK_TEXT, 'Download').click()

    time.sleep(30)

    element.click()

# Initialize an empty DataFrame to hold all appended data
datasource = pd.DataFrame()

# Iterate over all files in the specified directory
for file in os.listdir(download_dir):

    print(file)

    # Construct the full file path
    file_path = os.path.join(download_dir, file)

    # Check if the file is an Excel file
    if file.endswith('.xlsx'):
        try:
            # Read the Excel file into a DataFrame
            file_df = pd.read_excel(file_path, engine='openpyxl')

            # Append the DataFrame to the master DataFrame
            datasource = pd.concat([df, file_df], ignore_index=True)
        except:
            print('not working')

datasource = datasource.drop_duplicates()

datasource['year'] = datasource['Introduction Date'].str[-4:]

datasource.sort_values(by='year', inplace=True)

#datasource = datasource[pd.notna(datasource['Act/Res Number'])]

# Reset the index
datasource.reset_index(drop=True, inplace=True)

for index, row in datasource.iterrows():
    print(index)
    url = 'https://lims.dccouncil.gov/Legislation/' + row['Legislation Number']
    act_urls.append(url)
    try:
        broken_act_urls.append('NA')
        driver.get(url)
        time.sleep(randint(1,5)*0.01)

        session_elements = WebDriverWait(driver, 60).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'body'))
        )

        try:
            driver.find_element(By.LINK_TEXT, 'Enrollment').click()
        except NoSuchElementException:
            print("Enrollment link not found, trying Introduction link...")
            try:
                driver.find_element(By.PARTIAL_LINK_TEXT, 'View Introduction').click()
            except NoSuchElementException:
                print("Introduction link not found, trying download icon...")
                try:
                    # Assuming 'i.download.icon' is a CSS class, not link text
                    driver.find_element(By.CSS_SELECTOR, 'i.download.icon').click()
                except NoSuchElementException:
                    print("Download icon not found. No action could be taken.")
                    try:
                        # Assuming 'i.download.icon' is a CSS class, not link text
                        driver.find_element(By.CSS_SELECTOR, '.download').click()
                    except NoSuchElementException:
                        print("Download icon not found. No action could be taken.")
    except:
        broken_act_urls.append(url)


df = pd.read_csv('/Volumes/SSD/AFRI/Data/Raw_Data/DC/DC_Leginfo_broken_urls.csv')

df = df.drop_duplicates()

broken_act_urls = []

for url in df['urls']:
    print(url)
    try:
        driver.get(url)

        broken_act_urls.append('NA')

        time.sleep(randint(1,5)*0.01)

        session_elements = WebDriverWait(driver, 60).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'body'))
        )

        try:
            driver.find_element(By.LINK_TEXT, 'Enrollment').click()
        except NoSuchElementException:
            print("Enrollment link not found, trying Introduction link...")
            try:
                driver.find_element(By.PARTIAL_LINK_TEXT, 'View Introduction').click()
            except NoSuchElementException:
                print("Introduction link not found, trying download icon...")
                try:
                    # Assuming 'i.download.icon' is a CSS class, not link text
                    driver.find_element(By.CSS_SELECTOR, 'i.download.icon').click()
                except NoSuchElementException:
                    print("Download icon not found. No action could be taken.")
                    try:
                        # Assuming 'i.download.icon' is a CSS class, not link text
                        driver.find_element(By.CSS_SELECTOR, '.download').click()
                    except NoSuchElementException:
                        print("Download icon not found. No action could be taken.")
    except:
        broken_act_urls.append(url)



act_txts = []

path = "/Volumes/SSD/AFRI/Data/Raw_Data/DC/PDFs"
files = glob.glob(os.path.join(path, '*.pdf'))

for file in files:
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        act_txt = ' '.join(pgtxts)
    act_txts.append(act_txt)
    doc.close()

file_df = pd.DataFrame({
    'file_name': files,
    'original_text': act_txts
})


bill_nums = []

for name in file_df['file_name']:
    result = re.search(r'B\d+-\d+', name)
    if result:
        extracted_text = result.group(0)
        print(extracted_text)
        bill_nums.append(extracted_text)
    else:
        bill_nums.append('NA')

file_df['bill_name'] = bill_nums

datasource = pd.read_csv('/Volumes/SSD/AFRI/Data/Raw_Data/DC/DC_Leginfo.csv', index_col=0)

datasource = pd.merge(datasource, file_df, left_on='Legislation Number', right_on='bill_name', how='left')

# Drop rows where 'file_name' is NaN
datasource.dropna(subset=['file_name'], inplace=True)

#  reset the index after dropping rows
datasource.reset_index(drop=True, inplace=True)

df.drop(columns=['bill_name','Status', 'Legislation Number'], inplace=True)

df.rename(columns={'Act/Res Number': 'act_num'}, inplace=True)



datasource.to_excel('DC_Leginfo.xlsx')
datasource.to_csv('DC_Leginfo.csv')
datasource.to_pickle('DC_Leginfo.pkl')
datasource.to_json('DC_Leginfo.json')


# End of DC_Leginfo.py


# Start of MN_Leginfo.py

# import libraries
## selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
## others
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import fitz

import time
import requests
from bs4 import BeautifulSoup

# set up working directory
working_directory = '/Volumes/SSD/AFRI/Data/Raw_Data/MN'
print(os.path.abspath(working_directory))

# Change the working directory
os.chdir(working_directory)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Specify the desired download directory
download_dir = "./PDFs/"
print(os.path.abspath(download_dir))

# Create a Firefox profile and configure preferences
options = Options()


# Initialize the Firefox driver with the specified options
executable_path='../../geckodriver'
print(os.path.abspath(executable_path))

service = Service(executable_path)
driver = webdriver.Firefox(service = service)

# session years: 1975-2021

file_names = []
act_urls = []
bill_nums = []
chapter_nums = []
years = []
sessions = []
broken_act_urls = []
act_txts = []

for index, year in enumerate(range(1975, 2022)):

    print(year)

    for session_num in range(0,3):
        # there might be extra two sessions in one year.
        url = "https://www.revisor.mn.gov/laws/" + str(year) + "/"+str(session_num)+"/"

        print(url)

        if session_num == 0:
            session = "rs"
            #"regular_session"
        elif session_num == 1:
            session = "fs"
            #"1st special session"
        else:
            session = "ss"
            #"2nd special session"
        print(session)

        try:
            # Open the website in Chrome
            driver.get(url)
            time.sleep(5)
            try:
                # Wait for the page to load and find all the chapter links
                chapter_elements = WebDriverWait(driver, 60).until(
                    EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, "Chapter"))
                )

                chapter_urls = []

                for element in chapter_elements:
                    bill_nums.append(element.find_element(By.XPATH,'../following-sibling::*').text)
                    chapter_nums.append(element.find_element(By.XPATH,'..').text.replace(" ",""))
                    chapter_urls.append(element.get_attribute("href"))

                for index,chapter_url in enumerate(chapter_urls, start=1):

                    print(index)

                    print(chapter_url)

                    years.append(year)

                    sessions.append(session)

                    if year < 1983:
                        act_url = chapter_url
                        act_txts.append("")
                    else:
                        driver.get(chapter_url)

                        WebDriverWait(driver, 60).until(
                            EC.presence_of_element_located(
                                (By.LINK_TEXT,
                                 "PDF"))
                        )
                        # If accessing the URL directly fails, get the URL from the link with text "PDF" and navigate to it
                        pdf_link = driver.find_element(By.LINK_TEXT, "PDF")
                        pdf_url = pdf_link.get_attribute("href")
                        act_url = pdf_url
                        #act_txts.append(driver.find_element(By.CSS_SELECTOR,".xtend > pre:nth-child(1)").text)

                    act_urls.append(act_url)

                    name = str(year) + str(session) + "chap" + str(chapter_url.split("Chapter/")[1].replace("/",""))+ ".pdf"
                    file_names.append(name)

                    path = download_dir + name
                    print(os.path.abspath(path))

                    time.sleep(randint(1, 100) * 0.005)

                    response = requests.get(act_url)
                    # Ensure the request was successful
                    if response.status_code == 200:
                        # Write the content of the response to a file
                        with open(path, 'wb') as file:
                            file.write(response.content)
                    else:
                        print("Failed to fetch PDF. Status code:", response.status_code)
                        broken_act_urls.append(act_url)
            except TimeoutException:
                print("Failed to retrieve website content.")
        except:
            pass


act_txts = []

folder_path = "./PDFs"
print(os.path.abspath(folder_path))


files = glob.glob(os.path.join(folder_path, '*'))

broken_files = []

for file in files:
    try:
        doc = fitz.open(file)
        pgtxts = []
        for page in doc:
            pgtxt = page.get_text()
            pgtxts.append(pgtxt)
            acttxt = ' '.join(pgtxts)
        act_txts.append(acttxt)
        doc.close()
    except:
        broken_files.append(file)

file_names = []

for file in files:
    file_names.append(file.split("./PDFs/")[1])


datasource =  pd.DataFrame({
    'original_text': act_txts,
    'bill_num': bill_nums,
    'chapter_num': chapter_nums,
    'year':years,
    'session':sessions,
    'link': act_urls,
    'state': "MN",
    'filename': file_names
})

datasource = datasource.drop_duplicates(subset='link', keep='first')

datasource.to_excel('MN_Leginfo.xlsx')
datasource.to_csv('MN_Leginfo.csv')
datasource.to_pickle('MN_Leginfo.pkl')
datasource.to_json('MN_Leginfo.json')

# End of MN_Leginfo.py


# Start of GA_Leginfo.py

# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import pytest
import unittest
from random import randint
import PyPDF2
import glob


# driver_path = 'C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe' #for windows
# driver_path = '/Users/Yanxu/OneDrive/Projects/AgPolicyCoding/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Change the working directory
os.chdir('/Users/long/OneDrive/Projects/AFRI')
# os.chdir('/Volumes/GoogleDrive/Shared drives/UCSC-UMN AFRI project/Data/GA')
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

chromeOptions = webdriver.ChromeOptions()
dnldpath = {"download.default_directory" : "/Users/long/OneDrive/Projects/AFRI/data/GA/"}
chromeOptions.add_experimental_option("prefs",dnldpath)
driver = webdriver.Chrome(executable_path='./chromedriver', options=chromeOptions)
driver.get("https://www.legis.ga.gov/legislation/signed-by-governor")
yrs = []
sts = []
seyrs = []
titles = []
bfsums = []
introdts = []
sigdts = []
effdts = []
epddtes = []
introducers = []
txtlks = []
acttxts = []
stas = []
hoscoms = []  # house committee
sponsors = []

sessions =[]
urls = []
billnums = []

WebDriverWait(driver, 300).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#session > option"))
)
session_list = driver.find_elements_by_css_selector('#session > option')

for index, i in enumerate(session_list):
    WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#session > option"))
    )
    session_list = driver.find_elements_by_css_selector('#session > option')
    session_list[index].click()
    session = session_list[index].text
    print(session)

    search_btn = driver.find_element_by_css_selector('button.btn.btn-primary')
    search_btn.click()

    sleep(2)

    try:
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.noResults'))
        )
        driver.find_element_by_css_selector('div.noResults')
        print("No results found")
    except:
        WebDriverWait(driver, 300).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.resultCount b:nth-child(2)"))
        )
        total_bills = driver.find_element_by_css_selector('div.resultCount b:nth-child(2)').text

        # total_bills = 40
        if int(total_bills) % 20 == 0:
            total_pages = int(int(total_bills)/20)
        else:
            total_pages = int(int(total_bills)/20)+1
        print(total_pages)

        # temporary setup for developing
        #total_pages = 6

        for page in range(total_pages):
            WebDriverWait(driver, 300).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".text-nowrap"))
            )
            bill_list = driver.find_elements_by_css_selector('.text-nowrap')
            for bill in bill_list:
                url = bill.get_attribute('href')
                urls.append(url)
                billnum = bill.text
                billnums.append(billnum)
                sessions.append(session)
            print("page",page+1)

            if total_pages > 0:
                WebDriverWait(driver, 300).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "ul > li:last-child > a > span"))
                )
                next_page = driver.find_element_by_css_selector('ul > li:last-child > a > span')
                try:
                    next_page.click()
                    sleep(1)
                except:
                    print("this session is done for urls")
            else:
                print("this session is done for urls")

for idx,url in enumerate(urls):
    driver.get(url)
    sleep(1)
    WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.card div.card-body"))
    )
    try:
        bfsum= driver.find_elements_by_css_selector('h2.card-title + div.card-text-sm')[0].text
        #Briefsummary = driver.find_elements_by_css_selector('div.card div.card-body')[1].text
    except:
        bfsum = ''
    bfsums.append(bfsum)

    title = driver.find_element_by_css_selector("div.h1.subHead").text
    titles.append(title)

    WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "app-status-history-list div  table tbody"))
    )
    sleep(1)
    ## Status History Dates
    try:
        dates = driver.find_element_by_css_selector('app-status-history-list div  table tbody').text

        if "Senate Date Signed by Governor" in dates:
            sigdt = dates.split("Senate Date Signed by Governor")[0].split("\n")[-1].replace(" ", "")
        else:
            sigdt = dates.split("House Date Signed by Governor")[0].split("\n")[-1].replace(" ", "")
            sigdts.append(sigdt)

        effdt = dates.split(" Effective Date")[0]
        effdts.append(effdt)

        introdt = dates.split("\n")[-1].split(" ")[0]
        introdts.append(introdt)

        yr = effdt.split("/")[2]
        yrs.append(yr)

        '''
        # if (int(yr) % 2) == 0:
            seyr = str(int(yr) - 1) + "-" + str(yr)
        else:
            seyr = str(yr) + str("-") + str(int(yr) + 1)
        '''
    except:
        effdt =''
        effdts.append(effdt)
        sigdt =''
        sigdts.append(sigdt)
        introdt =''
        introdts.append(introdt)
        yr =''
        yrs.append(yr)

        '''
        seyr =''
        seyrs.append(seyr)
        '''

    WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li.list-inline-item"))
    )
    version_btn =driver.find_elements_by_css_selector('li.list-inline-item')
    version_btn[0].click()

    WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.col-6.justify-content-end.text-right > button"))
    )
    dnldbtn = driver.find_element_by_class_name('btn.btn-outline-primary.btn-sm.mr-2.mb-1')
    dnldbtn.click()

    sleep(randint(1, 2))
    path = "/Users/long/OneDrive/Projects/AFRI/data/GA/"
    list_of_files = glob.glob(os.path.join(path, '*.pdf'))
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    with open(os.path.join(os.getcwd(), latest_file), 'r') as f:  # open in readonly mode
        # creating a pdf File object of original pdf
        pdfFileObj = open(latest_file, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        pagenumber =  pdfReader.numPages
        acttxt =[]
        pgtxts = []
        for p in range(pagenumber):
            pageObj = pdfReader.getPage(p)
            pgtxt = pageObj.extractText()
            pgtxts.append(pgtxt)
            acttxt = '\n'.join(pgtxts)
    acttxts.append(acttxt)
    st ="GA"
    sts.append(st)


    print(idx+1)
    # sleep(randint(1, 2))

print("Downloading is finished")


#'sponsors': sponsors
#'HouseCommittees': HouseCommittees,
datasource = pd.DataFrame({
    'Session Year': sessions,
    'Bill Number': billnums,
    'Link to full text': urls
})


datasource = pd.DataFrame({
    'Year': yrs,
    # 'State': sts,
    'Session Year': sessions,
    'Bill Number': billnums,
    'Title': titles,
    'Brief Summary': bfsums,
    # 'Introduced Date': introdts,
    #'Date it was signed':sigdts,
    # 'Date effective':effdts,
    # 'Full Text':acttxts,
    'Link to full text': urls
})

#'House Committee': hoscoms,
# drop duplicates
#datasource.drop_duplicates(subset = ['Bill Number'],
# keep = 'first', inplace = True, ignore_index= True)

# save bill info into excel file
datasource.to_excel('GA_Leginfo.xlsx')
print("Web-scrapting finished")

# End of GA_Leginfo.py

